# OVERVIEW
Using a 2018 NYPD stop, question, and frisk dataset, the goal of this analysis is to use machine learning to predict whether someone would be arrested by an NYPD officer after being stopped by one. The models are supposed to, in a sense, mimic the thought process of the NYPD as a whole before determining whether an arrest is made.

It is important to note that the goal here is not to verify the justness or legitimacy of arrests. The analysis thus includes features that are highly correlated with arrests and some that may even play on stereotypes or racial biases commonly held against certain people groups.

In [1]:
import pandas as pd
import numpy as np

import pickle
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 200)
data = pd.read_csv('../sqf-2018.csv', low_memory=False)
data.head(10)

,STOP_FRISK_ID,STOP_FRISK_DATE,Stop Frisk Time,YEAR2,MONTH2,DAY2,STOP_WAS_INITIATED,RECORD_STATUS_CODE,ISSUING_OFFICER_RANK,ISSUING_OFFICER_COMMAND_CODE,SUPERVISING_OFFICER_RANK,SUPERVISING_OFFICER_COMMAND_CODE,SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED,LOCATION_IN_OUT_CODE,JURISDICTION_CODE,JURISDICTION_DESCRIPTION,OBSERVED_DURATION_MINUTES,SUSPECTED_CRIME_DESCRIPTION,STOP_DURATION_MINUTES,OFFICER_EXPLAINED_STOP_FLAG,OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION,OTHER_PERSON_STOPPED_FLAG,SUSPECT_ARRESTED_FLAG,SUSPECT_ARREST_OFFENSE,SUMMONS_ISSUED_FLAG,SUMMONS_OFFENSE_DESCRIPTION,OFFICER_IN_UNIFORM_FLAG,ID_CARD_IDENTIFIES_OFFICER_FLAG,SHIELD_IDENTIFIES_OFFICER_FLAG,VERBAL_IDENTIFIES_OFFICER_FLAG,FRISKED_FLAG,SEARCHED_FLAG,OTHER_CONTRABAND_FLAG,FIREARM_FLAG,KNIFE_CUTTER_FLAG,OTHER_WEAPON_FLAG,WEAPON_FOUND_FLAG,PHYSICAL_FORCE_CEW_FLAG,PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG,PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG,PHYSICAL_FORCE_OC_SPRAY_USED_FLAG,PHYSICAL_FORCE_OTHER_FLAG,PHYSICAL_FORCE_RESTRAINT_USED_FLAG,PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG,PHYSICAL_FORCE_WEAPON_IMPACT_FLAG,BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG,BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG,SUSPECTS_ACTIONS_CASING_FLAG,SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG,SUSPECTS_ACTIONS_DECRIPTION_FLAG,SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG,SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG,SUSPECTS_ACTIONS_LOOKOUT_FLAG,SUSPECTS_ACTIONS_OTHER_FLAG,SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG,SEARCH_BASIS_ADMISSION_FLAG,SEARCH_BASIS_CONSENT_FLAG,SEARCH_BASIS_HARD_OBJECT_FLAG,SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG,SEARCH_BASIS_OTHER_FLAG,SEARCH_BASIS_OUTLINE_FLAG,DEMEANOR_CODE,DEMEANOR_OF_PERSON_STOPPED,SUSPECT_REPORTED_AGE,SUSPECT_SEX,SUSPECT_RACE_DESCRIPTION,SUSPECT_HEIGHT,SUSPECT_WEIGHT,SUSPECT_BODY_BUILD_TYPE,SUSPECT_EYE_COLOR,SUSPECT_HAIR_COLOR,SUSPECT_OTHER_DESCRIPTION,STOP_LOCATION_PRECINCT,STOP_LOCATION_SECTOR_CODE,STOP_LOCATION_APARTMENT,STOP_LOCATION_FULL_ADDRESS,STOP_LOCATION_PREMISES_NAME,STOP_LOCATION_STREET_NAME,STOP_LOCATION_X,STOP_LOCATION_Y,STOP_LOCATION_ZIP_CODE,STOP_LOCATION_PATROL_BORO_NAME,STOP_LOCATION_BORO_NAME
0,1,1/1/2018,19:04:00,2018,January,Monday,Based on C/W on Scene,APP,POM,1,SGT,1,Y,(null),P,PSB,0,MENACING,18,Y,(null),Y,N,(null),N,(null),Y,(null),(null),(null),Y,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),Y,Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),DE,COOPERATIVE,(null),MALE,WHITE,5.1,170,MED,BRO,BLK,(null),1,G,(null),VARICK STREET && FRANKLIN STREET,(null),VARICK STREET,982327,201274,(null),PBMS,MANHATTAN
1,2,1/1/2018,23:00:00,2018,January,Monday,Based on Radio Run,APP,POM,34,SGT,34,N,O,P,PSB,1,CPW,15,Y,(null),N,N,(null),N,(null),Y,(null),(null),(null),Y,Y,N,(null),(null),(null),N,(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),(null),Y,(null),DE,NERVOUS,26,MALE,BLACK HISPANIC,6.1,250,HEA,BRO,BLK,(null),34,C,(null),DYCKMAN STREET && POST AVENUE,(null),DYCKMAN STREET,1004892,253548,(null),PBMN,MANHATTAN
2,3,1/1/2018,23:55:00,2018,January,Monday,Based on Radio Run,APP,POM,808,SGT,808,Y,I,H,Housing,0,GRAND LARCENY,10,Y,(null),N,N,(null),N,(null),Y,(null),(null),(null),Y,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),Y,(null),(null),(null),Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),DE,RELIEVED,40,MALE,WHITE HISPANIC,5.5,150,THN,BRO,BLD,(null),43,B,4M,2245 RANDALL AVENUE,(null),RANDALL AVENUE,1026706,237776,(null),PBBX,BRONX
3,4,1/1/2018,3:23:00,2018,January,Monday,Based on Radio Run,APP,POM,63,SGT,63,Y,O,P,PSB,2,ROBBERY,15,Y,(null),Y,Y,ROBBERY,N,(null),Y,(null),(null),(null),Y,Y,N,(null),(null),(null),N,(null),(null),(null),(null),(null),Y,Y,(null),Y,(null),(null),(null),Y,(null),(null),(null),Y,(null),(null),(null),(null),(null),Y,(null),DE,COMPLAIN,38,MALE,AMERICAN INDIAN/ALASKAN NAT

# FEATURE CLEANING

## Drop columns

In [2]:
data = data.drop(['RECORD_STATUS_CODE', 'ISSUING_OFFICER_COMMAND_CODE', 'YEAR2', 'SUPERVISING_OFFICER_COMMAND_CODE',
                  'SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED', 'LOCATION_IN_OUT_CODE',
                  'JURISDICTION_DESCRIPTION','SUSPECT_ARREST_OFFENSE', 'SUMMONS_ISSUED_FLAG', 'SUMMONS_OFFENSE_DESCRIPTION',
                  'STOP_LOCATION_APARTMENT','STOP_LOCATION_PREMISES_NAME', 'STOP_LOCATION_ZIP_CODE', 'DEMEANOR_CODE',
                 'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION', 'STOP_LOCATION_STREET_NAME', 'OBSERVED_DURATION_MINUTES',
                 'STOP_LOCATION_PRECINCT', 'STOP_LOCATION_FULL_ADDRESS','STOP_LOCATION_SECTOR_CODE',
                 'PHYSICAL_FORCE_RESTRAINT_USED_FLAG','PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG',
                 'PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG'], axis=1)

# FEATURE ENGINEERING

## Duration minutes

In [3]:
# Converting stop durations into categories.
def stop_duration_bin(df):
    label_names = ['Brief', 'Short', 'Medium', 'Long']
    cut_points = [0, 5, 15, 30, 999]
    df["stop_duration_group"] = pd.cut(df['STOP_DURATION_MINUTES'],cut_points,labels=label_names)
    return df

data = stop_duration_bin(data)

## Suspected crime description

In [4]:
# Consolidating crime descriptions into more general categories.
data['weapon'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'CPW') |
                               (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL POSSESSION OF FORGED INSTRUMENT'), 1, 0)

data['endangerment'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'RECKLESS ENDANGERMENT') |
                               (data.SUSPECTED_CRIME_DESCRIPTION == 'MENACING'), 1, 0)

data['physical_harm'] = np.where ( (data.SUSPECTED_CRIME_DESCRIPTION == 'TERRORISM') |
                                 (data.SUSPECTED_CRIME_DESCRIPTION == 'MURDER') | 
                                 (data.SUSPECTED_CRIME_DESCRIPTION == 'ASSAULT'), 1, 0)

data['theft'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'PETIT LARCENY') |
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'GRAND LARCENY') |
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'CPSP') | 
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'THEFT OF SERVICES'), 1, 0)

data['property_damage'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL MISCHIEF') |
                                  (data.SUSPECTED_CRIME_DESCRIPTION == 'MAKING GRAFFITI'), 1, 0)

data['trespass'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'BURGLARY') |
                           (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL TRESPASS'), 1, 0)

data['auto'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'GRAND LARCENY AUTO') |
                       (data.SUSPECTED_CRIME_DESCRIPTION == 'UNAUTHORIZED USE OF A VEHICLE') |
                       (data.SUSPECTED_CRIME_DESCRIPTION == 'AUTO STRIPPIG'), 1, 0)

data['sexual_assault'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'RAPE') | 
                                 (data.SUSPECTED_CRIME_DESCRIPTION == 'FORCIBLE TOUCHING'), 1, 0)

data['drugs'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL POSSESSION OF MARIHUANA') |
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL SALE OF MARIHUANA') |
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL POSSESSION OF CONTROLLED SUBSTANCE') |
                        (data.SUSPECTED_CRIME_DESCRIPTION == 'CRIMINAL SALE OF CONTROLLED SUBSTANCE'), 1, 0)

data['robbery'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'ROBBERY'), 1, 0)

data['other'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'OTHER'), 1, 0)

data['prostitution'] = np.where( (data.SUSPECTED_CRIME_DESCRIPTION == 'PROSTITUTION'), 1, 0)

## Suspect arrested

In [5]:
# Converting to binary feature.
data.loc[data.SUSPECT_ARRESTED_FLAG=='Y', 'SUSPECT_ARRESTED_FLAG']=1
data.loc[data.SUSPECT_ARRESTED_FLAG=='N', 'SUSPECT_ARRESTED_FLAG']=0

## Officer identification

In [6]:
# Converting to binary features.
data.loc[data.VERBAL_IDENTIFIES_OFFICER_FLAG=='V', 'VERBAL_IDENTIFIES_OFFICER_FLAG']=1
data.loc[data.VERBAL_IDENTIFIES_OFFICER_FLAG=='(null)', 'VERBAL_IDENTIFIES_OFFICER_FLAG']=0

data.loc[data.ID_CARD_IDENTIFIES_OFFICER_FLAG=='I', 'ID_CARD_IDENTIFIES_OFFICER_FLAG']=1
data.loc[data.ID_CARD_IDENTIFIES_OFFICER_FLAG=='(null)', 'ID_CARD_IDENTIFIES_OFFICER_FLAG']=0

data.loc[data.SHIELD_IDENTIFIES_OFFICER_FLAG=='S', 'SHIELD_IDENTIFIES_OFFICER_FLAG']=1
data.loc[data.SHIELD_IDENTIFIES_OFFICER_FLAG=='(null)', 'SHIELD_IDENTIFIES_OFFICER_FLAG']=0

## Weapon on suspect

The `WEAPON_FOUND_FLAG` feature is supposed to consolidate the preceding three features, but its total does not equal the combined total of those three. I'm using np.where to consolidate all four.

In [7]:
data['weapon_found'] = np.where( (data.FIREARM_FLAG=='Y') | (data.KNIFE_CUTTER_FLAG=='Y') |
                                (data.OTHER_WEAPON_FLAG=='Y') | (data.WEAPON_FOUND_FLAG=='Y'), 1, 0)

## Physical force

Here, I'm trying to consolidate all uses of police force on the person stopped.

In [8]:
# Converting to binary features.
data['aggressive_force'] = np.where( (data.PHYSICAL_FORCE_CEW_FLAG=='Y') |
                                   (data.PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG=='Y') |
                                   (data.PHYSICAL_FORCE_OC_SPRAY_USED_FLAG=='Y') |
                                   (data.PHYSICAL_FORCE_OTHER_FLAG=='Y') |
                                   (data.PHYSICAL_FORCE_WEAPON_IMPACT_FLAG=='Y'), 1, 0 )

## Suspect background

Background information on the suspect to prime the officer for potential danger.

In [9]:
# Converting to binary features.
backgrounds = ['BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG', 'BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG']

replacements = {'Y':1, '(null)':0}

for bg in backgrounds:
    data[bg] = [replacements[item] for item in data[bg]]

data = data.rename(columns={'BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG' : 'BACKGROUND_SUSPECT_KNOWN_CARRY_WPN'})
data = data.rename(columns={'BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG' : 'BACKGROUND_VIOLENT_CRIME'})

## Suspect actions

Information on whether the suspect was doing anything to strongly imply criminal conduct.

In [10]:
actions = ['SUSPECTS_ACTIONS_CASING_FLAG', 'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG',
          'SUSPECTS_ACTIONS_DECRIPTION_FLAG', 'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG',
          'SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG', 'SUSPECTS_ACTIONS_LOOKOUT_FLAG', 'SUSPECTS_ACTIONS_OTHER_FLAG',
          'SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG']

replacements = {'Y':1, '(null)':0}

for action in actions:
    data[action] = [replacements[item] for item in data[action]]

## Suspect search

In [11]:
# Converting to binary features.
replacements = {'Y':1, '(null)':0}

search_bases = ['SEARCH_BASIS_ADMISSION_FLAG', 'SEARCH_BASIS_CONSENT_FLAG', 'SEARCH_BASIS_HARD_OBJECT_FLAG',
               'SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG', 'SEARCH_BASIS_OTHER_FLAG', 'SEARCH_BASIS_OUTLINE_FLAG']

for sb in search_bases:
    data[sb] = [replacements[item] for item in data[sb]]

## Suspect age

Filling in the null values with the mean age of 29 (as a whole number).

In [12]:
age_mean = data[data.SUSPECT_REPORTED_AGE != '(null)']['SUSPECT_REPORTED_AGE'].astype(int).mean()
age_mean

29.348903251077164

In [13]:
data.loc[data.SUSPECT_REPORTED_AGE=='(null)', 'SUSPECT_REPORTED_AGE'] = 29
data.SUSPECT_REPORTED_AGE = data.SUSPECT_REPORTED_AGE.astype(float)

# Bins the ages into age groups.
def age_bin(df):
    label_names = ['child', 'teen', 'young_adult', 'adult', 'middle_aged', 'older', 'senior']
    cut_points = [0, 12, 17, 25, 35, 55, 69, 150]
    df["suspect_age_group"] = pd.cut(df['SUSPECT_REPORTED_AGE'],cut_points,labels=label_names)
    return df

data = age_bin(data)

## Suspect sex, race description

In [14]:
# Small amount of null values, so I'm filtering these out.
data = data[data.SUSPECT_SEX != '(null)']
data = data[data.SUSPECT_RACE_DESCRIPTION != '(null)']

## Suspect height

In [15]:
data.loc[data.SUSPECT_HEIGHT == '(null)', 'SUSPECT_HEIGHT'] = 0
height_nums = data['SUSPECT_HEIGHT'].astype(float)
# Splits the height values at the decimal.
height_feet, height_inches = divmod(height_nums, 1)

# Converts the heights expressed in full feet to inches.
height_feet_inches = height_feet * 12
height_feet_inches = pd.DataFrame(height_feet_inches)

# Multiplies the inches values by 10.
height_inches_10 = height_inches * 10
height_inches_10 = pd.DataFrame(height_inches_10)
height_inches_10['SUSPECT_HEIGHT'] = height_inches_10['SUSPECT_HEIGHT'].astype(float)

# Converts values of 1.1 to 11 inches.
height_inches_10[(height_inches_10['SUSPECT_HEIGHT'] > 1.1) & (height_inches_10['SUSPECT_HEIGHT'] < 2)] = 11
height_total = height_feet_inches + height_inches_10
height_total.mean()

SUSPECT_HEIGHT    65.49899
dtype: float64

In [16]:
data['SUSPECT_HEIGHT'] = height_total
# Filling missing height values with the mean of ~65 inches.
data.loc[data.SUSPECT_HEIGHT==0, 'SUSPECT_HEIGHT'] = 65.0

def height_bin(df):
    label_names = ['very_short', 'short', 'average', 'tall']
    cut_points = [-1, 63, 68, 72, 144]
    df["suspect_height_group"] = pd.cut(df['SUSPECT_HEIGHT'],cut_points,labels=label_names)
    return df

data = height_bin(data)

## Suspect weight

In [17]:
weight_mean = data.loc[data.SUSPECT_WEIGHT != '(null)', 'SUSPECT_WEIGHT'].astype(float).mean()
weight_mean

168.9017950422642

In [18]:
# Filling missing weight values with 170 lbs, since the data's weight values are to the nearest 10 lbs.
data.loc[data.SUSPECT_WEIGHT == '(null)', 'SUSPECT_WEIGHT'] = 170
data.SUSPECT_WEIGHT = data.SUSPECT_WEIGHT.astype(float)

def weight_bin(df):
    label_names = ['very_light', 'light', 'average', 'heavy', 'very_heavy']
    cut_points = [-1, 90, 140, 180, 220, 1000]
    df["suspect_weight_group"] = pd.cut(df['SUSPECT_WEIGHT'],cut_points,labels=label_names)
    return df

data = weight_bin(data)

## Suspect body build type

In [19]:
# Cleaning up unclear values. Referred to policde codes from other departments for guidance.
data.loc[data.SUSPECT_BODY_BUILD_TYPE == '(null)', 'SUSPECT_BODY_BUILD_TYPE'] = 'unknown'
data.loc[data.SUSPECT_BODY_BUILD_TYPE == 'XXX', 'SUSPECT_BODY_BUILD_TYPE'] = 'unknown'
data.loc[data.SUSPECT_BODY_BUILD_TYPE == 'U', 'SUSPECT_BODY_BUILD_TYPE'] = 'MUSC'

## Suspect eye color

In [20]:
# Cleaning up unclear values. Referred to policde codes from other departments for guidance.
data.loc[data.SUSPECT_EYE_COLOR == '(null)', 'SUSPECT_EYE_COLOR'] = 'OTH'
data.loc[data.SUSPECT_EYE_COLOR == 'ZZZ', 'SUSPECT_EYE_COLOR'] = 'OTH'

## Suspect hair color

In [21]:
# Cleaning up unclear values. Referred to policde codes from other departments for guidance.
data.loc[data.SUSPECT_HAIR_COLOR == '(null)', 'SUSPECT_HAIR_COLOR'] = 'OTH'
data.loc[data.SUSPECT_HAIR_COLOR == 'ZZZ', 'SUSPECT_HAIR_COLOR'] = 'OTH'
data.loc[data.SUSPECT_HAIR_COLOR == 'XXX', 'SUSPECT_HAIR_COLOR'] = 'OTH'

## Stop frisk time

My rationale here is that certain crimes (assaults, robberies, burglaries, etc.) are more likely to be conducted when there is no daylight and thus fewer witnesses able to see clearly.

In [22]:
data = data.rename(columns={'Stop Frisk Time' : 'STOP_FRISK_TIME'})
data.STOP_FRISK_TIME = data.STOP_FRISK_TIME.astype(str)
# Extracts the hour (military time) from the string.
data.STOP_FRISK_TIME = data.STOP_FRISK_TIME.str[:-6]
data.STOP_FRISK_TIME = pd.to_numeric(data.STOP_FRISK_TIME, errors='coerce')
# Filling 2 NaN values with an arbitrary time of 1 PM.
data.STOP_FRISK_TIME.fillna(13, inplace=True)

def stop_time_bin(df):
    label_names = ['late_night', 'early_am', 'morning', 'midday', 'afternoon', 'evening', 'night']
    cut_points = [-1, 4, 7, 10, 12, 16, 19, 23]
    # Left point of a bin is exclusive. Right point is inclusive.
    df["stop_time_period"] = pd.cut(df['STOP_FRISK_TIME'], cut_points, labels=label_names)
    return df

data = stop_time_bin(data)

## Other features

In [23]:
# Converting to binary features.
replacements = {'Y':1, 'N':0}

others = ['OFFICER_EXPLAINED_STOP_FLAG',  'OTHER_PERSON_STOPPED_FLAG', 'OFFICER_IN_UNIFORM_FLAG', 'FRISKED_FLAG',
         'SEARCHED_FLAG', 'OTHER_CONTRABAND_FLAG']

for other in others:
    data[other] = [replacements[item] for item in data[other]]

## Demeanor

Here, I tried to categorize each response based on what I thought the overall demeanor should be logged as, even if a single description value may contain multiple demeanors. I also had to contend with spelling errors and typos.

I tried to identify behaviors that would trigger suspicion in the officer, while also making sure to identify those that fall in the middle of the cooperative-antagonistic spectrum. For example, a suspect that is openly hostile toward an officer is likely to be arrested.

In [24]:
data.DEMEANOR_OF_PERSON_STOPPED = data.DEMEANOR_OF_PERSON_STOPPED.fillna('N/A')
demeanor_db = pd.DataFrame(data.DEMEANOR_OF_PERSON_STOPPED)

In [25]:
cooperative = ['CALM', 'NORMAL', 'COMPLIANT', 'COOPERATIVE', 'APPARENTLY NORMAL', 'CALM AND COOPERATIVE',
       'RELAXED', 'CALM AND COMPLIANT', 'CALM AND UNDERSTANDING', 'UNDERSTANDING', 'HAPPY', 'APP NORMAL', 'OK',
       'CALM, COOPERATIVE', 'CALM/COOPERATIVE', 'CALMED', 'CONTENT', 'FINE', 'CALM, UNDERSTANDING', 'APOLOGETIC',
        'CLAM', 'CALM, COMPLIANT', 'QUIET', 'COOPERTIVE', 'THANKFUL', 'RELIEVED', 'RESPECTFUL', 'CALM.', 'GOOD',
        'CALM/UNDERSTANDING', 'FRIENDLY', 'CALM AND RESPECTFUL', 'COMPLAINT', 'COMPLIENT', 'VERY COOPERATIVE', 'NEUTRAL',
        'SEEMED CALM', 'RESPECTFULL', 'EASY GOING', 'NICE', 'CALM/COMPLIANT', 'CALM/ UNDERSTANDING', 'PLEASANT',
        'HIGHLY UNCOOPERATIVE', 'GREATFUL', 'APPRENTLY NORMAL', 'CALM AND COOPERATIVE.', 'POLITE', 'NO UNUSUAL BEHAVIOR',
               'UNDERSTANDABLE', 'CALM AND RESPECTABLE', 'VERY COMPLIANT', 'CALM/NORMAL', 'NORMAL/CALM', 'NORM', 'RELAX',
               'CALM/CONFUSED', 'CALM COOPERATIVE', 'CALM, COOPERATIVE, COMPLIANT', 'CALM AND POLITE', 'UNDERSTANDING.',
               'CALM AND RELAXED', 'COMPLIED', 'COPPERATIVE', 'COOPERATIVE AND CALM', 'ACTING NORMAL', 'LAUGHING AND CALM',
               'VERY CALM', 'CALM UNDERSTANDING', 'APP. NORMAL', 'COMPLIANT AND UNDERSTANDING', 'THANKFUL FOR EXPLANATION',
               'GRATEFUL', 'CALM / COOPERATIVE', 'NORMAL AND CALM', 'CALM AND NORMAL', 'HIGHLY COOPERATIVE',
               'CALM AND COLLECTIVE', 'APPARENT NORMAL', 'APPARENTLY NORMAL/CALM', 'COMPLIANT, CALM',
               'COMPLIANT AND COOPERATIVE', 'CALM, AND COOPERATIVE', 'COOPERATIVE, CALM','COOPERATIVE, THANKFUL, HAPPY','OKAY',
               'CALM DEMEANOR', 'NERVOUS BUT UNDERSTANDING', 'NERVOUS BUT COOPERATIVE', 'NERVOUS/COOPERATIVE', 'NONCHALANT',
               'CALM SWEATY', 'CALM/NERVOUS', 'COOPERATIVE.', 'CORDIAL','REGULAR','UNDERSTANDING AND COOPERATIVE',
               'APPARENTLY NORMAL, CALM','HELPFUL','COOPORATIVE','CONFUSED, COMPLIANT','PLEASENT','CASUAL',
               'UNDERSTOOD REASON FOR STOP','DEFENDANT COMPLIED','CALM/ COOPERATIVE','THANKFUL AND UNDERSTANDING',
               'AGITATED BUT COMPLIANT','COOPERATIVE, UNDERSTANDING','CALM, NORMAL','CALM AND SWEATY','COOPERATIVE, NERVOUS',
               'CALM. UNDERSTANDING.','CALM, NERVOUS','CALM AND COMPLYING','APPOLOGETIC','CALM AND SHOCKED','COMPLIANT, SCARED',
               'COMPLAYING','CALM AND NERVOUS','NO EMOTION','QUITE','APP NORMAL, NO ISSUE','FAIR','CO-OPERATIVE',
               'APPARENTLY NORMAL, COOPERATIVE','UNDERSTANDING AND THANKFUL','OK AND COMPLIANT','FRIENDLY AND UNDERSTANDING',
               'CALM AND QUIET','CONTENT CALM','UNDERSTOOD WHY STOPPED','COMPLIANT TO BAG CHECK',
               'AT FIRST UPSET/CONFUSED AND THEN UNDERSTANDING AND AMICABLE','CALM AND THANKFUL HE WASNT BEING ARRESTED',
               'COAPERATIVE','GRATEFUL/ HAPPY','COMPLIANT, UNDERSTANDING AND CONFIDENT','CALM, COOPERTIVE',
               'GENERALLY COOPERATIVE','CALM, COOPORATIVE, UNDERSTANDING','UNDERSTANDING & COMPLIANT','VERY UNDERSTANDABLE',
               'COOPRATIVE','RELIEVED, HAPPY','CALM/COOPERATIVE W/VERBAL INSTRUCTIONS','EXCITED AT FIRST BUT THEN CALMED DOWN',
               'NORMAL AND COOPARATIVE.','UNCOOPERATIVE/IRRATE/IRRATIONAL','SURPRISED AND CALM','NERVOUS, COMPLIANT',
               'CALM AND HAPPY','COOPERATIVE WITH POLICE.','AGGITATED BUT COMPLIANT','CALM/SURPRISED',
               'COOPERATIVE, UNDERTANDING','WAS VERY CALM AND WAS VERY COMPLIANT, HE UNDERSTOOD THE REASON FOR THE STOP',
               'UNDERSTANDING AND NORMAL','COOPERATIVE AND COMPLIANT.','APART. NORMAL','VERY HAPPY AND THANKFUL',
               'ALARMED YET COOPERATIVE','AVOIDING EYE CONTACT BUT COMPLIANT','COOPERATED','VERY CALM AND COOPERATIVE',
               'SUBMISSIVE','COOPERATIVE, APPOLOGETIC','WELL','NORMAL, WORRIED','APP NORMAL, NO ISSUES',
               'AGGRIVATED, COOPERATIVE WITH INSTRUCTIONS GIVEN','CALM/ CONFUSED','CAILM','CALM/UPSET',
               'RELUCTANTLY COOPERATIVE','ANGRY BUT COMPLIANT','APPARENTLY NORMAL/ COMPLIANT',
               'UNDERSTOOD THE REASONING FOR THE STOP','CALM AND JUST WANTED TO BE ON HIS WAY','CALM, VERY TALKATIVE',
               'TURNED AROUND AND PUT HIS HANDS BEHIND HIS BACK','SLEEPY/CALM/RELAXED','AMICABLE','HE WAS COOPERATIVE',
               'VERY NORMAL','CALM AND CONFUSED','NORMAL & CALM','OK HE UNDERSTOOD','APPARENTLY',
               'COOPERATIVE AND UNDERSTANDING.','MALE WAS COMPLIANT','TALKITIVE',
               'THANKFUL AND UNDERSTOOD OFFICERS WERE DOING THEIR JOB','CALM/RESPECTFUL','UNDERSTOOD THE REASON FOR THE STOP',
               'APEEARENTLY NORMAL','UPSET AFTER STOP, RELAXED AFTER EXPLANATION OF STOP WAS GIVEN',
               'COMPLIANT, THANKFUL','POLITE/RESPECTFUL','NERVOUS BUT POLITE AND COMPLAINT','COMPLIENT AND UNDERSTANDING',
               'CALM AND COOPERATED','CALM, RELAXED','CALM COMPLIANT','SATISFIED','CALM COMPLIANT HELPFUL',
               'SUSPECT WERE COMPLIANT','CALM APOLOGETIC','OK CLAIM','CO','UPSET BUT UNDERSTANDING',
               'PERSON STOPPED WAS CALM, AND LEFT POLICE WITH A HANDSHAKE.','COMPLIANT/ NERVOUS','CALM AND UNDERSTADING',
               'IN A CALM MANNER','UNDERSTANDING, CALM & COMPLIANT','PERSON WAS COMPLIANT','CALM/AGITATED','PLESANT',
               'CALM , COOPERATIVE.','QUESTIONING, BUT COOPERATIVE','CALM; COOPERATIVE','CALM, UNDERSTANDING, COOPRATIVE',
               'CALM , TALKATIVE,','COOPERATIVE/COMPLIANT','PERON WAS COOPERATIVE AND CALM','AT EASE',
               'APPARENTLY NORMAL AND COMPLYING','INDIFFERENT, CALM','UPSET AND COOPERATIVE','VERY COMPLYING',
               'VERY CALM/ UNDERSTANDING','COMPLIANT WITH OFFICERS','CALM & PROFESSIONAL.','APPARANTLY NORMAL, CALM',
               'SUSPECT WAS CALM','PERSON WAS UNDERSTANDING AND COMPLIANT','COOPERATIVE AND CONFUSED',
               'CALM, COOPERATIVE, AND FRIENDLY','OK (SMILING)','APPRECIATIVE','CALM SITTING ON S/B "F" TRAIN',
               'COOPERATIVE, CALM, RESPECTFUL','CALM AND QUIZZICAL','FINE JUST WANTED TO GO HOME','COMPLAIANT',
               'VERY UNDERSTANDING AND CALM','COMPLIANT-NERVOUS','UPSET BUT THEN CALMED DOWN','CALM, WANTED TO LEAVE',
               'NERVOUS, BUT CALM','CLAM AND UNDERSTANDING','CALM, APPARENTLY NORMAL','SHOCKED AND COMPLIANT',
               'COMPLIANT/UNDERSTANDING','COMPLIANT BUT AGGITATED','COMPLIANT.','CALM AND CORDIAL','PLEASANT, APPRECIATIVE.',
               'UNDERSTANDING AND COMPLIANT','VERY UNDERSTANDING','COURTESY AND PROFESIONAL AND A GENTLEMAN',
               'CALM, SITTING ON A BENCH','CALM BUT BOTHERED','COMPLIANT/ CALM','SUBJECT WAS COMPLIANT',
               'CALM SHORT OF BREATH','SUSPECT WAS CALM AND COOPERATIVE','CALM AND SMILING','CALM  AND APOLOGETIC',
               'UNDERSTANDING/COOPERTIVE','CALM, COLLECTED AND COMPLIANT','COMPLIANT BUT NERVOUS','CONFUSED BUT COOPERATIVE',
               'CALM, AT EASE','SURPRISED BUT CALM DEMEANOR','CALM AND RELAX','NORMAL/ CONFUSED','RESPECTFUL AND UNDERSTANDING',
               'PLEASED WITH INTERACTION','APARENTLY NORMAL','ANGRY THEN RELAXED','NORMAIL','CALM AND HELPFUL',
               'UNDERSTOOD THE REASON FOR STOP','NORMAL BODY LANGUAGE','NORMAL/COOPERATIVE','CALM/ QUIET',
               'CONFUSED; CALM; COMPLIANT','AFTER STOP SUSPECT WAS SLOWLY COMPLIANT AND PROVIDED ID TO OFFICER WITHOUT ARGUM',
               'VERY UNDERSTANDING ON THE SITUATION.',
               'INITIALLY RESISITED, CALM AFTER. STATED HE THOUGHT HE WAS BEING \"JUMPED BY THE P',
               'SUSPECT WAS CALM AND COMPLIENT','CALM/UNDERSTOOD/COOPERATIVE','PERSON WAS COOPERATING WITH OFFICERS.',
               'SUSPECT WAS COMPLAINT AND UNDRSTOOD THE REASON FOR THE STOP','CALM/ TIRED',
               'SURPRISED, BUT UNDERSTOOD WHEN HEARD DESCRIPTION','UPSET AT FIRST/HAPPY AND THANKFUL AFTER',
               'CONFUSED AND COMPLIANT','APPERANTLY NORMAL','VERY CLAM','CALMN','COOPERATIVE WITH VERBAL INSTRUCTIONS',
               'UPSET BUT UNDERSTANDABLE','QUIET AND RESERVED','CALM & POLITE','CALM/RESPECTFUL/COOPERATIVE',
               'LOUD BUT COMPLIANT','COMPLIANANT','COOPERATVIE','PERSON WAS UNDERSTANDING / CALM.',
               'VERY COMPLIANT, APPARENTLY HAPPY','UPSET THEN NORMAL','CALM, COOPEREATIVE, UNDERSTANDING','JOVIAL',
               'SURPRISED AND COMPLIANT WITH PD ORDERS','UPSET THEN CALM','CALM, IN PAIN','CALM, ANNOYED','NORMAL, RESPECTFUL',
               'NORMAL BODY LANGUAGE AND SPEAKING','NERVOUS, CALM, CORPORATIVE','CALM AND COOPERTIVE',
               'COOPERATIVE AND FRIENDLY','CALM/ COMPLIANT','CALM AND COOPERATIVE, COMPLIANT','UNDERSTANDING/ CALM',
               'VERY RESPECTFUL','NORMAL , CALM , UNDERSTOOD STOP.','CIVIL','REGULAR BODY LANGUAGE AND SPEECH',
               'POLITE, UNDERSTANDING','NORMAL.','CALM, COMPLIANT, UNDERSTANDING','COOL CALM','CONFUSED, CALM',
               'COOPERATIVE BUT ANGERY','CONFUSED AND COMPLIENT','COOL, CALM, AND COLLECTIVE','CALM/COOPERATING',
               'PERSONS DEMEANOR WAS CALM AND CURIOUS TO WHY HE WAS STOPPED.','CONFUSED ,COOPERATIVE',
               'UNDERSTANDING WHY SHE WAS STOPPED','UNDERSTANDING/CALM/COOPERATIVE','COOPERATIVE/UNDERSTANDING',
               'COMPLIANT, COOPERATIVE','CALM,COOL,COLLECTIVE','CALM AND CO-OPERATIVE','FRIENDLY AND POLITE',
               'COOPERATIVE, WORRIED.','UNDERSTANDING/ CALM/ GRATEFUL DUE TO INSTRUCTIONS AND COURTESY GIVEN',
               'NON COMPLIANT, THEN COMPLIANT','AGITATED AT FIRST AND THEN COMPLIANT AFTER EXPLINATION OF STOP.',
               'APPARENTLY NORMAL/ CALM','ANGRY THEN CALMED DOWN','UNCONCERNED, UPBEAT','COMPLAIN',
               'CALM/ AND UNDERSSTATNDING OF STOP','EXCITED/ COOPERATIVE','CALM/INDIFFERENT',
               'UNDERSTANDING AFTER HEARING DISCRIPTION OVER RADIO','COOPERATIVE WITH THE INVESTIGATORS','NOT UPSET',
               'CALM/QUESTIONING','CALM/CONCERNED','CALM, UNDERSTANDING.','FIRST NERVOUS THEN CALM','NERVOUS/COMPLIANT',
               'DELIGHTED','NERVOUS, THEN COMPLIANT','REASONABLE','COOPERATTIVE AND RELAXED','AGITATED THEN COOPERATIVE',
               'CALM/ APP. NORMAL','CALM AND COMPLIANCE','REPENTANT','CALM RELAXED','NORMAL:CALM','SWEATING BUT CALM',
               'CALM, COLLECTIVE','SLIGHTLY AGGITATED BUT COOPERATIVE TO AN EXTENT',
               'AGITATED AT FIRST, BUT THEN COMPLIANT AFTER EXPLINATION OF STOP.','NORMAL/ COOPERATIVE','TALKTIVE',
               'UNDERSTANDING AND VERY COOPERATIVE','COOL','AFRAID AND NICE','CALM, QUIET','CALM, COLLECTED, COOPERATING',
               'HAPPY AND UNDERSTANDING','COOPERATIVE WITH POLICE, EXPLAINING HIMSELF.','NOTHING',
               'CALM RESPECTFUL UNDERSTANDING','ANNOYED, BUT CALM','COMPLY','ARGUMENITIVE BUT COMPLIANT','AP NORMAL',
               'CALM AND COMPLAINT','COMPLIANT THAN AGGRAVATED','CALM AND UNDERSTOOD','PERSON WAS CALM AND COMPLIANT',
               'RELAXED, NOT COMBATIVE','COOPERATIVE AND COMPLIANT','VOLUNTEER COMPLAINACE','CALM / UNDERSTANDING',
               'PERSON WAS CALM, COOPERATIVE & APPRECIATIVE.','SURPRISED AND PERTURBED, BUT COOPERATIVE.',
               'CALM,RESPECTFUL, UNDERSTANDING','COMPLIANT/COOPERATIVE','NORMAL AND COOPERATIVE.',
               'RELATIVELY CALM. OUT OF BREATH FROM RUNNING','CALM, COOPERATIVE AND UNDERSTANDING','APPARENTLEY NORMAL',
               'NORMAL, HAPPY','COOPERATIVE & COMPLIANT','PERSON WAS CALM.','COOPERATED AND CLAM','CALM AND APOLOGETIC',
               'NORMAL AND MENTALLY SLOW','RELAXED CALM','VERY CALM , POLITE AND HIGHLY COOPERATIVE','REGREATFUL',
               'AGITATED AT FIRST FOLLOWED BY CALM','CALM, RELIEVED','NORMAL/QUIET','COMPLIANT HOWEVER NERVOUS',
               'COOPERATIVE/ COMPLIANT','IRATE THEN CALM','UPSET BUT COOPROTIVE',
               'WAS NERVOUS BUT REMAINED CALM AND WAS COMPLIANT','THANKFUL & REMORSEFUL.',
               'INITALLY COMBATIVE, THEN COOPERATIVE','THANKFULL AND CURTIOUS','LAUGHING ABOUT STOP AND UNDERSTANDING',
               'SLEEPY BUT COOPERATIVE','THEY UNDERSTOOD WHY','COPERATIVE','CAML','UNDERSTANDING AT THE END OF STOP',
               'VERY UNDERSTANDING TO THE REASON FOR STOP','CALM, NON COMBATTIVE','APPARENTLY NORMAL/COOPERATIVE',
               'DEFENSIVE, BUT COMPLIANT','CALM AND COLLECTED','CAL,M','PERSON WAS COOPERATIVE AND CALM.','GRACIOUS',
               'COOPERATIVE, BUT QUESTIONING WHY HE WAS STOPPED','THE PERSON WAS ANNOYED BUT CALMED DOWN',
               'IRRATIONAL AT FIRST THEN LATER COOPERATIVE.','REGRETFULL, REMORSEFULL, APOLOGETIC','CALM AND HONEST',
               'AGITATED/ COOPERATIVE','RELIEVED NOT ARRESTED','COOPERATIVE AND UNDERSTANDING','COOPERATIVE BUT NERVOUS',
               'NERVOUS; COMPLIANT','FINE AND UNDERSTANDING','PERSON WAS CALM UPON BEING STOPPED.','POSITIVE',
               'RESPESTFUL AND COMPLIANT.','RELAXED AND COOPERATIVE',
               'AGITATED AT FIRST, AND THEN COMPLIANT AFTER EXPLINATION OF STOP.','CALM AND WILLING TO BE SEARCHED',
               'ANNOYED, COOPERATIVE','COOPERATIVE WITH VERBAL INSTRUCTIONS AND PROMPTS','CALM, COOPERATIVE, CONTENT.',
               'UPSET BUT COOPERATIVE','UNDERSTADING','THANKFULL','CALM. COOPERATIVE','CALM, COOPRATIVE',
               'CALM AND FOLLOWED INSTRUCTIONS','CALME','COOPERATING','CALM AND WORRY','FRIENDLY AND COOPERATIVE',
               'APOLOGIGETIC','UPSET, LATER CALM','THEY UNDERSTOOD','NONCHALANT AND POSITIVE ATTITUDE','NERVOUS,COOPERATIVE',
               'CALM COOPERATIVE UNDERSTAND','EXCITED /COOPERATIVE',
               'MALE WAS UPSET, THEN UNDERSTOOD WHY HE WAS STOPPED WHEN EXPLAINED TO HIM','APPARANTELY NORMAL/ CALM',
               'VOLUNTARY COMPLAINCE','ALERTED','APPARENTLY NORMAL CALM','COMPLIAN','UNDERSTANDING AND GRATEFUL',
               'CALM AND COLLECTIVE ONCE STOP WAS EXPLAINED','REMORSEFUL','CLAIM','CALM/COOPERATVIE',
               'HE UNDERSTOOD WHY HE WAS SPOKEN TO.','CALM. QUIET.','COOPRATIVE/ NORMAL','APPRECAITIVE',
               'UNDERSTANDING AND CIVIL','VERY CALM AND COMPLIANT','CALM.  COOPERATIVE','RESPECFULL','APPPARENTLY NORMAL',
               'SURPRISED BUT UNDERSTOOD WHEN HEARING DESCRIPTION','CALM, IN A HURRY','NORMAL AND UNDERSTANDING.',
               'CALM, EXPLAINATIVE, AND COOPERATIVE','CALM,POLITE SHOOK HANDS WITH POLICE','NORMAL AND COMPLIANT',
               'CORDIAL/FREINDLY/HANDSHAKE','CALM/COPPERATIVE','NORMAL AND RESPECTFUL','CAL,'
              ]

nervous = ['NERVOUS', 'ANXIOUS', 'SCARED', 'VERY NERVOUS', 'CONCERNED', 'WORRIED', 'TALKATIVE', 'NERVOUS, SWEATING',
           'NERVIOUS', 'NERVOUS/AGITATED', 'THEY WERE NERVOUS', 'UPSET, NERVOUS', 'NERVOUSE',
           'NERVOUS, SWEATING, AND HEAVY BREATHING FROM RUNNING', 'AFRAID', 'AGITATED/NERVOUS', 'NERVOUS/ANXIOUS',
           'SWEATING', 'NERVOUS, ANXIOUS', 'FRANTIC', 'NERVOUS AND ANXIOUS', 'NERVOUS AND EVASIVE', 'NERVOUS/SWEATING',
           'NERVOUS, SCARED', 'NERVOUS.', 'CALM BUT NERVOUS', 'FRIGHTENED', 'TENSE', 'COMPLIANT, NERVOUS',
           'NERVOUS AND SWEATING', 'JITTERY', 'NERVOUS, OUT OF BREATH', 'EXTREMELY NERVOUS',
           'SWEATING, NERVOUS', 'NERVOUS AND TALKATIVE','UNEASY', 'TIRED/NERVOUS', 'NERVOUS / COMPLIENT',
           'NERVOUS, UPSET', 'NERVOUS, AGITATED', 'HESITANT, NERVOUS', 'NEVOUS','ANXIOUS/ NERVOUS','NERVOUS AND SWEATY',
           'NERVOUS/ AGITATED', 'CALM, COOPORATIVE','RELATIVELY CALM','FIDGETING','NERVES', 'SWEATING AND NERVOUS',
           'NERVOUS AND CONFUSED','AGITATED,UNCOOPERATIVE','NERVOUS,','NERVOUS AND CRYING','AGITATED, NERVOUS','ANTSY',
           'NERVOUS, CRYING','NERVOUS, JITTERY','NERVOUS/UPSET','SHAKEY','CAUTIOUS','APPREHENSIVE','TALKATIVE AND SURPRISED',
           'APOLOGIZING','HYPER','NERVOUS/CONFUSED','TENSED UP','TIRED, SWEATING AND NERVOUS.','SWEATING, APPREHENSIVE.',
           'NERVOUS/ APOLOGETIC','NERVOUS - HEAVY BREATHING','NERVOUS/HEAVY BREATHING',
           'SWEATING, ELEVATED HEART RATE, NERVOUSNESS','NERVOUS SWEATING','NERVOUS/EXCITED','NERVE, SWEATING PROFUSELY',
           'TALKATIVE AND NERVOUS','SCARED AND NERVOUS','NERVOUS AND ANXIOUS, HANDS WERE SHAKING','STUTTERING OF WORDS',
           'NERVOUS, SWEATING AND PANTING','FRIGHTENED, REMORSEFUL','NERVOUS, SWEATING PROFUSLEY','UPSET/JITTERY',
           'NERVOUS, PANICKED','NERVOUS. FIGITY.','NERVOUS/CRYING','VERY NERVOUS AND AGITATED','NERVOUS/ DISTRACTED',
           'CALM & NERVES','SCARED/ANNOYED','NERVOUSNESS','NERVOUS ,ALARMED AND STARLED.','FURTIVE',
           'TALKATIVE, NERVOUS, EVASIVE','ANXOUS','NERVOUS, APOLOGETIC','NERVOUS, EXCITED, UPSET',
           'ACTING NERVOUS, ANXIOUS,','SWEATING PROFUSELY BREATHING HEAVY','NERVOUS / SCARED','SCARED,NERVOUS',
           'NERVOUS,SCARED, CRYING','VERY NERVOUS, IRRITATED, WOULD NOT REMAIN STILL',
           'NERVOUS/FIDGETTY','ANIXIOUS, NERVOUS','NERVOUS, JITTERY, AGRRESIVE, BELIGERENT',
           'AGGRAVATED, NERVOUS, SWEATING, OUT OF BREATH','ACTING NERVOUS','NERVOUS AND ANXIOUS, SHAKING',
           'NERVOUS AND SCARIED','BLEEDING FROM HAND. TALKATIVE.','NURVOS','NERVOUS, IMMEDIATE RUSH','NERVOIUS',
           'IRATE, NERVOUS, EXHAUSTED, MORE CONCERNED ABOUT FIGHT HE WAS JUST IN','OUT OF BREATH, AND NERVOUS','SCARED NERVOUS',
           'NERVOUSE AND QUIET','ANXIOUS, DRUNK','NERVOUS,SCARED','NERVOUS/ CALM','NERVOUS, ALARMED, AND STARTLED',
           'NERVOUS, ERRATIC, INCOHERENT','NERVOUS; ANNOYED','HIGHLY INTOX AND AGITATED',
           'NERVOUS, SWEATING, HEAVY BREATHING FROM RUNNING','NERVOUS/AGGITATED','NERVOUS, SWEATING, EVASIVE ANSWERS',
           'NERVOUS, FLUSTERED','SCARED / COMPLIANT','NERVOUS, BREATHING HEAVY','NERVOUS, SUSPICIOUSLY LOOKING BACK',
           'NERVOUS , SWEATY','NERVOUS / SHAKING AND SWEATING','NERVOUS AND WALKING AROUND LOCATION OF STOP',
           'NERVOUS AND FIDGETY','FIGIDITY, NERVOUS','NERVOUS/TENSE','NERVOUS, SWEATING.','SHOCKED, NERVOUS',
           'NERVOUS AND UPSET','EXCITED / NERVOUS','SHAKING','ACTING NERVOUS, REACHING INTO POCKETS',
           'NERVOUS, SWEATING, FIDGITING','VERY NERVOUS AND CONFUSED','NERVOUS, JITTERY, SHAKY, AND CALM','NERVOUS/ FIDGETY',
           'NERVOUS, SWEATING, UPSET','UPSET AND NERVOUS','NERVOUS, ANXIOUS, COMPLIANT','NERVOUS, SUPRISED, STARTLED, SWEATY',
           'NERVOUS, SHAKING HAND','NERVOUS, OUT OF BREATH, SWEATING','NERVOUS, ANXIOUS ,','NERVES AND SWEATING',
           'NERVOUS AND MOVING ALOT','NEVEROUS','NERVOUS/ FIDGETING','RUNNING AND NERVOUS SWEATING','NEVROUS',
           'NERVOUS AND AGITATED','ANXIOUS NERVOUS','WORRIED/ SWEATING','AGITATED, SWEATY','A NERVOUS CALM,  SWEATY',
           'NERVOUS, SWEATING AND LOOKING ALL AROUND IN ALL DIRECTION PRIOR TO STOP','SCARED AND NERVES','QUIET, NERVOUS',
           'FIDGITING AND CALM','NERVOUS AND ERATIC','NORMAL/NERVOUS','SWEATING,NERVOUS','NERVOUS,ALARMED,STRATLED',
           'ANXIOUS, NERVOUS','NERVOUS/ OUT OF BREATH','NERVOUS,  MOVING AROUND','NERVOUS/FIDGET BODY MOVEMENT',
           'NERVOUS/SWEATY','NERVOUS, TALKING ABOUT AN ARGUEMENT THAT TOOK PLACE','NERVOUS ABOUT PAROLE VIOLATION',
           'NERVOUS/ ANCIOUS','NERVOUS/SURPRISED','NERVOUSE AND SCARED','NERVUS','NERVOUS AND BREATHING HEAVY','SCARED/CRYING',
           'A LITTLE NERVOUS','NERVOUS, EXCITED','NERVOUS,ALARMED,AND SCARE','NERVOUS/FRANTIC',
           'NERVOUS, SWAYING AND UNCOMFORTABLE','HEART RATE UP, NERVOUS, OUT OF BREATH, SWEATING, JITTERY.',
           'NERVOUS, SWEATING AND LOOKING AROUND A LOT','NERVOUS AND ANXIOUS.','JITTERY AND NERVOUS','SCARED AND ANNOYED',
           'NERVOUS, FIDGETY','UNEASY/NERVOUS','FURITIVE MOVEMENTS','NERVOUS AND EXCITED','NERVOUS , ANXIOUS',
           'EXTREMLEY NERVOUS','NERVOUS AND IN A RUSH','AGGRESIVE AND JITTERY','NERVOUS, HESITANT, MOVING AROUND',
           'AGITATED, BREATHING HEAVILY, FURTIVE MOVEMENTS','DEFENSIVE FELT NERVOUS','NERVOUS, MAKING FURTIVE MOVEMENTS',
           'NERVOUS, VISIBLY SHAKING','NERVOUS AND LOOKING AROUND','NERVOUS, SWEATING, TREMBLING','SLEEPING, NERVOUS',
           'NERVOUS AND VERY TENSE','TALKATIVE, NERVOUS','NERVOUS/ AGGITATED','NERVOUS/ HEAVY BREATHING','PANICKING',
           'CALM THEN NERVOUS','NERRVOUS','NERVOUS, SWEATY , HEAVY BREATHING,','CONFUSED/NERVOUS',
           'NERVOUS/ APPREHENSIVE/ SWEATING','NERVIOUSE','NERVOUS AND APOLOGETIC','NERVOUS, RACING HEART BEAT',
           'COOPERATIVE, POSITIVE ATTITUDE','HE JUST QUESTIONED WHY HE WAS STOPPED.','CALM, FRUSTRATED',
           'PROBLEMATIC AT FIRST, AFTERWARDS WAS CALMED DOWN AND UNDERSTOOD THE REASONING FO','ANXIOUS AND NERVOUS',
           'NERVOUS , SCARED','FIDEGTING','VISIBLY NERVOUS','NERVOUS AND ANXIOUS AND UPSET','NERVOUS; FURTIVE MOVEMENTS',
          ]
                                                                                 
antagonistic = ['UPSET', 'ANGRY', 'AGITATED', 'ANNOYED', 'UNCOOPERATIVE', 'AGGITATED', 'IRATE', 'COMBATIVE',
                'AGGRESSIVE', 'IRRATE', 'IRRITATED', 'AGGRAVATED', 'EVASIVE', 'HOSTILE', 'MAD', 'CONFRONTATIONAL', 'DEFENSIVE',
                'INDIFFERENT', 'AGRESSIVE', 'FRUSTRATED', 'AGGRESIVE', 'ARGUMENTATIVE', 'BELLIGERENT', 'ANGRY ATTITUDE',
                'NON COMPLIANT', 'PASSIVE AGGRESSIVE', 'ARROGANT', 'DISRESPECTFUL', 'HESITANT', 'IMPATIENT', 'ANGERED',
                'ANGRY/UPSET', 'AGAITATED', 'AGITATED, COMBATIVE', 'RESISTING', 'VERBALLY COMBATIVE', 'BELIGERENT',
                'HOSTILE AND AGITATED', 'UPSET/ANGRY', 'ARGUEMENTATIVE', 'COMBATIVE, UNCOOPERATIVE', 'IRATE, NERVOUS',
                'SCREAMING', 'IRRATIONAL', 'DISGRUNTLED', 'CONDESCENDING','COMBATIVE AND HOSTILE', 'ANGRY AND HOSTILE',
                'IRATE, UNCOOPERATIVE','AGITATED AND ANGRY','VERY IRATE','AGGIATED','COMBATIVE AND NERVOUS',
                'AGGRESSIVE, BELLIGERENT', 'UNCOPERATIVE', 'NERVOUS, IRATE','ANGRY, UPSET','RESISTING ARREST','VERY ANGRY',
                'COMPLAINING','UNCOOPERATIVE/VULGAR','NERVOUS, AGITATED, IRRATE','DENIAL','NERVOUS/UNCOOPERATIVE',
                'AGGITATED, NERVOUS','ANGRY/ UPSET','VERBALLY CONFRONTATIONAL','VERBALLY AGGRESSIVE','UPSET / ANGRY',
                'ANGRY, AGITATED','AGGRESSIVE. UNCOOPERATIVE','NERVOUS, UNCOOPERATIVE','NERVOUS AND AGGITATED','NERVOUS/SCARED',
                'NERVOUS,JITTERY','FURTIVE MOVEMENTS','SWEATING/NERVOUS', 'SLIGHTLY UNCOOPERATIVE','UPSET AT C/V',
                'ANGRY/NERVOUS','SUSPICIOUS','CALM THEN AGITATED','UNCOOPERATIVE AND ANGRY','BOTHERED','LOUD AND ARGUMENTATIVE',
                'ENRAGED','LOUD AND AGITATED','INCONSISTENT, EVASIVE,NERVOUS','WANTING TO LEAVE, DID NOT WANT TO BE TOUCHED',
                'CALM, AGITATED, UPSET','NERVOUS AND ANGRY','AGGRAVATED.','UPSET/ANGRY/NERVOUS','COMBATIVE, ANXIOUS',
                'ANGRY / ANNOYED','INQUISITIVE, ANNOYED,','VERY AGITATED AND OUT OF BREATHE,','DEFT DID RESIST ARREST',
                'SLIGHTLY AGGRESSIVE','ARROGENT, RELIEVED','HOSTILE/ HIGHLY DISCONTENT','WHY AM I BEEN STOP',
                'IRRATIONAL, ANGRY','ANGRY AND LOUD','COOPERATIVE ON DURING. COMBATIVE/HOSTILE AFTER ARREST.','UPSET, LOUD',
                'SCREAMING AND IRATE','JOKING AROUND MAKING SLY REMARKS','ARGUMENTATIVELY DOMINEERING','ANNOYED/ANXIOUS',
                'DISGRUNTELED','ANGRY, VERBALLY COMBATIVE','NERVOUS AND NOT LISTENING TO DICRECTIONS','DISTANT',
                'AGGRESSIVE, AGGITATED','ANNOYED, ANGRY','HOSTILE, VIOLENT, COMBATIVE, IRRATE,',
                'AGITATED AFTER A BRIEF FOOT PURSUIT.','VERBALLY DEFENSIVE','SWEATING, ARROGANT, EVASIVE, AGITATED',
                'UNCOOPERTIVE AND COMBATIVE','AGGRESSIVE, ANGRY','COMPLIANT, THAN HIGHLY AGGRAVATED.','AGITATED, FURIOUS',
                'LOUD, CAUSING ALARM, REFUSING TO COMPLY WITH DIRECTIONS, DISORDELY','VERY SUSPICIOUS','DECEPTIVE',
                'AGRESSIVE, NON COMPLIANT','ANGRY EXCITED','AGITATED/EXCITED/DISORDERLY','IRRATATED AND INTOXICATED',
                'BELIGERANT, UPSET AND ANGRY.','NERVOUS, COMBATIVE','VERBALLY MANIPULATIVE','NOTICEABLY IRATE',
                'HOSTILE, BELIGIRANT, AGGRESSIVE','NERVOUS, AGGRESSIVE','VEHEMENCE HOSTILITY','SLIGHTLY AGGITATED',
                'INCONSISTENT STATMENTS','BEING IRATE','AGGRESSIVE AND COMBATIVE','IRATE SCREAMING AND MAD',
                'CALM, THEN BECAME AGITATED','NERVOUS. NOT COOPERATIVE','EXCITED, YELLING, SCREAMING, NON-COMPLIANT',
                'ANGRY AT SITUATION','ANGRY THE STORE GAVE HIM A HARD TIME','CALM THEN IRATE',
                'HIGHLY AGITATED AND UNCOOPERATIVE','DEFENSIVE AND AGITATED','VERY HOSTILE AND ANGRY',
                'UNCOOPERATIVE, COMBATIVE, IRATE, SCREAMING, CONFRONTATIONAL','IRATE, ANNOYED','ANGRY AND IRATE',
                'CONFUSED, SLIGHTLY AGITATED','YELLING AND UPSET','FRUSTRATED AND ANGRY','UNCOOPERATIVE AND FRUSTRATED',
                'ANGRY , AGGRESIVE,COMBATIVE','ANGRAY AND INTOX','DEFENSIVE/IRATE','ERRATIC, UPSET AND NERVOUS',
                'IRRITATE AT THE SISTER','IRATE, AGGITATED, YELLING','SWEATING, BLOODY, ANGRY','ANGRY/COMBATIVE',
                'BOISTEROUS IRRATIONAL','AGGRAVATED, YELLING.','INITIALLY CALM, THEN COMBATIVE / DEFENSIVE','AGGRIVATED',
                'EXTREMELY UNCOOPERATIVE AND IRATE.','INDIFFERENCE, SLIGHT AGITATION','AGGRESIVE COMBATIVE',
                'PERSON REFUSED TO LISTEN TO POLICE OFFICERS, WAS VERY HOSTILE AND SWEATING PROFU','CALM BUT UNCOOPERATIVE',
                'ANGRY AND ANTAGONIZING','LOUD AND LOTS OF SIDE TO SIDE MOVEMENT AND LOOKING AROUND.',
                'COMBATIVE/DEFENSIVE/ARGUMENTATIVE','SUSPECIOS','IRATE, THREATING, CLENCHING FISTS WANTING TO FIGHT',
                'HIGHLY COMBATIVE/UNCOOPERATIVE','FRUSTRATED, ANNOYED','ANGRY/ HIGHLY UNCOOPERATIVE',
                'AGGRESSIVE /UNCOOPERATIVE','BOISTEROUS','RUDE AND DISORDERLY',
                'VERBALLY ABUSIVE, AGGRESSIVE AND VERY CONFRONTATIONAL','HE WAS VERY AGITATED AND UPSET.',
                '(IRATE) (AGITATED)','HYPER VIGILANT, APPREHENSIVE AND EVASIVE','IRATE, AGGITATED, ARGUMENTATIVE',
                'AGGRESIVE AND VIOLENT','PASSIVE AGRESSIVE AND AGITATED','FLAILING ARMS, BEING IRATE.','IRRITABLE',
                'VERBALLY AGRESSIVE','VERY RUDE AND AGGITATED THAT P.O STOPPED HIM AND MADE HIM MISS HIS TRAIN',
                'HOSTILE, AGGRESSIVE','RELUCTANT TO COMPLY','BELLIGERENT AND AGGRESSIVE','EXCITED AND IN DENIAL',
                'AGGRESSIVE, AGITATED, RESISTANT, COMBATIVE','DISHONEST','EXTREMLY IRATE',
                'WOULD NOT KEEP HIS HANDS DOWN, MADE FIST AT ME PRIOR TO BEING HANDCUFFED','FIGHTING STANCE',
                'ARGUMENTIVE/FIGGITY','REFUSING TO ANSWER ANY QUESTIONS AS TO WHY HES INSIDE THE LOT',
                'AGRESSIVE, BELLIGERENT, ANNOYED','IRRATE AND NON COMPLIANT','UNCOOPERATIVE, FIDGETY',
                'ANGERED, NERVOUS, UNCOOPERATIVE, VERBALLY COMBATIVE','AGITATED AND UPSET','YELLING, FIGHTING',
                'UPSET, YELLING','ANNOYED/AGGITATED','CONFUSED, ARGUMENTATIVE','AGGRESIVE AND RESISTING',
                'COMBATIVE, ARGUMENTATIVE','AGGRAVATED, HOSTILE, NERVOUS, AVOIDING EXPLANATION','AGITATED & IRATE',
                'HOSTILE UNCOOPERATIVE','AGGIATED, AGRESSIVE','LOUD','NERVOUS AND AGGRESSIVE.','IRRATE/ BELLIGERENT',
                'AGITATED, ANNOYED, NON-COMPLIANT','ARROGANT AND HOSTILE','ARROGANT, CARELESS,',
                'EXTREMELY NERVOUS AND COMBATIVE','FRUSTRATED/FLUSTERED','AGGRESSIVE AND AGITATED','HOSTILE/ COMBATIVE',
                'UNCOOPERATIVE, AGITATED','BELLIGERANT, IRATE, HOSTILE','IRRATE, ANGRY','INTIALLY COMBATIVE',
                'AGGRESSIVE , COMBATIVE','CALM AT FIRST BUT ONCE IN CUFFS, STARTED TO STRUGGLE',
                'ANNOYED, WON?T MAKE EYE CONTACT','AGGRESSIVE, HOSTILE. VERBALLY ABUSIVE','ANNOYED.','IRATE, NERVOUS, UPSET',
                'VERBALLY ABUSIVE, THEN CRYING','VULGAR, HOSTILE, AGGRESIVE','HE SEEMED ANNOYED','NERVOUS, BLADING HIS BODY.',
                'UPSET, CURSING, BLADING BODY.','IRRATE, COMBATIVE','AVOIDING POLICE CONTACT','MODERATELY AGITATED',
                'IRRITABLE, FIDGETY','AGGRIVATED , APPREHENSIVE','BELIGERENT, ANGRY',
                'NERVOUS, SWIFTLY WALKED AWAY AS SOON AS SIGHT OF MARKED RMP APPEARED','ARGUMENTIVE',
                'TALKATIVE, AGITATED AND IRRITATED','AVOIDING UNDERSIGNED','UNCOOPERATIVE AND MISLEADING','ANGRY, BELLIGERENT',
                'NERVES AND ANGRY','QUIET AND INDIFFERENT','IRATE AND YELLING','VERY COMBATIVE/ UNCOOPERATIVE',
                'HE WAS FIDGETING, APPEARED HE WOULD FLEE','IRATE THREATENING REMARKS TOWARDS MOS','IRATE, AGRESSIVE',
                'UNCOOPERATIVE/RESISTING ARREST','EVASIVE AND COMBATIVE','AGITATED AND IRRATE',
                'AGGRESSIVE, UNCOOPERATIVE, IRATE','FLEEING / NERVOUS/ TRYING TO ESCAPE / FURTIVE MOVEMENTS',
                'NERVOUS, IRRITATED, BELLIGERENT','REFUSE POLICE COMMANDS, BLADING BODY','ANGRY/ NERVOUS','COCKY,ARROGANT',
                'ANGRY AND SURPRISED','ARGUMENTIVE, UNCOOPERATIVE','TALKATIVE, COMBATIVE','NERVOUS, SCARED, IRATE','REFUSING',
                'NERVOUS, DEFENSIVE.','AGITATED/AGRESSIVE POSTURE','HOSTILE AND AGGRESSIVE','ACTING AS A LOOK OUT',
                'EVASIVE WHEN QUESTIONED ABOUT WHY HE KEPT RUNNING FROM OFFICERS.',
                'RUNNING, FLAILING ARMS, PUSHING AND SWATTING','NERVOUS; EVASIVE; UNCOOPERATIVE','YELLING',
                'COMPLIANT, THAN AGGRAVATED.','IRRATE, LOUD AND CURSING AND AGGREIVE STANCE','AGITATED/AGGRESIVE',
                'AGITATED AND SCARED','NONCOMPLIANT AND UNCOOPERATIVE','NERVOUS, ANXIOUS, ARGUMENTIVE',
                'ANGRY, CONFRONTATIONAL, DEFENSIVE, EVASIVE IN ANSWERING QUESTIONS','REFUSING TO COMPLY',
                'INITIAL DEMEANOR WAS CONFRONTATIONAL','ABRASIVE, AGGITATED, INDIVIDUAL SHORT OF BREATH',
                'CRYING, ABRASION ON HAND, ANGRY','AGRESSIVE, UNCOOPRATIVE','AGRESSIVE, ATTEMPTING TO ESCAPE',
                'AGGRESSIVE BEHAVIOR','CUSING AND WALK OVER TO UNMARKED RMP AND TOOK PICTURE OF VEHICLE','EVASIVE AND ANNOYED',
                'IRATE AND UNCOOPERATE','AT FIRST CALM, THEN AGITATED','SUSPICIOUS, AGGRESSIVE AND VERY UNCOOPERATIVE.',
                'PERSON BECAME VERY UPSET AND HOSTILE','AGIGTATED','AGITATED, EVASIVE','DISRESPECTFUL AND RUDE',
                'AGGITATED/ANGRY','TOOK FLIGHT THEN BECAME COMPLIANT AFTER APPREHENSION','IRRATE, ANGRY, AGGRESSIVE',
                'NERVOUS/AGITATED/AGGRESSIVE','TALKATIVE, AGGRESSIVE','AGRIVATED',
                'PRETENDING THE PERSON DID NOT DO ANYTHING WORNG','VERBALLY AND PHYSICALLY COMBATIVE','ANGRY- WALKED AWAY',
                'ERATIC AND COMBATIVE','ANGRY, AGGRESSIVE','UPSET TRYING TO RUN','ANGRY/ AGGRESSIVE','NERVOUS/ ANGRY',
                'ANGRY/ANNOYED','UNCOOPERTIVE','IRRATIONAL NERVOUS','VIOLENT, UNCOOPERATIVE, COMBATIVE','VERY DISGRUNTELD',
                'COOPERATIVE WITH PO INSTRUCTIONS HOWEVER DEMEANOR WAS ANTAGONISTIC','ONCOOPERATIVE',
                'READY TO RUN AS THE OFFICER APPROACHED','AGGRESSIVE. UNCOOPERATIVE. NERVOUS',
                'VERY ANGRY, HIGHLY HOSTILE , COMBATIVE,, VIOLENT, AND UNCOOPERATIVE',
                'ERRATIC, ANGRY, COMBATATIVE, ATTEMPTED TO FLEE','TALKATIVE, UPSET',
                'DISORDERLY BY BEING LOUD, WAVING HANDS, REFUSING TO TALK TO OFFICERS AND WALKING',
                'VERY MAD, UPSET, STREMELY ANGRY, IRATE','UPSET, UNCOOPERATIVE','ACTING DISORDERLY.',
                'INITIALLY CALM THEN AGGREVATED','INCOHERENT, NOT ANSWERING QUESTION',
                'NERVOUS AND NOT LISTENING TO OFFICERS/ WASNT FOLLOWING DIRECTIONS','COMBATIVE, BELLIGERENT',
                'NERVOUS, DEFIANT, INSISTENT ON INNOCENCE','EXTREMLY NERVOUS AND COMBATIVE','AGITATIED.',
                'EVASIVE/UNCOOPERATIVE','ANGRY, CONFUSED, UPSET, SHOCKED','ANNOYED/ATTITUDE','ANGRY, YELLING, IRRATIONAL',
                'SARCASTIC','ANGRY/UNCOOPERATIVE','AGRESSIVE, NOT COMPLIANT','DISSATIFIED, ARGUMENTATIVE',
                'NERVOUS/ COMBATIVE','CONFUSED/BELLIGERENT','AGGRESIVE AND HOSTILE',
                'CALM , THEN BECAME AGITATED AFTER BEING QUESTIONED','IRATE SPEAKING OUT',
                'NERVOUS, UNCOOPERATIVE, UPSET, ANGRY','NERVOUS, SWEATY, AGITATED','ANXIOUS, NERVOUS, ANGRY','ARGUMANTATIVE',
                'COMBATITVE AND THREATENING','ANGRY AND IRRATE.','ANTAGONISTIC','ANGRY, AND BEING UNCOOPERATIVE',
                'UNCOOPERATIVE. AGGRESSIVE. NERVOUS','IRATE/ANGRY','POOR ATITUDE','IRATE, CONFRONTATIONAL',
                'VERY UPSET AND AGGRESSIVE','PANICKY, JITTERY, ANXIOUS, AGGRESIVE, SWEATY AND  ALSO CONCEALING HIS LEFT HAND',
                'PERSON WAS BELIGERENT','NERVOUS AND ARGUMENTATIVE','AGITAED','EXTREMELY AGITATED',
                'EXTREMELY AGITATED, DEFENSIVE','EXTREMELY COMBATIVE','THE PERSON STOPPED ACTED ERRATIC','UPSET , AGGITATED',
                'DISTRACTED AND AGIATATED','SURPRISED, ANGRY','CALM, EVASIVE','UPSET, ANGRY','AGITATED AND NERVOUS',
                'NERVOUS, EXCITED, NON COMPLIANT','VIOLANT/ARGUMENATIVE','STRANGE, DISMISSIVE','AGRESSIVE ,UNCOOPERATIVE ,',
                'AGITATED/COMBATIVE','NERVOUS COMBATIVE','CALM TO AGGRESSIVE','CRYING AND UNCOOPERATIVE',
                'ANGRY AND  COMBATATIVE','ARGUMENTATIVE, LOUD, ANGRY, FLAILING ARMS AND THREATS WERE MADE','ANGRY AND VOCAL',
                'VERY AGGRESSIVE BEHAVIOR','AGITATED/ ANGRY/ UNCOOPERATIVE','SEMI COOPERATIVE','AGGRIVATED, AGITATED, BOTHERED',
                'ANGRY, IRATE','DESPONDENT','DESPERATE','UNCOOPERATIVE/ COMBATIVE','ANGRY, UPSET, FRUSTRATED',
                'UPSET AND VERBALLY COMBATIVE','IRATE, COMBATIVE, VIOLENT','UNBOTHERED','ANGRY AND ANNOYED',
                'EXTREMELY HOSTILE, AND IRRATE','YOU STOP ME BECAUSE I AM BLACK','ANGRY, HOSTILE, ARGUMENTATIVE',
                'ANGRY UNCOOPERATIVE','IRRATE (UPSET, YELLING)','NOT COOPERATIVE','AGGRESIVE, NERVOUS',
                'NERVOUS, SAT ON THE FLOOR, SCREAMING','NERVOUS, AGGITATED','NERVIOUS/AGITATED','ANGRY AND NON COMPLIANT',
                'SILENT','IRATE, VERY AGGRESSIVE,','ANGRY AND BELLIGERENT',
                'AGITATED, PANTING, INTOX, BELLIGERENT, UNCOOPERATIVE','ANGRY, AGGITATED, ARGUMENTATIVE','NON-CHALANT',
                'ANNOYED, MAD','EXCITED, VERBALLY COMBATIVE','IIRATE AND VERY VULGAR TOWARD OFFICERS','ANNOYED & UPSET',
                'UPSET/NONCOMPLIANT','EVASIVE AND TRYING TO LEAVE','FIDGETY, UNCOOPERATIVE','ANGRY, UPSET, IRATE, BELLIGERENT',
                'CONFUSED, CONTRADICTORY, NERVOUS AND EVASIVE.','NERVOUS, ANGRY','MAD/FRUSTRATED','ANGRY, HOSTILE',
                'AGGRIVATED AND NERVOUSE','AGGRESSIVE TOWARDS POLICE','AGRESSIVE,ANGRY, DISORDERLY,NONCOMPLIANT',
                'AGGRESSIVE, VIOLENT, SELF INFLICTING INJURY, SUICIDAL','EVASIVE.','NON COMPLIANT, EVASIVE',
                'COMBATIVE AT FIRST APPROACH','NERVOUS, AGGRESSIVE.','VULGAR','AGGRESSIVE/RESISTANT','ANGRY AND AGGRESSIVE',
                'AGITATED / EVASIVE','RESISTING AND AGGRESIVE','INDEFFERENT AND IGNORING POLICE COMMANDS',
                'NERVOUS, EVASIVE IN HIS ANSWERS','IRATE, DISCOURTESY,','UPSET ANGRY','COMBATIVE UNCOOPERTIVE',
                'AGITATED, CONFUSED','NON COMPLIANT, AGITATED','HYPER, AGITATED','ANGRY AND AGGITATED','ANXIOUS, SARCASTIC',
                'NERVOUS AND MISLEADING','LOUD/ AGGRESSIVE','REFUSED TO GIVE PEDIGREE','UNCOOPERATIVE/ ANNOYED',
                'AGITATED/ NERVOUS','PRETENDING PERSON DID NOT DO ANYTHING WORNG','ANGRY ADN UPSET','CONFUSED/ANGRY',
                'IRRATE AND VERBALLY THREATENING.','UNCOMPLIENT/ANNOYED','NERVOUS, ARGUMENTATIVE','IRATE, SCREAMING',
                'NERVOUS, UNCOOPERATIVE, HOSTILE','UNRULY/ DISRESPECTFUL','EXTREMELY IRATE',
                'LOUD, CAUSING ALARM, REFUSE TO COMPLY WITH DIRECTION, DISORDERLY','INCONSISTENT',
                'UPSET, NERVOUS, UNCOOPERATIVE.','VERY AGGRESSIVE AND VIOLENT','NERVOUS, AND EVASIVE',
                'AGGRESSIVE, UNCOOPERATIVE','SARCASTIC/POLITE','IRRATE/ UNCOOPERATIVE','VERBALLY AGGRESSIVE/COMBATIVE',
                'RESISTANCE, AND UNCOOPERATIVE','UPSET / ANNOYED',
                'AGGRIVATED, ANNOYED, THREATEING, YELLING AT C/W, AND OFFICERS','RESISTANT/ AGGRESSIVE/ NON-COMPLIANT',
                'CALM. ROLLING EYES. SHAKING, ACTING NERVOUS','CRYING, EVASIVE, REPETITIVE',
                'CALM THEN BECAME DISRUPTIVE AND YELLING',
                'ANGRY, EVASIVE, UNCOOPERATIVE, ARROGANT, DIDN/T FOLLOW DIRECTIONS THE FIRST TIME',
                'AGGRAVATED, UPSET AND CONFRONTIVE','AGGRESSIVE / FIGHTING','REFUSED TO COMPLY WITH LAWFUL ORDER','ANNYOYED',
                'NERVOUS, AGGRESSIVE, AGITATED','AGITATED, ANGRY','IRATE, UNCOOPERATIVE, USING PROFANE LANGUAGE',\
                'UNCOOPERATIVE/AGGITATED','LITTLE ANNOYED','CALCULATING','LOUD AND BOISTEROUS','ARROGANT/RELIEVED',
                'VERY VIOLENT RESISTING ARREST','ANGRY/ BELLIGERENT','ANGRY AND AGITATED','ACTING OUT',
                'CONFRONTATIONAL AND UNCOOPERATIVE','LOUD - IRATE-AGITATED-THREATENING','AGGRESSIVE; COMBATIVE;',
                'AGITATED/ANGRY/DISRESPECTFUL','ANGRY, DEFIANT','ANGRY, AGGITATED, REFUSAL TO COMPLY WITH OFFICERS',
                'AGGRESSIVE, BELLIGERENTS','IRRATE,','BLADED BODY/BENT FORWARD','NERVOUS. AGGRESSIVE','SUSPICOUS',
                'NON-COMPLIANT, HOSTILE, COMBATIVE,','HOSTILE/ IRRITATED','UNCOORPERATIVE','EMOTIONLESS, UNTALKATIVE',
                'ABRASIVE AT FIRST','VERY HOSTILE','CALM DURING STOP, BELLIGERENT AFTER ARREST',
                'HIGHLY UNCOOPERATIVE AND NERVOUS','AGGRAVATED AND NERVOUS','AGGRAVATED, NASTY','COMBATATIVE, ANGRY',
                'EVASIVE, UNCOOPERATIVE','NERVOUS. INITIALLY NON COMPLIANT.','FIGIDITY, ANGRY','AGGRESSIVE/AGGITATED',
                'BELIIGERANT','NONCOMPLIANT','INCONSISTENT, NERVOUS','AGGITATED, COMBATIVE','AGGRESIVE, THREATENING',
                'ANNOYED AND DISCOURTEOUS','IRRATE/ANGRY','AGITATED.','NERCOUS UNCOOPERATIVE','ELUSIVE',
                'UNCOOPERATIVE/REFUSED PEDIGREE, ANGRY, NERVOUS, VERBALLY COMBATIVE','BELIGERANT',
                'AGRESSIVE, LOUD, CURSING, SHOUTING','HOSTILE, ANGRY','NOT FOLLOWING ORDERS','DISCONTENT','IRATE AND ODD',
                'IGNORING POLICE WANTED TO SLEEP','AGGRESSIVE AND AGITATIVE','ANNOYED/IRATE','VERBALLY AGITATED',
                'IRRITATED, AGGRESSIVE, LOUD, NON-COMPLIANT','EXCITED / ARGUMENTIVE','MAD/UNHAPPY',
                'COMBATIVE AND RESISTING ARREAT','COMBATIVE AND AGITATED','NERVOUS, AGITATED, AND APPREHENSIVE',
                'UPSET, UNCOOPERTIVE','AGGRESSIVE/ANGRY','DISORDERLY, AGGIATED','AGGREVATED',
                'SCARED WAS HIDING UNDERNEATH A VEHICLE','COMBATATIVE','COMBATIVE AND DISRESPECTFUL.','AGIGITATED',
                'EXTREMELY UNCOOPERATIVE, BELIGERANT','ACTING CONFUSED, WANTED TO MAKE PURCHASE','ANGRY AGITATED IRATE',
               ]
                                                                 
surprised = ['SURPRISED', 'EXCITED', 'SHOCKED', 'SUPRISED', 'ALARMED', 'SURPRISE', 'STARTLED','SURPISED','CALM, SHOCKED',
             'SURPRISED/ AGITATED',
            ]

other = ['A/N', 'NONE', 'COMPLACENT', 'PASSIVE','RESTLESS','SAME AS DEMEANOR PRIOR', 'CONCERN','UNKNOWN',
         'ANXIOUS TO LEAVE', 'CONFUSED / EXCITED', 'CONFUSED', 'IN A HURRY','IN A RUSH / HURRY','NORNAK',
         'SAME AS PRIOR TO STOP','CONFUSED ON WHY HE WAS BEING STOPPED','CONCERNED ABOUT PRE STOP INJURY','ARRESTED',
         'CONFUSED.','UNAWARE','ALERT','FELONY','CALM, CONFUSED','WORKED UP','CONSERNED',
         'SUSPECT IMMEDIATELY PUT HIS HANDS UP AND TURNED AROUND FROM OFFICER','HAPPY , LAUGHING','STONE COLD','STRESSED',
         'NONE SHOWN','SUPRISED/LOST','NEGATIVE RESULTS FROM C/V','HUNGRY','N.A','RESTLESS/ CONFUSED','SAME','IN RUSH',
         'CALCULATED','ADA WITNESS A CRIME','JUST WANTED TO LEAVE TO CATCH A BUS.','SLEEPING ON THE STAIRWELL',
         'CONFUSED FOR REASON OF STOP','UNK','CONCERENED','UNCERTAIN','CURIOUS FOR THE REASON OF THE STOP','VERY APOLOGETIC',
         'HIGHLY TALKATIVE','CURIOUS','CAREFREE','SORRY','IN A RUSH TO LEAVE','UNCARING','HUMBLE','CONFUSED/ LETHARGIC',
         'IN A RUSH','RELIVIEVED','SURPRISE AND CONFUSED','DISAPPOINTED','EMOTIONLESS','AIDED GOING TO HOSPITAL',
         'JOKINGLY ASKED IF HE CAN HAVE A FEW DOLLARS'
        ]                                                  

tired = ['TIRED', 'OUT OF BREATH','TIRED FROM RUNNING','PANTING AND TIRED', 'EXHAUSTED', 'OUT OF BREATHE',
         'PERSON WAS OUT OF BREATH','OUT OF BREATH/ AGITATED','OUT OF BREATH TIRED','HEAVY BREATHING AND SWEATING',
         'TIRED  SWEATY  CALM','OUT OF BREATH AND SCARED','FATIGUED','SWEATING, OUT OF BREATH','CALM BUT OUT OF BREATH.',
         'NERVOUS, SWEATING FROM RUNNING.','TIRED, SWEATING','OUT OF BREATH AND NERVOUS','SWEATY, HEAVY BREATHING',
         'ANXIOUS; NERVOUS; SWEATING; BREATHING HEAVILY','HEAVY BREATHING/SHORTNESS OF BREATH','NERVOUS , OUT OF BREATH',
         'BREATHING HARD','SWEATING, OUT OF BREATH, STUMBLING AROUND FENCE','OUT OF BREATH AND SWEATING'
        ]

upset = ['UNHAPPY', 'EMOTIONAL', 'CRYING', 'SAD', 'FUSTRATED', 'UPSET/CRYING', 'NOT HAPPY','DEJECTED', 'VISIBLY UPSET',
         'UPSET, CRYING','UPSET AND TENSE','INCONVENIENCED','UPSET, BOTHERED','SUSPECT UPSET','AFRAID, UPSET',
         'PERSON STOPPED WAS UPSET','EXCITED/UPSET','UPSET, CALM, AGITATED','AGITTATED','DISTRESSED','A LITTLE UPSET',
         'HIGHLY UPSET','UPSET DUE TO FAMILY MEMBER PASSING AWAY','UPSET AND CONFUSED','UPSET, CONFUSED','UPSET/ANNOYED',
         'UPSET AT PARK SECURITY','CALM, UPSET','PERSON WAS UPSET AND CRYING','UPSET, CONCERNED','ANGRY AND UPSET',
         'SWEATING AND IRRATE','AGITATED AND ANNOYED','UPSET/TENSE','IRRATE, UPSET','WORRIERD AND CRYING','CRYING, ANXIOUS',
         'CALM BUT UPSET','EMOTIONAL AND CRYING','AGITATED, UPSET','UPSET, IRRATE.','DEFEATED','DISTRAUGHT','UPSET, AGITATING',
         'SLIGHTLY UPSET','UPSET/NERVOUS','WAS UPSET BECAUSE HE WAS RUSHING TO GO TO HIS BOXING TRAINING',
        ]

inebriated = [ 'INTOXICATED', 'INTOX', 'DISORIENTED', 'INCOHERENT','AGGITATED, INTOXICATED','HIGHLY INTOXICATED','CALM/INTOX',
              'UNDER THE INFLUENCE','APPARENTLY INTOX NARCOTICS','APPARENTLY INTOXICATED','APPARENTLY  INTOX','DRUNK',
              'AGITATED AND INTOXCICATED','NERVOUS, INTOX TURNING HIS HEAD LOOKING BEHIND HIM & ONCE SAW POLICE HE WENT BEH',
              'NERVOUS,INTOX','HIGHLY INTOX, UPSET, YELLING AT WITNESS','SUSPECT WAS INTOXICATED','AGGRESSIVE, INTOXICATED',
              'INTOXICATE BY MARIJUANA','ON DRUGS & ANNOYED','CALM AND DAZED','INTOX/COMPLIANT','RUDE, INTOX, LOUD',
              'SWEATING. INTOX , BLOOD ON HANDS','HIGH ON HEROIN','CALMED/DRUNK','UPSET /INTOX','INTOXICATED AND UNCOOPERATIVE',
              'SHAKING AND SLURRING WORDS','POSSIBLE INTOXED','SCARED/INTOX','POSSIBLY INTOX OR DRUG USE',
              'APARRENTLY INTOX','HIGHLY INTOX, IRATE',
              'SUBJECT APPEARED TO BE UNDER THE INFLUENCE OF AN UNKNOWN SUBSTANCE/ VERBALLY ABU','INTOX AND COMBATIVE',
              'ALCOHOL INTOXICATION','AGGITATED/INTOXICATED','ANGRY INTOX PERSON','INTOXICATED, IRATE, ANGRY','ELEVATED',
              'UPSET,  INTOX','ANNOYED/DRUNK','INTOX BUT FUNCTIONAL','JOVIAL/INTOX','APPARENTLY INTOX','INTOX ALCOHOL',
              'HIGHLY UNCOOPERATIVE/ HIGHLY INTOXICATED','INTOXICATED, AGITATED, AGGRESSIVE.',
              'INTOX, CRYING, UPSET AT HIS ACTIONS','COOPERATIVE,  INTOXICATED','DISORIENTATED','NERVOUS, INTOX',
              'AGGRESSIVE, CONFRONTATIONAL, INTOXICATED','INTOXICATED/CONFUSED','INTOXICATED - ALCOHOL',
              'INTOXICATED, AGGRAVATED','INTOXICATED/LOUD','DRUNK AND AGGRIVATED','NERVOUS, INTOXED',
              'INTOXICATED AND UPSET AT WIFE','HIGHTLY INTOXICATED','DRUNK BUT CALM','CONFUSED AND INTOXICATED',
              'INTOX AND HAPPY','SEMI-CONSCIENCE','INTOX/COMBATIVE','INTOX/PEACEFUL'
             ]
                                                                        
unstable = [ 'ERRATIC', 'DISORDERLY', 'LAUGHING', 'UNSTABLE','EMOTIONALLY DISTURBED','HYSTRICAL','PARANOID',
            'ERRATIC, DELUSIONAL','MUMBLING TO SELF',
           ]

fled = ['FLED ON FOOT', 'FLED','TRIED TO FLEE','WALKED AWAY','IRRITATED AND ANNOYED AND WALKING AWAY','RUNNING',
        'DEFT DID TOOK OFF RUNNING S/B AFTER BEING APPROACHED BY POLICE OFFICERS','RUNNING AND EXCITED',
        'DEFT. DID RUN FROM A/O','RAN','NERVOUS RAN FROM MOS','UNCOOPERATIVE, RUNNING','NERVOUS, WALKING AWAY','FLEED ON BIKE',
        'DIDN\'T REPLY TO QUESTIONS, IMMEDIATELY LEFT','NERVOUS, RAN AWAY','CALM THEN FLED','STARTED TO FLEE, HAD TO BE CHASED',
        'FLED AFTER CONTAC WITH POLICE, NERVOUS','EVASIVE, FURITIVE, ANGRY, GAVE FLIGHT','CALM AND WALKED AWAY',
        'CHANGE TRAVEL AT SIGHT OF POLICE','SUSPECT FLED UPON APPROACH OF PD','ATTEMPTED FLIGHT.',
        'QUIET AND CALM, BUT ATTEMPTED TO WALK AWAY TWICE',
       ]

In [26]:
# for coop in cooperative:
#     demeanor_db['demeanor_cooperative'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda coop: 1 if coop in cooperative else 0)

for nerv in nervous:
    demeanor_db['demeanor_nervous'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda nerv: 1 if nerv in nervous else 0)
    
for ant in antagonistic:
    demeanor_db['demeanor_antagonistic'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda ant: 1 if ant in antagonistic else 0)

for surp in surprised:
    demeanor_db['demeanor_surprised'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda surp: 1 if surp in surprised else 0)

for oth in other:
    demeanor_db['demeanor_other'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda oth: 1 if oth in other else 0)
    
for tire in tired:
    demeanor_db['demeanor_tired'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda tire: 1 if tire in tired else 0)
    
for ups in upset:
    demeanor_db['demeanor_upset'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda ups: 1 if ups in upset else 0)
    
for ineb in inebriated:
    demeanor_db['demeanor_inebriated'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda ineb: 1 if ineb in inebriated else 0)
    
for uns in unstable:
    demeanor_db['demeanor_unstable'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda uns: 1 if uns in unstable else 0)
    
for fl in fled:
    demeanor_db['demeanor_fled'] = demeanor_db.DEMEANOR_OF_PERSON_STOPPED.apply(lambda fl: 1 if fl in fled else 0)

In [27]:
data = pd.concat([data, demeanor_db], axis=1)

## Suspect other description

Here, I tried to select the traits of the suspect's appearance that would potentially trigger an officer's bias against a particular people group or their suspicion of the suspect's motives. I also had to contend with spelling errors and typos.

A bag on a suspect could lead the officer to sense that the suspect has stolen something. Fresh wounds may suggest that the suspect was involved in a physical altercation.

In [28]:
data.SUSPECT_OTHER_DESCRIPTION = data.SUSPECT_OTHER_DESCRIPTION.fillna('N/A')
appearance_db = pd.DataFrame(data.SUSPECT_OTHER_DESCRIPTION)

In [29]:
normal = ['KAREN','CAST ON LEFT ARM','GREEN SWEATER, BLACK PANTS','BEIGE TRENCHCOAT','RED T-SHIRT, DARK SHORTS',
         'RED SWEATER ,GRAY SWEAT PANTS','NAVY BLUE FLEECE','BLACK SHIRT AND BLUE PANTS',
         'DARK COLOR SHIRT','BLACK T-SHIRT, BLACK JEANS, BLACK SANDALS','GREEN, JACKET, BLUE JEANS',
         'BLUE SHIRT, BLUE JEANS, BLACK SNEAKERS.','DARK JACKET, KHAKIS, SNEAKERS','EYE GLASSES, BLUE T-SHIRT, BROWN SHORTS',
         'DENIM JACKET, BLACK PANTS','PINK JACKET, BLACK PANTS','DARK COAT, BLUE JEANS, WHITE SNEAKERS',
         'BLUE SHIRT, PLUE PANTS, BACKPACK','ORANGE/ RED LONG SLEEVE SHIRT, BLACK SHORTS.','BLACK SHORTS, BLACK SHIRT',
         'GRAY SHIRT, RED WHITE BAG, BLUE WHITE SNEAKERS','BLACK SHIRT AND DENIM JEANS','MA',
         'TANK TOP, JEANS / OPERATING VEHICLE OCCUPIED BY SUBJECT','BLACK T-SHIRT, BLACK PANTS',
         'GREY SHIRT, BLUE JEANS, GREY SNEAKERS','BLACK PANTS WITH WHITE LINES',
         'MALE HISPANIC DRIVING  BLACK NISSAN MAXIMA IN THE V/O OF THE ASTORIA HOUSES',
         'DARK BLUE TEE SHIRT, BLACK SHORTS, BLACK SOCKS, BLACK SNEAKERS','GRY/BLK SWEATER, BLK TIGHTS, BLUE SNEAKERS',
         'BLACK SWEATER, BLACK PANTS, BLACK AND WHITE SNEAKERS','LONG DARK JACKET','GREEN WINTER COAT WITH FUR HOOD',
         'BLACK COAT, BLACK SHADES, BLACK LUGGAGE SUITCASE','HEADSCARF','PINK HEADPHONES',
         'BLACK JEANS WHITE TOP','BLUE JEAN SHORTS WHITE TSHIRT','GREEN SWEATER, JEANS','GREEN BRACELET',
         'BLACK SWEATPANTS, BLACK BOMBER JACKET, BLACK BASEBALL HAT','RED SWEAT SHIRT,RED PANTS','JACKET MATCHES WANTED POSTER',
         'BLUE JACKET, BLUE JEANS, WHITE SNEAKERS.','LONG BLACK JACKET','WEARING ALL WHITE CLOTHING',
         'BLACK HAIR, WHITE SHIRT, BLACK SHORTS, BLACK SHOES.','WHITE HEAD PHONES','GOLD BEATS HEADPHONES','BLACK HEADPHONES',
         'RED SWEATER, BLACK BUBBLE JACKET, LIGHT BLUE JEAN, AND RED SNEAKERS','BLACK SHIRT, RED PANTS, WHITE SNEAKERS',
         'DARK GREEN BUBBLE JACKET, BLUE JEANS','BLACK JACKET, BLACK PANTS, POSESSION OF STOLEN PROPERTY',
         'BLACK SNEAKERS, BLUE PANTS, BLUE T-SHIRT','FLANNEL SHIRT AND JEANS','ROYAL BLUE SNEAKERS',
         'BLUE SWEATER, BLUE JEANS, BLACK SNEAKERS, S&P HAIR (SHORT)','WHITE T-SHIRT RIDING A BICYCLE',
         'VARSITY JACKET MATCHING DISCRPTION','N.A','BLUE TEE SHIRT','GREEN JACKET, GREY SWEATSHIRT','WEARING SWEAT PANTS',
         'BLACK SHIRT, GREY PANTS','SKINNY','RED JACKET AS DESCRIBED IN 911 CALL','BLUE KNICKS JERSEY, JEANS',
         'GLASSES, BLACK SWEATSHIRT, WHITE UPTOWNS','GRAY SWEATSHIRT, BLACK/ORANGE SWEATPANTS','BLACK  JACKET, BLUE JEANS',
         'BLACK ADIDAS SWEATSHIRT','WHITE TANK TOP, BLACK BASKETBALL SHORTS','GRAY JUMPSUIT',
         'BLACK SWEATER BLACK PANTS GRAY SNEAKERS','GARMENTS: BLACK SHIRT, BLUE JEANS',
         'WHITE NISSAN MAXIMA, WITH FLORIDA LICENSE PLATE #IJIP84','GRAY SWEAT PANTS, BLACK SWEATPANTS, BLACK SNEAKERS',
         'BRACES','BLACK BUBBLY JACKET','DARK BLUE SHORT SLEEVE SHIRT, JEANS, 6\'2"','BLACKJACKET, BLACK PANTS, BLACK SHOES',
         'DARK SHIRT','EYES','GRAY SWEATER','BLACK PANTS, BLACK JACKET, WHITE SNEAKERS','BLACK SHIRT, GRAY PANTS',
         'RED JACKET, YELLOW METAL CHAIN','GREY ARMY FATIGUE JACKET','RED TRACK JACKET','WHITE T-SHIRT, DENIM JEANS, LONG HAIR',
         'GRAY T-SHIRT AND GRAY PANTS','RED AND WHITE TRACK SUIT','BLACK JACKET AND BLACK PANTS','LARGE SIGNATURE AFRO',
         'BLACK JACKET, BLACK PANT WITH WHITE STRIPES AND BLACK SNEAKERS','LONG LIGHT HAIR','PEACE SYMBOL AND XI.XXVII.XVII',
         'BLACK SWEATSHIRT BLUE JEANS','WHITE SHIRT AND DARK COLORED PANTS','BROWN HOT','DARK PANTS',
         'GRAY TANK TOP, GRAY TIGHTS, BLACK AND GRAY SNEAKERS','BLACK JACKET WITH NUMBERS AND A CITIBIKE.',
         'BLACK PANTS AND BICYCLE','BLACK SWEATSHIRT, BLUE SHIRT','GLASSES, GOATEE','PONY TAIL, RED SHIRT ,CARGO SHORTS',
         'WEARING BLACK CANADIAN GOOSE JACKET, BLACK JEANS, BLACK SNEAKERS','PINK T-SHIRT, BLUE JEANS,',
         'GRAY SWEATER, BLACK JEANS,','DEMIN BLUE JACKET','BLACK JACKET,BLACK SWEATPANT WITH WHITE STRIPES AND WHITE SNEAKERS',
         'GREEN JACKET, WHITE SHIRT, BEIGE PANTS','BLUE TOP','BLACK JACKET, RED SHIRT',
         'PURPLE THRASHERS SHIRT, BLUE SWEAT PANTS, GREY JORDAN SNEAKERS','RED PANTS, BLACK TANK','UNKNOWN/JACKET',
         'PURPLE JACKET','BLACK COAT WITH DARK GRAY SWEAT PANTS AND BLACK SANESKERS','BLUE JACKET AND GRAY SWEAT PANTS',
         'GRAY T SHIRT GRAY SWEATPANTS WHITE SNEAKERS','N//A','CAPTAIN AMERICA SHIELD ON LEFT ELBOW','JEANS AND SWEATSHIRT',
         'GREY SHIRT AND HAS A CHAIN','BLUE BUTTON DOWN SHIRT WITH POLKA DOTS','GRAY JACKET','RED PANTS RED SHIRT',
         'GRAY DRESS','BLACK SWEATSHIRT, BLACK SWEATS, BLACK SNEAKERS.','WHITE SHIRT, BLUE JEANS, BLUE SNEAKERS.',
         'BEIGE PANTS','CLOTHING AND FLEEING IN DIRECTION IN DESCRIPTION','WHITE T-SHITRT, NIKE SWEATS , RED JORDANS',
         'WHITE DRESS WITH FLOWERS','BLUE BUBBLE COAT, BLACK AND BLUE PLAID SHIRT, BLACK AND YELLOW SNEAKERS',
         'RED SWEATER AND GRAY PANTS','WHITE SWEATER','OUTER GARMENTS','BLACK JACKSET','MULTI COLORED LEATHER JACKET',
         'CAMOFLAGED JACKET AND PANTS','BLUE JERSEY','WHITE SNEAKERS, BLACK SWEATPANTS, BLACK SWEATSHIRT','WHITE TANK TOP',
         'LIGHT BLUE SHIRT','MOLE UNDER LEFT EYE','MULTI COLOR JOGGING SIUT','HUNCH BACK','CAMOUFLAUGE JACKET',
         'LEATHER VEST OUTER GARMENT','BLACK T-SHIRT, BLACK JEANS, BLACK HEELS','RED PANTS, WHITE SHIRT',
         'BLACK SWEATSHIRT, GREY PANTS','BK JACKET + GRAY SWEATPANT','GOLDEN STATE WARRIORS BASKETBALL JERSEY',
         'BLACK SWEAT SHIRT, DARK COLOR JEANS, GREY SHOES','BLUE T-SHIRT, BLACK JEANS','BLUE SHORTS AND BLACK TEE SHIRT',
         'WINDY CITY JACKET AND BLACK SWEATS','BLACK SWEATER, CAMOFLOUGE PANTS, BLACK SANDALS','BLUE JEANS, BLACK T-SHIRT',
         'BLACK JACKET WITH BLACK AND GRAY ON THE INSIDE OF JACKET, WHITE COLORED HEADPHON','BLACK LEATHER COAT, ON SCOOTER',
         'BALCK PARKA JACKET','GREEN SWEATER, BLACK JEANS, GRAY SNEAKERS','WHITE SWEATER AND BLUE JEANS',
         'WHITE TSHIRT AND BLACK SWEAT PANTS.','WHITE TSHIRT AND BLACK SWEAT PANTS.','SWEAT SUIT',
         'BLACK SWEATER/BLACK PANTS','LIGHT SKINNED / WHITE TSHIRT // BLUE SHORTS','GREEN JERSEY, BLACK PANTS',
         'RED WHITE AND BLUE NIKE JACKET','BLUE JEANS, BLACK JACKET','BROWN SCARF','ARMY JACKET',
         'BLUE SHIRT, BLUE JEANS, BLACK SHOES.','RED SHIRT WITH STRIPES','BLACK LEATHER JACKET,BLUE JEANS',
         'BLACK SHIRT, BLUE JEANS. NO VISIBLE SCARS OR TATTOOS','BLK SWEATSHIRT',
         'WHITE T-SHIRT, CAMOUFLAGE SHORTS, BLACK SNEAKERS','BURGUNDY JACKET WITH FUR TRIMMING','YELLOW JUMPSUIT',
         'BLACK JACKET, CURLY HAIR, BLACK JEANS, BLACK PATTERNED LEATHER REEBOK SNEAKERS,','BLACK JACKET / CAMO STYLE PANTS',
         'ORANGE VEST, BLUE SWEATER W/ SCHOOL SAFETY  PATCHES, BLACK PANTS, GRAY SNEAKERS,','BLACK SWEATPANTS BLACK CLOTHING',
         'BEATEN GRAY TEE SHIRT WITH RIPPED BLUE JEANS. SEVERAL YELLOW COLORED METAL RINGS',
         'WHITE TANK TOP, BLACK SHORTS, GRAY SNEAKERS','GRAY SHIRT, DARK JEANS','BLACK VEST','GRAY JACKET BLACK PANTS',
         'ORANGE WHITE BLUE JACKET, BLACK SWEATPANTS, MAROON BEANIE, BLACK SNEAKERS.BROWN',
         'WHITE SHORTS, RED SHIRT','BLUE SHIRT BLUE JEANS','RED AND BLUE JACKET','RED JACKET, RED PANTS, BLACK SHOES',
         'BLACK PUFFY COAT, BLACK SEATPANTS, BLACK/WHITE/BLUE SNEAKERS','BLUE SHORTS WHITE SHIRT','BROWN SWEATSHIRT NO SHOES',
         'GREY T SHIRT','GREY SHIRT AND BLACK PANTS','BLACK SWEATER WITH WHITE X',
         'PUERTO RICO TANK TOP, BLUE JEANS, BLACK SNEAKERS','BLACK AND GREY SHIRT','BLACK JACKET, BLUE JEANS',
         'BLACK SHOES, BLACK JEANS AND BLACK T-SHIRT','BLUE JACKET, GRAY SWEAT PANTS, GRAY SNEAKERS',
         'WHITE TRACK SUIT','RED SNORKEL JACKET, BLUE JEANS, WHITE SNEAKERS, CARRYING GOLF CLUBS ON SHOULDER',
         'BLUE JEANS JACKET, BLUE CAP, BLUE PANTS','DIRTY WORK JACKET,','GRAY SHIRT BLACK PANTS',
         'BLK T SHIRT W / RED LETTERS "RUN" , BLK JEANS W RED BELT ,PONY TAIL','WINTER COAT - BLUE','GREY SWEATER, SCARF',
         'BLUE NIKE SWEATSHIRT, BLUE SWEATPANTS, MULTI COLORED SHOES.','TANK TOP','BLACK SWEAT PANTS, BLACK SWEAT SHIRT',
         'BLACK SWEATSHIRT, GRAY SWEATPANTS','GREY T-SHIRT, GREY SHORTS','SKELETON JACKET',
         'RED WHITE A BLUE JACKET, BLACK PANTS','DISTINCT SHAPED CHIN/JAW',
         'NAVY BLUE T-SHIRT, NAVY BLUE PANTS, DARK COLORED SHOES',
         'BLACK JACKET, BLACK JEANS, BLACK SNEAKERS, WALKS WITH A LIMP.','CARRYING BLACK BUBBLE-JACKET','N/ATHE',
         'LIGHT CREAM COLORED JACKET, BLUE JEANS, WHITE SNEAKERS','PEA-COAT','BLUE SHIRT, BLUE PANTS','ALL RED CLOTHING',
         'GREY SWEATSHIRT, BLUE SWEAT PANTS AND RED SNEAKERS',
         'BLACK BROOKLYN NETS JACKET WHITE AND SILVER STRIPES ON SHOULDERS','WHITE TSHIRT, RIPPED JEANS AND WHITE SNEAKERS',
         'BROWN JACKET, BLUE JEANS, BEARD,BLACK SNEAKERS.','BLUE JACKET, BLUE JEANS, BLACK SNEAKERS',
         'RED SWEATPANTS, WHITE T SHIRT, THIN BUILD, CLEAN SHAVEN','GREEN SHIRT AND KHAKI PANTS',
         'RED LONG SLEEVE SWEAT SHIRT','CAMOUFLAGE JACKET & BLUE JEANS',
         'CAMOUFLAGE ARMY JACKET AND BURGUNDY PANTS','NONE SHOWN','WHITE LONG SLEEVE','GREY SHIRT, BLACK PANTS',
         'BLACK SWEATSHIRT, BLACK SWEAT PANTS,BLACK AND WHITE SNEAKERS','GREY SNEAKERS, RED ADIDAS SWEATS, BLACK JACKET',
         'MALE BLACK OPERATING A BLACK BMW WITH REGISTRATION FLX1505','JEANS AND SWEATER','RED SWEAT SHIRT JACKET',
         'BLUE SWEATER, BLACK BELT, BLUE JEANS','BLACK T SHIRT WITH WHITE WRITING, BLACK PANTS','RED PUFFY WINTER COAT',
         'WHITE T-SHIRT, GREY/BLACK SHORTS, BLACK SNEAKERS','BIEGE JACKET/BLUE JEANS','TAN PANTS',
         'BLACK T-SHIRT, BLACK SHORTS, ORANGE NIKE SNEAKERS.','BLACK SWEAT PANTS/ BLACK SHIRT/ BEARD',
         'WHITE TAN TOP BLUE SHORTS AND BLACK SANDLES','WHITE SHIRT, BLUE JEANS','TEAL TSHIRT BLUE JEANS BLACK SNEAKERS',
         'YELLOW T-SHIRT, BLUE JEANS','BLACK T SHIRT, RED SHORTS, BLACK SNEAKERS, BLACK AND WHITE SOCKS',
         'BLACK JEANS, BLUE COAT, BLACK SNEAKERS',
         'WEARING A BLK SHIRT, GRY JOGGING PANTS, AND RED SHOES','THIN MUSTACHE','BLACK PANTS, BLACK SHIRT',
         'BLACK LEATHER JAKET, BLUE JEANS, WHITE SNEAKERS',
         'MUSTACHE, NAVY BLUE WAIST LENGTH JACKET WITH A RED & WHITE STRIPE ON COLLAR, BLU',
         'BLUE JACKET WITH FUR HOOD, BACKPACK','WHITE TEE , BLACK SHORTS','T-SHIRT THAT SAID SECURITY',
         'BLACK HOODED JACKET, BLACK JEAN PANTS, BLACK NIKE SHOES.','BLACK JACKET WITH HOOD',
         'BLACK PARKA JACKET WITH FUR HOOD, WITH GRAY HOODIE SWEATSHIRT, AND BLACK PANTS',
         'BLK SHOES, BLK JEANS, BLK HOODIE JACKET WITH FUR ON HOOD, BLK SCARF, BLK HAT',
         'BLACK HOODED JACKET, BROWN PANTS, BLACK SNEAKERS','GRAY T-SHIRT, BURGUNDY PANTS','BLACK JACKET AND BLUE JEANS.',
         'RED SWEATER, RED SWEATPANTS, RED SNEAKERS','WHITE SHORT SLEEVE POLO, GRAY JEANS, BROWN SANDALS',
         'GREEN JACKET, RED PANTS, WHITE SNEAKERS','BIKE HELMET','KHAKI SHORTS AND NAVY BLUE/WHITE POLO',
         'OUTTER BOMBER JACKET WITH DESIGNS','BLACK VEST, OVER WHITE AND BLACK POLO, BLACK JEANS',
         'BLACK SHIRT, DARK BLUE SHORTS','DARK BLUE BUBBLE JACKET, JEANS AND WHITE SNEAKERS','RED HAT, BLACK NIKE TSHIRT',
         'CURLY HAIR, BLACK JACKET AND BLACK PANTS','BLACK JACKET/COAT AND DARK RED SWEAT PANTS',
         'BLACK SWEATPANTS, BLUE SWEATSHIRT, BLUE AND BLACK SNEAKERS, LIGHT BLUE TEE SHIRT',
         'DARK BLUE JACKET, BLUE JEANS, MULTICOLORED SNEAKERS','LONG MESSY HAIR','OUTE GARMENTS MATCHED DESCRIPTION',
         'GREEN COAT, WITH FUR AROUND THE HOOD, BLUE JEANS.','BLUE SWEATER, GRAY PANTS, BLUE SNEAKERS',
         'GRAY GLOVES, RED SWEAT JACKET, BROWN PANAJAMA PANTS, WHITE SNEAKERS','TALL AFRO, BLUE JACKET, BLACK JEANS',
         'GREEN JEANS, BLACK SWEATER, WHITE SNEAKERS','RED TEE SHIRT, BLACK JEANS, BLACK SNEAKERS','BLUE BASKETBALL JERSEY',
         'BLACK T-SHIRT, BLACK JEANS','BLACK JACKET, BLUE JEANS, BLACK SHOES WITH YELLOW STRIPES','GRAY TEE SHIRT, BLUE JEANS',
         'BLUE BOMBER JACKET','GREEN JACKET AND GREEN PANTS','ORANGE SHIRT','BLUE TRACKMJACKET',
         'GREEN PANTS, BLACK SWEATER, BLACK AIR FORCE ONES','GREEN JACKET, BLUE SNEAKERS','RED SWEATSHIRT',
         'PURPLE SWEATER, BLACK JEANS, BLACK SNEAKERS','BROWN SHIRT, BLUE JEANS',
         'MALE BLACK, APPROXIMATELY 6 FEET TALL, WEARING A BLACK JACKET WITH GREY SLEEVES.','RED SHIRT, RED SHOES',
         'BLACK JACKET AND HAD A NEON PINK BICYCLE','GRAY AND BLUE JACKET, GRAY SWEAT PANTS',
         'GREY SWEATSHIRT, GREY SWEATS, BLACK SNEAKERS','LARGE JACKET, JEANS','WHITE T-SHIRT, BLUE JEANS, WHITE SNEAKERS.',
         '8 BALL JACKETT, BLACK JEANS, RED AND BLACK SNEAKERS','EYE GLASSES, GRAY SWEATER, BLUE JEANS',
         'GREY SWEAT JACKET, BLUE JEANS AND WHITE SNEAKERS','BLACK JACKET, BLUE JEANS, WHITE SNEAKERS',
         'GREY SWEAT PANTS, BLACK T-SHIRT','WHITE T-SHIRT/40-50 YEARS OLD','PURPLE NORTH FACE JACKET, WHITE PANTS, PONEY TAIL',
         'BLACK TRENCH COAT','RED JACKET AND RED SNEAKERS','ADIDAS JUMPSUIT','WHITE T-SHIRT, GRAY SHORTS, BLACK SNEAKERS',
         'BLACK SWEATSHIRT STATING "STARVE"','BLACK JACKET BLUE JEANS BLACK BOOTS','BLACK JEANS / DARK SWEATSHIRT',
         'BLK JEANS GREEN AND BLACK JACKET','HAIR PONY TAIL','RED SHIRT AND SLACKS','BLACK SKI JACKET',
         'RED SWEATER, BLUE JEANS RED SWEATER BLACK & YELLOW PATCHES IN HAIR','YELLOW SWEATER','BLACK PANTS, BLACK JACKET',
         'BLUE JACKET, GREY SWEATPANTS','RED SWEATPANTS, NO SHIRT, GRAY SNEAKERS','GRAY SHIRT DARK COLOR PANTS',
         'RED BULKY DOWN JACKET','RED AND YELLOW FLORAL TANK TOP','BLACK JACKET, BLACK JEANS, BLACK SNEAKERS',
         'RED SWEATER WITH R','BLUE JEANS, BLACK JACKET, GRAY SNEAKERS WHITE SWEATSHIRT',
         'WHITE SHIRT WITH STRIPES, BLACK PANTS, LIGHT SKIN','GRAY T-SHIRT, RED SHORTS',
         'BLACK SWEATER, GREY SWEATPANTS, HEAVY SET','EYE GLASSES, KHAKI PLAID SWEATER, KHAKI COLOR JEANS',
         'WHITE T-SHIRT, KHAKI SHORTS, WHITE SNEAKERS, BLACK GLASSES, WHITE METAL NECKLACE','WHITE UNDERSHIRT',
         'WEARING AL BLACK CLOTHING','SUPERMAN T SHIRT','DARK SWEATSHIRT DARK PANTS','CAMOFLAUGE SWEATER',
         'BLACK SHOES, BLUE JEANS, BLACK T-SHIRT','CREAM COAT / DARK COLORED PANTS','LIGHT SKIN MALE, LIGHT SKIN FEMALE',
         'WHITE T SHIRT BLACK SPORTS JACKET AND PANTS WEARING BLACK SHOES','BLACK SHIRT, BLACK JEANS, BLACK SNEAKERS',
         'KHAKI PANTS, SHIRT WITH BLUE AND WHITE STRIPES','BLUE SKI JACKET','BLUE SHIRT, KHAKI SHORT','MONCLER JACKET',
         'BLUE SWEATER. BLACK PANTS, MULTI-COLOR SNEAKERS','GRAY SWEATSHIRT, BLUE JEANS, GLASSES, BLACK VEST, WHITE SNEAKERS',
         'BLUE JEANS JACKET, WHITE T SHIRT, WHITE SNEAKERS, BLACK PANTS','STRIPPED BLACK& WHITE SHIRT',
         'GREEN DRESS, BLACK SANDALS','BLACK LEATHER JACKET BLACK PANTS','BLACK SHIRT WITH PONY TAIL',
         'BROWN WAIST LENGH JACKET','BLACK LETHER JACKET','PONYTAIL, BLACK TANK TOP, BLUE SHORTS',
         'BLACK SHIRT; BLACK SHORTS; BLACK SNEAKERS','CURLY HAIR','BLACK JACKET DRIVING A RED VOLKSWAGON',
         'T-SHIRT WITH THE WORDS SECURITY','RED PANTS','CAMOFLOUGE JACKET','RED SHIRT, BLACK PANTS, BLACK SHOES/SNEAKERS',
         'TAN JACKET','HAIR IN BUM GREY BRA','BLACK SNEAKERS, BLACK JEANS, GREEN SHIRT','GREEN CAMO JACKET, GRAY JEANS',
         'BLUE JEANS, WHITE SNEAKERS, WHITE T-SHIRT','BUBBLE COAT','BLK TEE, RED SHORTS, WHT SNEAKERS',
         'MALE BLACK WHITE SHIRT RED BASKETBALL SHORTS','RED PUFFER JACKET','PURPLE JERSEY','LARGE BLACK PURSE',
         'BLUE JACKET, BLACK SWEATPANTS, BLACK SNEAKERS','RED JACKET & GREY SWEAT PANTS','BLACK SHIRT BLUE SHORTS',
         'BLACK JACKET WITH PAINTING OF PEOPLE PRINTED ON IT','BLUE JEANS BLUE WINDBREAKER','BLUE SCHOOL UNIFORM',
         'WHTE T SHIRT','BLACK T-SHIRT, RED SWEATPANTS, RED HAIR','BLACK COAT, WHITE BIKE','BIRTH MARK ON BACK OF RIGHT ARM',
         'JACKET AND SWEATPANTS','BLUE WINTER JACKET, BLUE SWEAT PANTS, WHITE SNEAKERS',
         'BACK PACK BLUE JACET BLACK ADIDAS TRACK PANTS','BLACK SWEATSUIT','BLUE HOOD, BLACK JACKET',
         'BLK COAT, BLUE JEANS, DARK COLORED HAIR IN BUN','BLACK TSHIRT, BLUE JEANS','DARK SWEATRSHIRT DARK PANTS',
         'BLACK JACKET WITH BLUE JEANS AND WHITE SNEAKERS','PINK SWEATER, GREY SWEATPANTS',
         'BASEBALL JERSEY WITH NUMBER 22 ON THE BACK','BLACK PARKA COAT, CAMO SWEATPANTS','DARK HOODIE, BASEBALL CAP, SUITCASE',
         'BLUE BASEBALL HAT','DARK YANKEE CAP, WHITE T-SHIRT, SHORTS','PINK PANTS','GREEN JACKET, WHITE SNEAKERS',
         'LIGHT COLORED LONG SLEEVE SHIRT, BLACK PANTS, BLACK SNEAKERS','DARK JACKET','GRAY AND RED JACKET, CAMO SWEATPANTS',
         'BLACK JACKET, WHITE SNEAKERS','BLUE SHIRT AND GRAY SHORTS','JACKET WITH FUR HOOD','MULTI COLOR T-SHIRT, BLUE JEANS',
         'CONSTRUCTION VEST','GREEN SHIRT, BLUE JEAN SHORTS','BLACK AND RED HAIR','GREY SPORTS BLAZER',
         'BLACK SWEATER, BLUE JEANS.','SWEATSHIRT. SWEAT PANTS.','BLACK JUMP SUIT','BAGGY SWEATSHIRT, BAGGY SWEATPANTS',
         'BLACK','BLUE HOODED SWEATER AND BLACK PANTS','ORANGE SWEATSHIRT','BLUE VEST, GREY SWEATSHIRT, GREY SWEAT PANTS',
         'SWEATSHIRT','ADIDAS PANTS','BLUE SWEATS,','TAN SHIRT, BLACK PANTS, TAN BOOTS',
         'BLACK T-SHIRT, BLACK SWEAT PANTS, BLACK SNEAKERS','N/A.','BLUE T SHIRT, ARMY SHORTS, BLACK SANDALS',
         'WEARING A BLACK COAT AND BLUE JEANS','BLACK WINTER COAT','ATLANTA FALCONS FOOTBALL JERSEY',
         'RED SWEATSHIRT BLUE JEANS','BLUE/WHITE CHECKER BUTTON DOWN SHIRT. BLUE SNEAKERS. PANTS. BINDER. WORK ID.',
         'KHAKI PANTS','BLUE SWEAT SUITE','BLACK JACKET, RIPPED JEANS, RED SNEAKERS','BLK JACKET. BLK PANTS. WHITE SNEAKERS.',
         'BLUE JAYS JERSEY','BLUE SHIRT, TAN PANTS','BLACK GLASSES','WHITE SNEAKERS, BLUE JEANS, WHITE/BLUE DRESS SHIRT',
         'BLACK SHIRT, BLUE JEANS.','PARKA JACKET','CAMOFLAUGE HOODED JACKET','BLACK HOODED SWEATSHIRT, UNDER COAT',
         'GRAY HOODY, GRAY SWEAT PANTS','PINK JACKET WITH TAN SCARF','NON VISIBLE','YELLOW/RED JACKET','BLACK TRACK SUIT',
         'WHITE BLOUSE','BLACK JACKET, HAIR IN PONY TAILS','BLACK SWEATPANTS, BLACK TANK TOP, BLACK HOODED SWEATER',
         'BLACK TRACK JACKET WITH WHITE STRIPES AND BLACK TRACK PANTS WITH WHITE STRIPES',
         'HAIR STYLE PONY TAIL, BLACK JACKET.','T-SHIRT AND SWEAT PANTS','BLACK JACKET, DARK CLOTHING (PANTS)',
         'DARK GREY SWEATPANTS AND DARK GREY SWEATSHIRT','GREY SHIRT, BLACK SWEATS',
         'BLACK TSHIRT, BLACK PANTS, TWISTY HAIR','WHITE TSHIRT, BLUE SHORTS, BLACK SNEAKERS, BLACK EYEGLASSES',
         'HEAVY DARK JACKET','BLUE AIR JORDAN SNEAKERS, BLACK JACKET WITH PATCHES, GRAY HOODED SWEATSHIRT',
         'ALL BLACK OUTER GARMENTS','CAMOFLAUGE JACKE','WHITE SHIRT, BLUE JEANS, RED SNEAKERS',
         'WH SNEAKERS, BL JEANS, RED JACKET W/ CHAMPION LOGO, WH HEADPHONES','BLACK SWEATPANTS',
         'WHITE SHIRT, BLUE SHORTS, BLACK HAT AND WHITE SNEAKERS.','WHITE DRESS SHIRT, BROWN PANTS, BLACK SHOES',
         'WHITE SHIRT AND BLUE JEAN PANTS','BLUE BMW SWEATER','BLUE JEANS, WHITE T-SHIRT',
         'GREEN CAMOUFLAGE JACKET, BLUE JEANS','BLACK SKI JACKET AND BLACK PANTS','GREY TANK TOP, BLACK PANTS, WHITE SNEAKERS',
         'KHAKI JACKET','ARMY SWEATSHIRT, BLACK PANTS','WHITE T-SHIRT, CAMO SHORTS','BLACK JACKET WITH ORANGE LINER',
         'DARK BLUE WAIST LENGTH JACKET','BLACK JACKET WITH FUR ON HOOD','BLUE TEE SHIRT, BLACK SWEAT PANTS','BLACK JACKET.',
         'BLACK FUR COAT, BLACK JEANS','WEARING GREEN SHIRT','GREEN PANTS . BLK SWEATER AND BLK AIR FORCE 1',
         'BLACK OVERCOAT','WEARING ALL BLACK','HOLDING ORANGE SWEATSHIRT','BLUE SWEATSHIRT, GRAY SWEATPANTS, BAREFOOT',
         'BLACK WAIST LENGTH JACKET WITH BLACK JEANS','BLACK SHIRT, BLUE SHORTS, BLACK SHOES',
         'WHITE TEE SHIRT, BLACK SWEATPANTS','ARMY JACKET AND ARMY FATIQUE PANTS','BLACK BABBLE JACKET, & GREY PANT.',
         'BLUE JACKET BLUE SWEATER','JACKET, SHORTS','ALL GREY HOODIE & SWEARPANTS','BLACK AND RED MOTORCYCLE JACKET',
         'WHITE TSHIRT, GRAY SWEATPANTS, WHITE GLOVES','BLUE COAT, BLUE JEANS, AND GREY SENAKERS',
         'RED AND WHITE BUBBLE JACKET','BLACK SHIRT, BLACK SHORTS, YELLOW SNEAKERS','WHITE SHIRT, LONG HAIR IN PONY TAIL',
         'WHITE SHIRT BLACK SWEAT PANTS BLACK SNEAKERS','BURGUNDY PANTS','CONSTRUCTION HELMET','BLACK AND WHITE SPOTTED PANTS',
         'BLACK SWEAT SHIRT AND BLUE SHIRT','BLACK JACKET, GREY SWEATPANTS','RED PANTS AND BLACK SPEAKER',
         'RED TRACK SUIT JACKET BLUE SWEAT PANTS','BLACK KNICKS JACKET','FUR COAT','BLACK SHIRT/PANTS',
         'GREEN JACKET AND BLACK PANTS','BLACK TSHIRT / TAN PANTS','BLACK SWEATSHIRT, GREY SWEATPANTS, BACKPACK',
         'RED T-SHIRT','COOPERATIVE','LEG CAST, WHITE SHIRT, BLUE JEAN SHORTS','DARK JACKET, GREY PANTS',
         'BLUE BUBBLE JACKET,HOODIE,SCARF, RED NIKE SNEAKERS','UKN','WHITE T-SHIRT, RED VEST, TAN PANTS, WHITE SNEAKERS',
         '8 BALL JACKET, BLACK PANTS, BROWN TIMBERLAND BOOTS','GRAY SHORTS/ WHITE SHIRT',
         'MALE, WHITE, JACKET, BLACK SHIRT BLACK PANTS. WHITE LOGO ON SHIRT','BLACK SHIRT, BLUE JEANS BLACK SNEAKERS',
         'WHITE SHIRT BLUE JEANS','BLACK SHIRT, BLACK PANTS, GREY SNEAKERS','GRAY COAT ORANGE JACKET',
         'WHITE T-SHIRT DARK JEANS','BLACK PANTS, NO SHIRT, BLACK BICYCLE','BLACK AND WHITE STRIPED SHIRT, PINK VEST',
         'JACKET, HEAD SCARF','DARK COAT','WHITE JACKET','BLACK T-SHIRT, GRAY SHORTS','BLACK WITH RED SWEATER',
         'BLACK SWEATER, WHITE HAT, BLUE JEANS, BLACK SHIRT','BLACK JACKET, BEIGE PANTS','WHITE T-SHIRT,BLUE SHORTS.',
         'GRAY SWEAT SHIRT, BLUE JEANS, SHORT HAIR','BLUE JEANS,f GREY SHIRT','PURPLE PANTS AND GRAY JACKET',
         'BLACK JEANS WITH RED PAINT SPOTS, CAMO BACKPACK, BLACK JACKET WITH FUR HOOD','BROWN JACKET',
         'BLACK JACKET, BLUE JEANS, ON WHITE AND SKY BLUE HUFFY BIKE','BLACK ADIDAS SWEATPANTS BLACK SWEATSHIRT',
         'PUFFY JACKET, CHAMPION SWEATSHIRT','MONCLER TEAL JACKET','NAVY BLUE PANTS, NAVY BLUE SHIRT WITH FRITO LAYS ON IT',
         'BLUE JACKET, RED SNEAKERS','RED SHORTS AND BLACK TEE SHIRT','PURPLE T-SHIRT -BLUE JEANS',
         'BLACK JACKET, BLACK T-SHIRT ,GRAY CARGOS, YELLOW WORKOBOOTS','BLACK JACKET, GRAY SWEATER, BLACK PANTS',
         'SUSPECT STOPPED WAS WEARING A RED JACKET, BLACK JEANS, AND BLACK SNEAKERS','BLACK JEANS',
         'BLACK JACKET, BLUE JEANS, MULTICOLORED SNEAKERS','GLASSES WEARING BURGUNDY SWEAT SHIRT AND BURGUNDY SWEAT PANTS',
         'DARK PARKA JACKET','BLACK SHIRT, BLUE PANTS','GRAY T SHIRT','BLUE SHIRT AND BLUE JEANS','RED SWEATSHIRT, JEANS',
         'BLACK SHIRT, BLACK SHOES, CARGO PANTS','YELLOW VEST','WHITE T-SHIRT, GRAY SWEAT PANTS, AND WHITE SNEAKERS',
         'PONY TAIL.','BLACK JACKET, BLACK JEANS, GRAY SNEAKERS','SMALL AFRO','ORANGE SHIRTN','GREEN SHIRT AND SHORTS, AFRO',
         'BLUE BIKE, AFRO STYLE HAIRCUT','B&W SHORTS, BK SHIRT W/ TOM & JERRY, BK SNEAKERS','LIGHT COLORED SHIRT',
         'BLUE SKI JACKET AND BLUE JEANS','MAROON SHIRT, GREEN ARMY JOGGERS','WHITE JACKET, BLACK PANTS, BLACK SHOES',
         'BLACK JACKET W/ HOOD, LONG HAIR','NONE VISEABLE','RED JACKET, LIGHT BLUE JEANS','WHITE T-SHIRT BLACK SLIKY SHORTS',
         'SMALL EYES','WEARING BLACK PANTS AND BLACK HOODED JACKET','GLASSES,','BLACK PELLE PELLE LEATHER JACKET',
         'BLK SHOES, BL JEANS, BLK WINDBREAKER WITH GREY LINER, DARK SHIRT UNDER','GRAY SWEAT SUIT','GO TEE, BLACK JACKET',
         'BLUE JEANS AND WHITE T SHIRT','BLACK SWEATSHIRT, BLUE SWEATPANTS','RED SHIRT, BLACK SHORTS, BLACK SNEAKERS',
         'BLACK JACKET WITH RED AND ORANGE LINING, BLUE JEANS, GLASSES','RED DRESS SHIRT, BLACK PANTS, AND BLACK BOOTS',
         'WHITE SNEAKERS, LONG CARGO PANTS, BLACK JACKET WITH LEATHER SLEEVE, HEADPHONES','HEAVY COAT','BURGUNDY JACKET',
         'WHITE SHIRT BLACK SHORTS','EYE GLASSES, BLACK SWEATSHIRT','GREY SHIRT, BLACK JEANS, BROWN SHOES',
         'YAMAKA','BLUE SWEATSUIT','PINK/RED SWEATSHIRT','UNKNOW','BLUE JEANS, BROWN JACKET','SHORTS& TANK TOP',
         'BLUE NFL ODELL BECKHAM JERSEY','BLACK SWEATER,FATIGUE PANTS','PINK TEE SHIRT, BLACK JEANS, WHITE SNEAKERS',
         'BROWN JACKET, BLACK SWEATS','PINK CREW NECK SWEAT SHIRT & BLUE JEANS','STRIPES ON OUTTER JACKET','NONCE',
         'BLUE SWEATSHIRT','RED SWEATSHIRT AND GRAY PANTS','RED AND GREEN PLAID JACKET','GREY JACKET, GRAY SWEATS',
         'BLACK SWEATSHIRT WHITE WRITING, WHITE AIR FORCE ONES','BLUE SWEATSHIRT, CAMOUFLAGE NIKES','BLUE WINTER COAT',
         'RED BASEBALL JACKET, BLACK JEANS','GREEN SWEATSHIRT, BROWN JEANS','RIPPED JEANS, TAN JACKET','BLUE OVERALLS',
         'BROWN PURSE','BLACK TSHIRT, BLUE SHORTS, GRAY SNEAKERS','RED SWEATSHIRT, RED SHOES','CAMO JACKET, GLASSES',
         'GREEN NIKE SWEAT SUIT, BLACK SNEAKERS, EYE GLASSES','BLUE JEANS, BEIGE SWEATER, WHITE SOCKS','TAN WOOL JACKET',
         'BROWN JACKET, CAMO PANTS','GREY SWEATSHIRT','BLACK SHIRT W/ 6 ON IT.  TAN SHORTS','BLUE JACKET AND BIEGE PANTS',
         'GREY SWEATER AND GREY SWEAT PANTS','WHITE T-SHIRT, PINK/KHAKI SHORTS','BLACK CHAMPION SWEAT SUIT',
         'WHITE SHIRT WITH RED STRIPE','SHIRT COLOR','FUZZY HAIR, GRAY SWEATPANTS, BLACK JACKET WITH RED ON BOTTOM.',
         'DARK COLORED JACKET / DARK PANTS','GREEN PANTS, WHITE SNEAKERS, BLACK SHIRT WITH LOGO AND BLACK CAP WITH LOGO',
         'GREEN WAIST LENGTH JACKET, BLACK JEANS, BLACK SWEATER','WHITE T-SHIRT, BLACK SHORTS, BLACK SNEAKERS',
         'GREY NIKE SWEATSUIT','AMERICAN FLAG SHORTS, BLUE/WHITE PATTERN SHIRT','BLACK PANTS / BLACK SHIRT',
         'WHITE BUTTON SHIRT, WHITE JEANS, GREEN SHOES','GRAY SNEAKERS, BLUE JEANS, BLUE SHIRT','WHITE TSHIRT, BLACK SHORTS',
         'GREEN HOODED JACKEK, BLAK PANTS','BLUE SWEATPANTS , GRAY SWEATER ON A BIKE','WHITE T SHIRT, BLACK SHORTS',
         'BLACK JACKET WITH HOOD, SOCKET WRENCH IN HAND','DARK BROWN WINTER JACKET, PUSHED BACK BLACK AND WHITE HAIR,',
         'DARK BLUE SWEATSHIRT, BLACK SWEATPANTS','BLACK SWEATSHIRT, BLACK PANTS, BLACK SNEAKERS','GREEN/BROWN SHIRT',
         'DRESS SHIRT, SLACKS','BUBBLE JACKET, HOOD, LIGHT GREY SWEATPANTS, WHITE SNEAKERS','GREY JACKET, RED PANTS',
         'MALE BLACK, WHITE BUTTONDOWN SHIRT; 5\'4; BLACK AND WHITE SNEACKERS','BLK JACKET. GRAY CAMO PANTS. BLK SNEAKERS',
         'MALE HISPANIC WEARING RED T-SHIRT AND BLACK SWEATPANTS. FITTING ICAD #3664 ENTIR','BLACK JACKET GREY SWEATER',
         'BLUE JACKET AND KHAKI PANTS, WHITE SNEAKERS','PURPLE TSHIRT, BLACK COAT','RED PANTS BLACK JACKET',
         'ALL BLACK CLOTHING','BLACK SHORTS, GREY TSHIRT','BLACK FUR COAT WHITE PANTS','FEMALE THIN W/ BLACK JACKET',
         'UNKOWN/ SWEATSHIRT','WHITE SHIRT, WHITE PANTS','BLACK TANK TOP, BLUE JEANS, WHITE SNEAKERS, BLACK CURLY HAIR',
         'DARK/BLACK COLORED COAT WITH A HOOD','BROWN BOOTS, BIG PLAID SHOPPING BAG, BLUE JEANS','T SHIRT AND JEANS',
         'BLACK JACKET, BLACK JEANS, BLACK SHOES','ALL BLACK OUTER GARMENTS WITH BLACK AND WHITE JORDAN BRAND SNEAKERS',
         'BLUE TOP AND BLACK PANTS WITH RED STRIPES','BLACK JACKET/BLACK JEANS','BLACK JEANS, BLACK JACKET, STOCKY BUILD',
         'RED SWEAT PANTS, PURPLE SWEATSHIRT','BROWN JACKET, BLUE JEANS','BL JEANS, BLK SHOES, BLK BUBBLE COAT',
         'RED HOODIE LOGO "LIFESTYLE", BLACK JEANS AND BROWN SNEAKERS.','WHITE TSHIRT, BLUE SWEAT PANTS, RED SNEAKERS',
         'GRAY POLO, TAN JEANS.','T-SHIRT, SHORTS','SOCCER UNIFORM','GLASSES; ACCENT','RED/WHT JACKET, BLACK PANTS.',
         'BURGUNDY SNEAKERS, NAVY BLUE PEA COAT, CARGO PANTS, BALD','WHITE JACKET, BLACK PANTS','PINK JACKET,BLUE JEANS,',
         'GREEN FLEECE JACKET, BLUE SWEAT PANTS, GREEN AND BLACK SNEAKERS.','LIFESTYLE GRAY SWEATSHIRT','NAVY BLUE BUBBLE COAT',
         'NON VISABLE','BLACK DRESS, TAN PURSE','BLACK JACKET, BLACK JEANS. BLUE SNEAKERS','BLUE SHIRT, BLUE PANTS.',
         'NAVY BLUE SWEAT PANTS AND SWEAT SHIRT','BLACK SWEATSHIRT, GRAY SWEATPANTS, BLACK SNEAKERS','SPORTS COAT',
         'WHITE T-SHIRT, KHAKI PANTS.','BLUE JEANS, BLACK JACKET, BLUE SWEATER, WHITE SNEAKERS','BLACK AND BROWN JACKET',
         'BLACK CLOTHING AND WHITE SNEAKERS','BLACK SHIRT, BLACK PANTS, BLACK SNEAKERS','RED SWEATSHIRT AND BLACK SWEATPANTS',
         'PINK SWEATPANTS, WHITE TANKTOP','BLACK JACKET; PERMED HAIR','GRAY SWEATPANTS GREEN JACKET','WHITE AND RED JACKET',
         'BLACK SWEATER, GREY JEANS, AND GREY SNEAKERS','BLACK AND WHITE STRIPPED SHIRT WITH WHITE SNEAKERS',
         'BLUE JEANS, RED JACKET','TAN OUTER COAT,','SHIRT AND SHORTS','GRAY SWEATSHIRT, GRAY SWEAT PANTS',
         'BLACK SPANDEX PANTS, WHITE TEESHIRT','GREY TOP/ WHITE SNEAKERS','WHITE T SHIRT, RED SHORTS, DREDS IN A PONYTAIL',
         'GRAY SWEAT SHIRT, BLACK SWEAT PANTS','ARMY CLOTHING','WHITE T-SHIRT, MULTI-COLORED JEANS, RED SANDALS',
         'DEFT WAS WEARING BLUE JEANS WHITE SNEAKERS AND BLACK SHIRT','WHITE HAT, WHITE SHIRT, BLUE JEANS',
         'GRAY T-SHIRT, BLACK SHORTS','BLACK SWEATPANTS, BLACK AND WHITE JACKET, RED HAT, WHITE SNEAKERS','RED SHIRT',
         'BLUE JEANS BLACK JACKET','MULTI COLORED TRACK JACKET','GREEN JACKET, GRAY PANTS','DISTINCTIVE GREEN SHORTS',
         'FLORAL SHIRT, CURLY HAIR, RED STROLLER','BLACK FLEECE PULLOVER, ORANGE SHIRT, BLUE JEANS AND BLACK SNEAKERS.',
         'NAVY BLUE HOODED SWEATSHIRT','RED TEE SHIRT WITH BLACK VEST','AFRO','EYE GLASSES','BLACK COAT','RED JACKET',
         'WHITE DRESS SHIRT, BLACK TIE, BLACK SLACKS, BLACK SHIES','WHITE TANK TOP, BLACK SHORTS, GRAY SWEATSHIRT',
         'BLACK & WHITE STRIPED T SHIRT, BLUE SHORTS, BLACK SOCKS, WHITE SNEAKERS','BLACK SHOES, BLACK T-SHIRT, BLACK PANTS',
         'BLACK JACKET, EARINGS','ORANGE SWEATER','BLUE SHIRT AND JEANS','BLACK SWEATPANTS WHITE TANK TOP','RED SWEAT SHIRT',
         'GREEN LEGGINGS, BLACK JACKET, BROWN BOOTS','N/','BLUE STRIPED SHIRT','BLUE JEANS, GRAY BUTTON DOWN SHIRT.',
         'BLUE SHIRT W/ WHITE LOGO','DARK BLUE SHIRT','WHITE TANK TOP, BLUE JEANS, WHITE TANK TOP AROUND NECK',
         'BLACK T SHIRT, CAMO PANTS,  WHITE SNEAKERS','WAIST LENGTH BLACK JACKET','BLUE SWEAT PANTS','BLUE STAFF SHIRT',
         'GRAY AND BLACK NORTHFACE JACKET','RED LEATHER JACKET','GREY SWEATSUIT','BLUE JEANS BLUE JACKET','BLUE SWEATER',
         'RED T-SHIRT, RED SHORTS','BLACK SWEATSHIRT, BLACK SWEATPANTS, BLACK/RED SNEAKERS','BLUE JEANS, RED SHIRT',
         'BLK JACKET, BLACK SWEATS, BLUE SNEAKERS','BLACK SWEATSHIRT GRAY SWEAT PANTS YELLOW CHAIN','SHORT HAIR',
         'PLAID STYLED WHITE BUTTON UP SHIRT, DARK T-SHIRT, WHITE UNDERSHIRT, DARK PANTS,','BLACK SHIRT, BLACK PANTS, AFRO',
         'BLACK WAIST LENGTH JACKET, BLUE JEANS','BICYCLE','BLUE JEANS JACKET','RED SHIRT, BLUE JEANS','BUBBLE JACKET',
         'GRAY T-SHIRT, BLUE JEANS, GRAY SNEAKERS','BLACK SHIRT, BLACK JEANS','SWEATPANTS','WHITE TEE SHIRT','GRAY SWEATSHIRT',
         'BLACK BUBBLE JACKET','BLACK T SHIRT','WHITE TSHIRT','RED COAT','HOLDING AN UMBRELLA','DARK BLUE JACKET',
         'BLACK JEANS AND BLACK JACKET','EYEGLASSES','BLACK LEATHER JACKET','BLACK JACKET WITH PATCHES','PONYTAIL',
         'WHITE TANK TOP, JEANS','FRECKLES','BLACK PEACOAT','BLACK SWEATSHIRT, BLACK SWEATPANTS','BLACK JACKET BLUE JEANS',
         'RED JACKET, BLUE JEANS','BLACK SWEATSHIRT','HOODED JACKET','DARK CLOTHING','SUNGLASSES','BLUE T SHIRT, BLUE SHORTS',
         'CANE','RED SWEATER','WEARING ALL BLACK CLOTHING','MUSTACHE','READING GLASSES','BLACK SHIRT BLACK PANTS','EARRINGS',
         'LONG CURLY HAIR','BLACK SWEATSHIRT, BLACK PANTS','BLUE JEANS, WHITE SHIRT','GREY SWEAT SUIT','BLK JACKET',
         'BLACK JACKET, BLUE JEANS, BLACK SNEAKERS','BLACK JACKET, BLACK JEANS','BLACK PANTS','RED SWEATSUIT','WORK CLOTHES',
         'RED SWEAT SUIT','RED AND BLACK JACKET','PUFFY JACKET','PINK SHIRT & BLUE JEANS','WHITE T-SHIRT','JEAN JACKET',
         'BLUE JACKET','LONG HAIR','BLACK WAIST LENGTH JACKET','BLACK SHIRT, BLACK PANTS','BLACK SWEATER','BLUE JEANS',
         'WHITE SHIRT','BLACK T-SHIRT','BLUE SHIRT','PONY TAIL','BLACK JACKET','GLASSES','BLACK SHIRT','JACKET','GREEN JACKET',
         'WEARING A WHITE SHIRT & MULTI-COLORED SCARF ON HEAD','SCARF ON HEAD BLACK JACKET','ECZEMA ON FACE','ACNE','PSORIASIS',
         'SUN SPOTS ON HANDS AND FACE','SKIN CONDITION ON RIGHT HAND, MULTI COLORED T-SHIRT, BEIGE JEANS, RED SNEAKERS,',
         '(null)', 'NONE','UNK','UNKNOWN','N\A','NOT VISIBLE','NOT APPLICABLE','NONE VISABLE','NON APPLICABLE',
        'NONE VISIBLE','NONE OBSERVED','UNKOWN','NO','UKNOWN'
        ]     

bag = ['SHIP TATTOO ON NECK, GREEN BACKPACK','BLACK JACKET, BLACK SKULLCAP, BLACK PANTS, GRAY SCARF, LEATHER BROWN BAG.',
       'BLACK JACKET, H&M BAGS','MB, BLK PANTS, BLK TOP, SUNGLASSES, BLK BOOK BAG','BLACK T SHIRT, BACKPACK',
       'WHITE T-SHIRT, BLACAK BACKPACK, GLASSES','CAMOUFLAGE COLOR BACKPACK, BLUE SPEAKER, TATTOO OF STARS',
       'RED JACKET, GRAY SWEATPANT AND RED SNEAKERS WITH MCM BOOK BAG','WEARRING BACKPACK',
       'BROWN SHIRT, BLACK PANTS, CARRYING A METRO PCS BAG','ORANGE BOOKBAG','HIGHLIGHT BACKPACK',
       'WHITE SHIRT, WHITE PANTS, BLUE STRING BAG','BLUE JACKET BLUE JEANS WITH GREEN BACKPACK',
       'BROWN JACKET, ORANGE BACK PACK, CURLY LONG HAIR','BLACK BAG',
       'WAIST LENGTH DARK BLUE JACKET, DARK BLUE JEANS, WHITE SNEAKERS, GOATEE.',
       'WEARING RED SHIRT, DARK COLOR PANTS, BLACK BOOK BAG','BLACK PANTS, BLACK BACKPACK','BLACK JACKET ON ARM, SHOPPING BAGS',
       'NAVY BLUE SWEATER, BLACK SWEAT PANTS, BLACK BOOKBAG','WEARING A BLACK BACKPACK','LAUNDRY BAG',
       'YELLOW BAG, STRAIGHT HAIR','BLACK BACKPACK, JACKET, GLASSES','ADIDAS BACK PACK',
       'BLUE BACKPACK, RED BUBBLE COAT, AND BLUE JEANS','BLUE JEANS, LONG SLEEVE WHITE T-SHIRT',
       'ORANGE AND WHITE SHORTS, WHITE T-SHIRT, BLACK SNEAKERS',
       'WHITE HAT, BLACK COAT, BROWN BOOTS, RED BACKPACK, MOTORIZED BICYCLE','RED T SHIRT AND BACKPACK',
       'LOITTERING N STEPS LEADING TO ROOF IN LANDING','RED BEARD, BLACK JACKET, BLUE JEANS, GRAY SNEAKERS','BLACK CLOTHING',
       'BLACK PANTS, BLUE/WHITE SNORKEL JACKET, FLAME BACKPACK','GREY SWEATSHIRT, GREY SWEATPANTS',
       'BLACK JACKET, BLUE PANTS, HOLDING A MILK CARTON','GRAY SUIT, GRAY PANTS, BLACK SHOES',
       'RED SHIRT, RED BOOK BAG, BLUE JEANS','GRAY JEANS, GRAY SHIRT, BLACK SNEAKERS',
       'WHITE T-SHIRT, BLUE BASKETBALL SHORTS, BLACK AND WHITE SNEAKERS','BLUE SHIRT , BLUE PANTS , BLUE SNEAKERS',
       'YELLOW PANTS','WHITE T SHIRT','RED STRIPE SHIRT BLUE JEANS','GRAY SWET PANTS, BLACK JACKET, BLACK AND WHITE SNEAKERS.',
       'BLACK JACKET, BLACK SWEATPANTS, BLACK SNEAKERS','BK SWEATER WITH WRITING NIKE BASEBALL, BLUE JEANS, GRAY SHOES',
       'BLACK HIP LENGTH JACKET, GRAY SWEAT PANTS, BLUE SWEATSHIRT, BLACK CROC TYPE SAND',
       'RED JOGGING PANTS, BLACK JACKET, BLACK WORK BOOTS.','PINK SHORTS, SHIRTLESS, SANDALS',
       'PINK SHIRT, BLUE SHORTS, WHITE SNEAKERS','WEARING BLUE SWEATSUIT WITH RED STRIPE DOWN THE SIDE',
       'GLASSES, BLACK JACKET WITH GRAY SWEATPANTS','MONEY BAG IN LEFT HAND','RED T SHIRT/MULTI COLORED SHORTS.',
       'T-SHIRT AND SHORTS','BLACK JACKET, BLACK PANTS, BROWN SHOES','BLACK SWEATSHIRT, BLACK JEANS',
       'RED JACKET, WHITE SNEAKERS','BLUE POLO SHIRT; BLACK JEANS','BLACK JACKET, BLACK PANTS','RED T-SHIRT, BLACK SHORTS',
       'PURPLE SWEATSHIRT','ALL ORANGE OUTFIT','RED SOCCER JERSEY , RED SHORTS',
       'RED SNEAKERS, BEIGE BOOK BAG, BLACK JACKET, GRAY BASEBALL CAP, GRAY SWEATS.','RED JACKET, BLACK PANTS',
       'BLACK JEANS   GREY SHIRT','GRAY TANK TOP, SHORTS','OLIVE T-SHIRT, BLACK LEGGINGS, BACKPACK, BLACK SNEAKERS',
       'GRAY FITTED SWEAT PANTS','NONE TO REPORT','BLACK PANTS, BLACK SHOES/SNEAKERS, BLACK SWEATER','GREY SHIRT/BLUE JEANS',
       'RED JACKET, BLACK PANTS, BLONDE HIGH TOP PONY TAIL HAIR','GREY SHIRT, BLUE JEANS, BLACK SHOES',
       'BLACK COAT, BLACK PANTS.','HAIR PARTIALLY SHAVED WITH A BUN','WHITE SHIRT, BLUE JEANS, RED/WHITE SNEAKERS',
       'GRAY SWEAT PANTS , BLACK JACKET','BLUE JEANS, GREEN/GRAY SWEATER, GREEN SNEAKERS',
       'WHITE TSHIRT; BLACK SHORTS; LONG HAIR; MALE HISPANIC','RED PANTS, BRADED BLACK HAIR','RED TIE','CAST ON LEFT LEG',
       'GRAY SWEETSHIRT','WHITE SHIRT, RED SWEATER','WHITE T SHIRT, BLUE JEANS, SNEAKERS.',
       'BLACK SWEATS,WHITE SNEAKERS, MULTICOLORED JACKET','BLUE AND ORANGE SNEAKERS','GREEN COAT AND PINK SWEAT PANTS',
       'BLUE SHIRT, BLUE HAT, BLACK BACKPACK','BLACK BASEBALL HAT, BLACK T SHIRT, RIDING BICYCLE, WEARING BACKPACK',
       'JACKET / SWEATER , JOGGING PANTS,BACKPACK','GREY HOODIE, BOOK BAG','BLACK JACKET, DARK JEANS, BLACK BACK PACK',
       'OLDER FEMALE BLACK, WALKING WITH A CANE AND ROLLING BAG, HEAVY SET, WITH A BLUE/','BLACK BOOK BAG',
       'BLACK BACKBACK, BLACK JACKET AND BLACK PANTS','RED CAP, NAVY BLUE SHIRT, HOLDING BLACK GARBAGE BAG',
       'LONG DREADS. 2 BOOKBAGS. WEARING ARMY FATIGUE COLORS.',
       'BLACK JACKET, BLACK CAP, JEANS, BLACK BACK PACK','BLACK JACKET, LIME GREEN BACK PACK',
       'WHITE PANTS, BLACK JACKET, CROSS-CHEST BAG','BLACK PURSE, DARK SUNGLASSES','BLACK SWEATSHIRT, PURPLE BAG.',
       'GREY SWEETER, BLUE JEANS, WHITE AND GREEN SNEAKERS, BLACK BACKPACK','GREY JACKET AND BLACK BAG',
       'WHITE SHIRT, TAN PANTS, BACKPACK','GRAY HOODED SWEATSHIRT, BACKPACK, MAROON JEANS','BLUE BOOK BAG',
       'TAN JACKET, GRAY SCARF, BLACK SKULLY CAP, RED/BLACK LUGGAGE BAG.','BLACK NIKE BACKPACK',
       'GREY SWEATER, BLUE JEANS AND GREY BACKPACK','MODELLS BAG','PLUM SHIRT, BLACK BOOK BAG','BLACK JACKET WITH LUGGAGE',
       'SHORTS T-SHIRT WITH BACKPACK','WHITE CARGO SHORTS , GREY T-SHIRT, BLACK BACK PACK.',
       'BLACK FLOOR LENGTH JACKET, BLACK BOOTS, WITH BAG','BLACK PLASTIC BAG','BROWN JACKET, BLACK BAG',
       'BLACK LEATHER JACKET FUR TRIM, BLACK PANTS, BLACK NIKE SNEAKERS , 2 BAGS.','GREEN BACKPACK',
       'BLACK HOODIE, BLACK PANTS, BLUE SHOES, BLUE BOOKBAG','BLACK HAND BAG','WEARING BOOK BAG',
       'BLUE HEADBAND, BLUE SHIRT, BLUE PANTS, RED SHOES, RED BOOKBAG','BLACK SWEATSHIRT, BLACK PANTS AND BROWN BACKPACK',
       'WHITE T-SHIRT, WHITE/GRAY SHORTS (SQUARED), GRAY HAT, BROWN SHOES, BLACK BOOKBAG','BOOK BAG',
       'WHITE BUTTON DOWN SHIRT, BLACK PANTS AND CARRYING A BLUE BAG','GREEN JACKET, BLUE HAT, HAPPY BIRTHDAY BAG',
       'WITH TWO OTHER MALES WHO FIT DESCRIPTION WITH BACKPACKS','BLACK BUBBLE JACKET, CROSS CHEST BAG',
       'BLACK PANT, BLACK SHIRT, BLACK JACKET, BLACK KNAPSACK, BLACK NOVELTY UMBRELLA, B',
       'BLUE JACKET, DARK PANTS, DESIGNER BACKPACK','RED HAT, BLACK SHIRT, GREEN SHORTS, BLACK SNEAKERS AND RED BOOKBAG',
       'BLACK JACKET, BLACK PANTS, CARRYING TRADER JOES SHOPPING BAG','WEARING BLACK JACKET AND GRAY HOODIE WITH A BACKPACK',
       'BLACK SWEATER, BLUE SHORTS, RED SANDALS, BLACK AND GRAY BACKPACK','CATARACT LEFT EYE, BLACK BAG',
       'DESCRIP TALL ML BLK  WEARING GY JEANS BLK JAKCET BACKPACK. STOPPED 1 ML BLK WEAR','LONG BLACK COAT, RED SUITCASE',
       'WHITE T-SHIRT, BLACK PANTS, WHITE DURAG, BLACK BACKPACK, TATTOOS ON BOTH ARMS','MULTICOLORED BOOKBAG, GRAY HOODIE',
       'RED BOOK BAG AND BLUE JEANS','BLUE SHIRT, GRAY BACKPACK, BLACK HAT, BLUE SHORTS, BLACK SHOES',
       'BLUE SHIRT, TWO BACKPACKS','RED AND BLACK TOOL BAG','BLACK HOODIE, BLACK AND WHITE BACK PACK',
       'GREEN BACKPACK AND PONY TAIL','BROWN SLING STYLE BAG, RED JACKET TURNED INSIDE OUT HANGING OVER THE SLING BAG',
       'BLUE DU-RAG , GRAY SHIRT, BLACK PANTS, BLACK BOOK BAG','BLACK TIGHTS, RED BACKPACK','BLUE SHIRT/SHORTS/DUFFLEBAG/40\'S',
       'BLACK CLOTHING W/YELLOW BACKPACK','BLACK TOP, BLACK PANTS, CARRYING BACKPACK','BLACK JACKET, BLUE JEANS, RED BACK PACK',
       'BLACK DURAG, BLACK SWEATPANTS, BLACK SANDALS, WHITE SOCKS, BLACK BACKPACK AND GR','DARK SWEATSUITS, BOOK BAG',
       'GLASSES, GREY T-SHIRT, BLACK BACKPACK, BLUE JEANS, DARK SNEAKERS','BACKPACK WITH SNAKE ON IT',
       'BLK SNEAKERS, GREY ZIP UP HOODIE, BLUE JEANS, BLK & RED HAT, WHITE PLASITC BAG &','BLACK SWEATSHIRT, BLACK BACKPACK',
       'BLACK HOODED JACKET, GRAY PANTS, BLACK BOOTS, BLACK BAG','BLACK SHIRT, BLUE JEANS, BEIGE BAG','BACKPACK',
       'GREY SWEATER / WHITE SHIRT / HOLDING A BAG','BLACK JACKET, BLUE JEANS, CARRYING BROWN BLOOMINGDALE\'S BAG',
       'BLACK HOODED SWEATSHIRT, BLACK CAP, CAMO BACKPACK','GRAY BASEBALL HAT, GRAY SHIRT, PINK BACK PACK',
       'WEARING A FANNY PACK','GREEN JACKET, BLACK PANTS WHITE AND BLACK SHOE WITH A CAMO BOOKBAG','BLACK GAD AROUND SHOULDER',
       'WHITE SHIRT, BLUE JEANS, HEADPHONES, BLACK MESSANGER BAG','BLACK PANTS, BLACK SHIRT, GREEN BAG','RED BACKPACK',
       'BOOKBAG','BLACK BACK PACK','BLACK BACKPACK','BLACK BOOKBAG','BLACK JACKET, BLACK BACKPACK','WEARING BACKPACK',
      ]

facial_hair = ['BEARD, SUN GLASSES','MISSING TEETH, BEARD','BEARD/GOATEE, PONY TAIL, GREEN SWEAT SUIT','BEARD STUBBLE',
               'BEARD, READING GLASSES','BEARD, GRAY JACKET, WHITE SNEAKERS/ RED CHECK MARK','FACIAL HAIR (BEARD)',
               'WHITE FACIAL HAIR','BEARD, DRAY LONG SLEEVE SHIRT, JEANS','GOATEE, GLASSES',
               'FULL BEARD, GREY JACKET, GREY SWEAT PANTS, FLOP FLOPS','HANDLE BAR MUSTACHE','BEARD, RECEEDING HAIRLINE',
               'GOTEE, BLUE & GREEN WINTER COAT','BEARD, GRAY SWEAT PANTS, BLACK/DARK COLORED JACKET','WHITE BEARD',
               'GREY AND BLACK BEARD, DARK COLORED CLOTHING','BLACK JACKET, BLACK PANTS, BLACK SNEAKERS, FACIAL HAIR.',
               'BLACK AND WHITE GOATEE','GOATEE, CAMOUFLAGE JACKET','GRAY SHIRT','GOATEE BEARD',
               'RED SHIRT, PURPLE SNEAKERS, BLUE JEANS, FACIAL HAIR','FULL FACIAL HAIR',
               'FACIAL HAIR, CURLY HAIR, ALL BLACK CLOTHING','FACIAL HAIR, BLACK SWEATPANTS, BLUE JACKET',
               'BLACK JACKET, BLACK SWEATPANTS, BEARD','BEARD, WHITE BASEBALL HAT, GLASSES BLACK AND WHITE SNEAKERS',
               'BLACK JACKET, BLACK JEANS, BLACK SNEAKERS, BEARD','RED SWEATSHIRT, KHAKI PANTS, BLACK SHOES, GLASSES & A BEARD',
               'BLUE T SHIRT, GREY SHORTS, BLUE SNEAKERS, GOATEE, SHORT BLACK CURLY HAIR.','FACIAL HAIR',
               'BALD, GOATEE, GRAY SHIRT WITH PALM TREES AND GRAY PANTS','BEARD, GLASSES','BLACK FACIAL HAIR',
               'GRAY LONG SLEEVE SWEATSHIRT, BEARD W/ NO MUSTACHE','BLUE SWEATSHIRT, BEARD','GLASSES AND WHITE BEARD',
               'BEARD, PONY TAIL','BEARD ON HIS CHIN','BLACK JACKET, FACIAL HAIR','FULL BEARD','MUSTACE & GOTEE',
               'RED HAT, WHITE TSHIRT, BLUE JEANS, BLACK SNEAKERS. GOATEE','SCRUFFY BEARD','GAOTEE',
               'BLACK T-SHIRT; BLACK JEANS; GRAY SNEAKERS, BEARD','BLK KNIT CAP, BLK JACKET, BLK SCHOOLBAG, AND JEANS',
               'BURGUNDY SHIRT,BROWN SHORTS,WHITE SNEAKERS AND GREEN BOOKBAG','WHITE TOP BLK SHORTS AND BOOKBAG',
               'YELLOW BASEBALL CAP, BLACK JACKET, BLUE JEANS, BEARD','BEARD, DARK CLOTHING','UNKEPT GOATEE',
               'GOATEE, GLASSES, BLUE SECURITY GUARD SHIRT','PARTIALLY GRAY BEARD','GLASSES, BEARD','SLIGHT FACIAL HAIR',
               'WHITE TANK TOP, BLACK JEANS, BLACK SNEAKERS, BEARD','RED JACKET, GOATEE','GOATEE, READING GLASSES',
               'WHITE TANK TOP, GRAY SWEATPANTS, GRAY SHOES, BLACK GOATEE, DARK SKIN','LONG BEARD','BEARD','GOATEE',
              ]                                  

scars = ['MARKING ON RIGHT SIDE OF FACE','SCARS ON RIGHT SHOULDER','SCAR ON UPPER LIP','SCARS ON BOTH SIDES OF FACE',
         'SCAR ON BALD HEAD','LACERATIONS TO FACE AND NECK','CUT ON RIGHT HAND','SCAR ON EYEBROW',
         'FACIAL SCAR','SCAR ON LEFTSIDE FACE','SCAR IN RIGHT SIDE OF FACE','MULTIPLE HORIZONTAL SCARS ON LEFT FOREARM',
         'THREE SCRAS ON HIS NECK','SCAR ON LEFT FACE','SCAR ON LEFT FACE BLACK LEATHER JACKET BLUE JEANS WHITE SNEAKERS',
         'SCARS ON FACE, PONYTAIL, GRAY SHIRT','LARGE SCAR LEFT SIDE OF FACE','SCAR ON RIGHT SIDE OF FACE',
         'SCAR ON RIGH SIDE OF HER FACE BELOW THE RIGHT EYE','SCAR ON HIS NOSE','SCAR ON RIGHT ARM','BIRTHMARK ON FACE',
         'RIGHT EYE SCARS','SCAR ON BRIDGE OF NOSE','SCARS ON FACE AND NECK','SCAR ON LEFT EYEBROW','RR19 ON RIGHT ARM',
         'SCAR ON LEFT SIDE OF THE FACE','LEFT AND RIGHT SIDE OF FACE','LEFT SIDE OF NECK SMALL ABRASION',
         'SCAR BELOW LEFT EYE, MUTLICOLORED SHORTS, WHITE T-SHIRT','SCAR ON FACE, BEARD, GREEN ARMY FATIGUE PANTS',
         'SCAR/BIRTH MARK ON TOP LEFT SIDE OF FOREHEAD','SCAR ON RIGHT SIDE OF FACE AND RIGH TSIDE OF NECK','HAND SCAR',
         'SCAR TO RIGHT SIDE OF NECK','HEAD SCAR, DIRTY WHITE T SHIRT, BLUE PANTS','SCAR BY RIGHT EYE, TATTOO ON RIGHT HAND',
         'SCAR IN NOSE','SCAR ON CHEEK','SCAR ON RIGHT CHEEK','FACIAL SCARS , RED JACKET, BLUE JEANS , BLACK SNEAKERS',
         'SCAR RIGHT EYE','FACIAL SCARS','LARGE SCAR ON FACE','LARGE SCAR ON ARM','2 SCARS ACROSS THE RIGHT SIDE OF THE FACE',
         'SCARS UNDER LEFT EYE','RIGHT CHEEK MINI SCAR','LARGE SCARS ON LEFT ARM','SCAR ABOVE RIGHT EYE','SCAR ON RIGHT EYE',
         'SCARS TO FACE','SCARS ON LEFT LEG','BEARD, BROWN JORDAN?S','SCAR ON THE NECK','SCAR ON RIGHT SIDE OF CHEEK/NECK',
         'GLASSES AND SCAR ON IGHT INSIDE ARM','SCAR LEFT CHEEK','SCAR ON BACK SHOULDER BLADE','SCAR ON NOSE',
         'SCAR ON LEFT SIDE OF FACE (CHEEK)','SCAR LEFT SIDE FACE','FACE SCAR','SCAR ON FACE','SCAR ON LEFT SIDE OF FACE',
         'SCARS ON FACE','SCARS','SCAR RIGHT SIDE OF FACE'
        ]
                                                                          
braids = ['WHITE TSHIRT AND BLACK JEANS BRAIDS AND WHITE SNEAKERS','DREADLOCK HAIR','BRADS',
          'BLACK T-SHIRT, BRAIDED HAIR, FACIAL HAIR, BLUE JEANS','BLACK T SHIRT, BRAIDED HAIR, RED PANTS','WILD BRAID HAIR',
          'BURGUNDY JACKET, DREADS','DEADS','BRAIDED HAIR','WHITE TANK TOP; COLORED BEADS, GRAY SHORTS, WHITE SNEAKERS',
          'BLACK AND RED BEADS','NOSE RING, DREADLOCKS','BLOND TIPS ON LOCKS','SQUARED BRAIDS, CLOTHING MATCHING 911 CALL',
          'BRAIDS HAIRSTYLE','BLACK TSHIRT, WHITE SHORTS, BRAIDED HAIR','BRAIDES','LONG DREDLOCKS',
          'DREADS, WHITE SHIRT, RED SWEATER ON TOP OF HEAD, BLACK HAT ON TOP OF HEAD.','BROWN JACKET / DREDLOCKS',
          'MULTI COLORED HOODIE, JEANS, DREDLOCKS, IN BACKSEAT DRIVERS SIDE','GOATEE, DREADLOCKS',
          'DREADLOCKS, GRAY SWEATSHIRT ,CAMO PANTS,','GRAY PANTS, RED SWEAT SHIRT, SMALL BRAIDS IN HAIR',
          'WHITE T-SHIRT, DREADLOCKS','LONG CORN ROW HAIR CUT','SHORT DREDS','DREADLOCKS WITH BLONDE TIPS',
          'BLACK T-SHIRT, BLUE JEANS, BLACK SNEAKER, BLOND DREADLOCKS','DREAD LOCKS; WHITE SHIRT; BLUE JEANS',
          'GREEN SWEATSHIRT, GRAY SWEATPANTS, BLACK SNEAKERS, BRAIDED HAIR','CORN ROWS, GAOTEE','DREAD LOCK HAIRSTYLE',
          'BRAIDS, WHITE TSHIRT, RED SHORTS','MEDIUM LENGTH DREADLOCKS','WEARING RED SHIRT AND DREADS','DREDLOCKS',
          'BLACK BRAIDS, BLACK CHIN HAIR, BLACK SWEATS, GRAY SWEATSHIRT, BLACK VEST','FACIAL HAIR, CORN ROWS',
          'GREEN AND RED DREADLOCKS','DREADS ( HAIR)','BURGUNDY T-SHIRT, KHAKI SHORTS, BURGUNDY SNEAKERS, BRAIDS',
          'BLACK DREDS, GREY HOODIE, BLACK SWEATPANTS','WHITE T SHIRT, GREEN SWEAT PANTS, DREADLOCKS','DREAD LOCKS',
          'BLUE LEATHER JACKET/ SMALL DREAD BRAIDS','DREDS, RED T SHIRT. BLACK SHORTS, RED JORDANS','DREAD LOCKS BLUE SHIRT',
          'DREDS','CORN ROLLS','DREADLOCKS','BRAIDS','DREADS'
         ]

hoodie = ['WEARING A HOODIE WITH A HAT AND BLACK DICKIE PANTS.','BLK HOODIE. BLK JEANS. BLK SNEAKERS. WHITE/BLK BACKPACK',
          'BROWN HOODIE','RED SWEATSHIRT HOODIE','GRAY HOODY AND GRAY SWEATPANTS','BLUE JACKET, RED HOODY',
          'GRAY HOODIE, BLUE JEANS.','BLACK HOODIE, BLUE SWEATPANTS','BLK JACKET, BLUE NIKE HOODIE',
          'BLACK AND RED STRIPPED HOODED SWEATSHIRT ALONG WITH RED PANTS','DARK HOODIE, GLASSES','PURPLE HOODIE, BLUE JEAN',
          'RED HOODY AND GRAY SWEATPANTS','BLACK AND GREY HOODIE','DARK BLUE HOODIE, GRAY SWEATS',
          'GRAY HOODIE & GRAY SWEAT PANTS','BLACK JACKET, GREY HOODIE, GREY SWEATPANTS, WHITE SHOES',
          'POWDER BLUE COAT WITH FUR ON HOOD','BLUE HOODIE, BLACK JEANS',
          'GREY HAT, EYEGLASSES, BL HOODIE, BLK JACKET WITH FUR ON COLLAR, BL JEANS, BLK SN','BLACK HOODIE, WHITE SNEAKERS.',
          'GREY HOODED SWEATSHIRT, GREY PANTS, BLACK/WHITE JACKET','ARMY COLORED HOODIE',
          'GREY SWEATER/HOODIE, BLUE HAT','BLACK HOODED SWEATSHIRT, BLUE JEANS, BACKPACK',
          'BLACK PANTS, BLACK SHOES, BLACK HOODIE, SILVER SCOOTER WITH GREEN HANDLE BARS','BLACK HOODIE, RED SHOES, BACKPACK',
          'BLACK PANTS BLACK HOODIE BLACK SNEAKERS BLACK FITTED HAT','HOODED BLACK SWEATSHIRT, BLACKPANTS, BLACK SNEAKERS',
          'BLACK/GREY HOODIE AND JACKET','BLACK "CHAMPION" HOODIE, BLACK SWEATPANTS',
          'RED HOODY, GREY SWEATPANTS, BLUE AND RED SNEAKERS','HOODIE WITH SHARK DESIGN AND MGM WRITTEN ON BACK',
          'BLACK HOODIE AROUND NECK, DARK T SHIRT, BLUE JEANS, BLACK BACKPACK',
          'BLACK HOODIE, BLACK JEANS','CAMO JACKET WITH HOODY AND DARK PANTS',
          'BLUE BUBBLE COAT, JEANS, HOOD ON, BLACK SNEAKERS','BLACK LEATHER RED HOODY JACKET AND GLASSES',
          'BLACK SNEAKERS, DARK BLUE JEANS, BLACK JACKET, WHITE HOODIE SWEAT SHIRT',
          'CAMO SWEATPANTS, DARK HOODIE, WHITE SNEAKERS, BEARD.','RED HOODIE BLACK VEST',
          'BLACK HOODIE, BLACK SWEATPANTS, WHITE/RED FLIP FLOPS','BLUE HOODIE, BLUE SWEATPANTS',
          'RED AND BLACK PLAID SWEAT SHIRT WITH BLACK HOODY. GREEN CAMOUFLAGE PANTS','GRY HOODIE, GRY SWEAT PANTS',
          'DARK CLOTHING,BLACK HOODED SWEATSHIRT','GREEN NORTHFACE JACKET, BLACK HOODIE, BLACK JEANS',
          'BLACK COAT WITH WHITE HOODIE UNDERNEATH WITH HORIZONTAL BLACK LINES ACCROSS WITH','DARK HOODIE',
          'NAVY BLUE HOODY, NAVY BLUE SWEATPANTS, GREY/WHITE SNEAKERS','RED SNEAKES, GRAY CAP, BLUE JEANS, BLACK HOODIE',
          'BLUE HOODED SWEATSHIRT, BLUE JEANS, BROWN/GREEN BOOTS',
          'BLACK HOODY BLACK PANTS','BLACK JACKET WITH GREY HOODIE',
          'OLIVE GREEN HOODED SWEATER W THE LOGO NIKE WRITTEN ACROSS THE CHEST W/ A CHECK M',
          'OUTER GARMENT, YELLOW SWEATSHIRT HOODIE','WHITE HOODY / GREY SWEATPANTS / WHITE SNEAKERS',
          'ARMY GREEN HEAD WRAP AND BLACK CARGO PANTS WITH BLACK SNEAKERS','BLACK JACKET, GRAY HOODIE, BLUE JEANS.',
          'TAN PANTS. BLACK HOODED SWEATSHIRT','JEANS / HOODED SWEATSHIRT','BLACK HOODIE, BLACK PANTS, AND WHITE SNEAKERS',
          'BLACK JACKET BLACK HOODY','BLACK WOOL COAT, BLACK HOODIE, BLACK SWEATPANTS, AND BLACK SNEAKERS.',
          'GREY HOODIE, NIKE SWEATPANTS, BLACK SNEAKERS','BLACK HOODY SWEATER, BLACK JEANS , BLUE RUBBER GLOVES ON HANDS',
          'BLACK HOODED SWEAT TOP AND SWEAT PANTS.','DARK HOODIE , GRAY SWEATPANTS','GRAY HOODY,FATIGUE PANTS',
          'BLACK JACKET WITH RED AND WHITE STRIPES, BLACK HOODIE',
          'GRAY HOODED SWEATSHIRT, TAN PANTS, BLACK BACK PACK, WHITE MULTI COLORED SNEAKERS','GRAY HOODIE BLUE, BLUE JEANS',
          'BLACK JACKET, BLACK HOODIE','GRAY HOODY, GREY SWEATPANTS, FLIP FLOPS, GLASSES.',
          'GREY HOODIE, BLACK SLACKS, BLACK SNEAKERS','BLACK JACKET, RED HOODIE, BLACK SWEATPANTS, BLACK SNEAKERS',
          'BLACK HOODIE, BLACK SWEATS AND MULTI-COLOR SNEAKERS','RED JACKET AND RED HOODED SWEATSHIRT',
          'BEARD BLACK JACKET BLACK HOODIE BLACK CHAMPION SWEATPANTS BLACK NIKE SNEAKERS BL',
          'WHITE SHORTS, SHORT SLEEVE WHITE HOODED SWEATSHIRT.','WHITE HOODIE BLACK CURLY HAIR','GRAY SWEATPANTS BLACK HOODIE',
          'BLACK LEATHER JACKET, JEANS, HOOD','GREY AND GREEN HOODIE',
          'BLK HAT, BLK HOODIE, BLK JACKET, BLK SNEAKERS, DARK COLORED JEANS','BLACK HOODIE AND PANTS',
          'GREY HOODIE AND SWEATPANTS','BLACK HOODIE WITH FRONT POCKET','BLACK JEANS AND BLACK HOODIE',
          'GREEN HOODIE, BLUE JEANS, BLACK AND WHITE SNEAKERS','GRAY JACKET, PINK HOODIE','DARK HOOD AND DARK SWEATPANTS',
          'BEIGE HOODIE LIGHT BLUE DENIM JACKET BEIGE BLUE LACOSTE SKULLCAP DARK BLUE SWEAT',
          'WHITE SNEAKERS, BLUE JEANS, RED HOODIE,','GRREN VEST, HOODIE, GRAY AND BLACK HAT',
          'LARGE BLACK BULKY HOOD JACKET, BLACK JEANS,','BLUE PANTS, BLACK HOODIE, BASEBALL CAP',
          'MAROON HOODED SWEATSHIRT, MAROON SWEATPANTS','BLUE JACKET, BLUE PANTS, BLACK HOODIE, BLACK SNEAKERS',
          'RED JACKET BLACK HOODIE , RED HAIR, BLUE RIPPED JEANS, MULTI COLORED SNEAKERS',
          'BLUE JEANS, BLACK HOODED SWEATSHIRT AND BLACK SNEAKERS','BLUE HOOD','GREY HOODIE, BLUE JEANS',
          'GRAY HOODIE BLACK JACKET BLACK BASEBALL CAP','GREY HOODIE, WRAPPED IN BROWN BLANKET',
          'BLACK SWEATS, BLACK SNEAKERS, CAMO OUTTER JACKET, BLACK HOODIE','BLACK HOODED SWEATSHIRT AND GRAY PANTS',
          'GREEN HOODIE, BLUE RIPPED JEANS, BLUE SNEAKERS','GREEN JACKET AND GRAY HOODIE',
          'RED HOODIE, BLACK PANTS, WHITE SNEAKERS, AFRO','DARK JACKET WITH GRAY HOODIE','GREY HOODIE, GREY LEGGINGS',
          'GRAY HOODED SWEATER, BLACK FLIGHT JACKET,BLACK JEANS, TAN BOOTS',
          'GRAY HOODIE, BLUE JEANS','BLUE HOODIE AND BLACK SNEAKERS','BURGUNDY HOODY, BLACK JEANS, BRACES',
          'HOODY WITH HOOD UP WRAPPED TIGHT','BLUE HOODIE, BLUE SHORTS, WHITE SNEAKERS',
          'BLACK HOODIE, WEAVE, BLUE JEANS,YELLOW BOXCUTTER','BLACK DOWN JACKET WITH GRAY HOODY UNDERNEATH',
          'GRAY HOODIE, GRAY PANTS DRIVING KIA NY PLATE HYZ5848','RED SWEATPANTS AND WHITE HOODED SWEATSHIRT',
          'GRAY HOODED SWEAT SHIRT, BLACK PANTS','BLUE JEANS, BLACK HOODIE, BLACK SNEAKERS',
          'RED HOODIE WITH WHITE LETTERING , GRAY SWEATPANTS, GRAY SNEAKERS , AFRO','HAT, BLK JACKET, RED HOODY, BLUE PANTS',
          'DARK GRAY PANTS, GRAY HOODIE, BLACK BASEBALL HAT, RED SNEAKERS','WHITE HOODIE, BLACK JACKET',
          'RED HOODED SWEATSHIRT, BLACK JEANS, BLACK BASEBALL CAP, BLACK SNEAKERS','RED HODDIE','GRAY HOOD',
          'BLACK HOODIE, WHITE PANTS, WHITE SHOES.','BLACK HAT, BLACK HOODIE, BLACK SWEAT PANTS, BEARD.',
          'GRAY CUT OFF HOODIE, MULTI COLOR TANK TOP, DARK PANTS','GRAY HOODIE, BLACK/RED SNEAKERS, BLACK PANTS',
          'BLUE JEANS  WHITE HOODIE','GRAY HOODIE, BLACK JACKET/COAT AND BLACK PANTS','BLUE HOODIE, BLUE JEANS.',
          'GREEN HOODIE, BLACK HAIRS, BROWN EYES, BLACK PANTS','RED HOODIE W/BLK JACKET','BIEGE JACKET AND GRAY HOODIE',
          'BLACK JACKET AND GRAY HOODIE','BLACK JACKET, GRAY HOODIE','BLACK HOODIE, BLACK SWEATPANTS, FACIAL HAIR.',
          'BLACK JACKET, WHITE HOODIE, BLACK SWEAT PANTS, BLACK SNEAKERS','BLACK HOODED SWEATSHIRT, GREEN HEADBAND',
          'BLUE HOODIE, GRAY SWEAT PANTS, MULTI COLORED SNEAKERS','LIGHT GRAY HOODIE WITH LIGHT COLORED JEANS.',
          'BLK SHOES, WHITE SWEATPANTS, WHITE HOODIE','BLACK HOODIE AND CAP, BLUE JEANS, BLACK SNEAKERS','GREY HOOD',
          'BROWN JACKET, GREY HOODIE, BLACK PANTS, RED/BLACK SHOES','BLACK JACKET BLUE HOODY',
          'MATCHED DESCRIPTION OF MALE BLACK WEARING BLUE NIKE SHIRT AND BLACK HOODIE AND B','GREEN HOODY BLACK PANTS',
          'BLACK HOODIE, BLACK SHIRT, GREEN SHORTS, BLUE SOCKS, BLACK SNEAKERS','LIGHT SKIN, GREY HOODIE',
          'RED HOODIE, BLACK PANT AND WHITE AND BLACK SHOE, NO TATTOOS OR SCARS','PUFFY BLACK JACKET, DARK HOODED SWEATSHIRT',
          'BLACK JACKET, RED HOODED SWEATSHIRT, GLASSES, GREY JEANS','BROWN HOODIE BLACK SLACKS BLACK SNEAKERS',
          'WHEELCHAIR / BLACK HOODIE / BLACK JEANS','BLACK JEAN   GREY HOODIE','RED HOOD SWEATER','GREY HOODED SWEATSHIRT',
          'WEARING BLUE JACKET, BEIGE PANTS, BLACK SHOES, BLUE HOODIE, GLASSES WITH GREEN/B',
          'BRAIDS, RED HOODY, RED AND BLACK JORDANS, WHITE STRIPE PANTS','LIGHT BROWN BOOTS,TAN HOODIE,GRAY SWEATER PANTS',
          'BLUE HOODED SWEATSHIRT, BLUE JEANS, GRAY BEANIE.','BLK HOODIE, GREY NORTH FACE JACKET, BL JEANS, BLK SNEAKERS',
          'BLK HAT, BLK HOODIE, BR PANTS, BR BOOTS, BLK GLASSES','NAVY BLUE YANKEE HOODIE','GRAY HOODIE, JEANS',
          'BLUE HOODIE WITH YELLOW ON THE SLEEVES','HOODY','BLK HOODIE, BLK PANTS, RED/BLK BACKPACK, RED HAT, BLK SNEAKERS.',
          'BLACK HOODIE, BLACK JEANS, BLACK SNEAKERS, BLACK GLASSES, BLACK BICYCLE','RED HOODIE, BLACK JACKET, DARK PANTS',
          'BROWN JACKET, BLACK HOOD, BLACK JEANS.','RED HOODIE, BLUE JEANS','BLUE HOODY AND GRAY VEST',
          'BLACK JACKET, RED HOODIE, NAVY PANTS','RED HOODIE, RED JEANS, BLACK OVERCOAT.','BLACK HOODIE, GREY PANTS',
          'WHITE HODDY','HOODIE GREY AND RED','NAVY BLUE HOODY, BLACK JEANS, BLACK SNEAKERS','YELLOW HOODY, BLACK PANTS',
          'BLACK JACKET BLACK HOODIE','BLACK JEANS, WHITE AND BLUE SNEAKERS, BLUE JACKET, BLUE HOODIE WITH BIG DOG WRIT',
          'BLACK HOODIE, BAND AID ON NOSE','GRAY HOODIE, BLK SWEATS BLK SNEAKERS','BLACK HOOD','HOODY TOP WITH SHORTS',
          'ARMY FATIGUE HAT, GREISH SUIT BLAZER W BLACK HOODIE UNDERNEATH BLACK DENIEN JEAN','GREY HOODIE',
          'RED AND BLACK JACKET, BLACK HOODIE, DARK RED SWEATPANTS, RED AND WHITE SNEAKERS,',
          'FACIAL HAIR, TALL, BLACK HOODIE, BLACK ADIDAS PANTS AND WHITE AND BLACK SNEAKERS',
          'BLUE JEANS, GREY AND GREEN HOODED SHIRT.','HOODED BLACK JACKET WITH FUR HOOD','DARK HOODED SWEATER, BLUE JEANS',
          'BLACK AND WHITE HOODY BLACK PANTS','DARK COLORED HOODIE, DARK COLORED PANTS','GREY AND BLACK HOODIE, FACIAL HAIR,',
          'BLK HOODIE W/ WHITE PRINTS, BLK PANTS, BLK BEARD, BLK SHOES','GRAY COAT/BLACK HOODY','GREY HOODIE, SWEAT PANTS',
          'BLACK JACKET, BLACK HOODY.','BLACK HOODIE, BLUE JEANS,MULTI COLOR SNEAKERS','GRAY HOODIE DARK/BLACK JEANS',
          'BLACK HOODED SWEAT SHIRT, BLACK SWEAT PANTS','GREY BEARD, BLACK HOODIE SWEATSHIRT, BLACK BASEBALL CAP.',
          'GREY HOODED SWEATSHIRT, GRAY SWEATPANTS, WHITE SNEAKERS','WEARING DARK HOODY. DARK COLORED SWEAT PANTS.',
          'BLUE JEANS AND BLACK HOODIE','GREEN HOODIE, BLUE JEAN JACKET, CAMO HAT','BLACK HOODIE/BLACK BACKPACK',
          'WHITE HOODY, BLUE SHIRT, BLUE SHORTS, STANDING NEXT TO WHITE VEHICLE','BLACK HOODY, BLUE JEANS, FRIZZY HAIR',
          'DARK HOODIE, DARK SWEAT PANTS','MAROON HOODIE AND BLUE JEANS','RED JOGGING PANTS, BLACK HOODIE CURLY RED HAIR',
          'RED HOODIE, GRAY SWEAT PANTS, AND BLUE AND RED SNEAKERS','RED HOODY','RED HOODED SWEATER',
          'GRAY PANTS BLACK JACKET RED HOODED SWEATSHIRT BLACK SKI CAP','CAMOFLAUGUE JACKET, BLACK HOODIE',
          'BLACK GLASSES, RED AND BLACK HOODED SWEATSHIRT, RED AND BLACK SNEAKERS, BLUE JEA','BURGANDY JACKET, GRAY HOODIE',
          'WHITE HOODED SWEATSHIRT','BLACK JOGGING HOODIE, BLACK JOGGING PANTS, WHITE SNEAKERS','ORANGE HOODIE',
          'GRAY HOODED SWEATER SIMILAR TO PHOTO SHOWING WANTED PERSON WEARING AS WELL AS BR','BLACK HOODY',
          'BLACK HOODIE, BLACK JEANS, BLACK BASEBALL CAP, BLACK GLASSES','GRAY HOODIE BLACK SWEATPANTS BLUE SNEAKERS',
          'GRAY HOODIE, BLACK SWEATPANTS, BLACK SNEAKERS WITH THE WORDS AIR ON THEM','RED SWEATPANTS & BLACK HOODED SWEATSHIRT',
          'BLUE HOODIE, BLUE JEANS, BLACK SANDALS','BLACK JACKET; RED HOODIE;','RED USA SHIRT GRAY PANTS BLACK HOODIE',
          'ARMY FATIGUE HOODIE, BLUE JEANS, BLACK SNEAKERS','BLACK HOODIE, RED JACKET','BLUE HOODIE','BLACK HOODIE',
          'RED HOODED SWEAT TOP AND PANTS WITH WAIST LENGTH DARK COLORED JACKET','GRAY HOODIE','RED HOODIE',
          'WHITE SHOES, WHITE SOCKS, BLK SWEAT PANTS, BLUE JACKET, WHITE HOODIE','HOODED SWEATSHIRT',
          'PURPLE BASEBALL CAP COLORADO ROCKIES, EYE GLASSES, GRAY HOODED SWEATER, BLACK JA','HOODIE',
         ]                                                                                                                                      

hat = ['BROWN HAT','BASEBALL CAP, BLACK HOODY LONG COAT','BLUE YANKEES BASEBALL CAP','RED HAT, BLACK T-SHIRT, DARK JEANS',
       'NAVY BLUE HAT','BLACK JACKET AND BLACK HAT','BLACK BEANIE','RED JACKET, DARK COLORED HAT.','BASEBALL CAP, GREEN COAT',
       'GRAY OVERCOAT, BLUE JEANS, GRAY SKULL CAP','BLUE SWEATSHIRT, GRAY WINTER HAT,BLACK JEANS, WHITE SNEAKERS',
       'RED HAT, BLACK JEAN JACKET, YELLOW FANNY BAG, BLACK PANTS WITH YELLOW STRIPES','BASEBALL CAP',
       'GRAY KNIT CAP, BLACK JACKET, BLUE SWEAT PANTS AND GRAY SNEAKERS.','RED HAT / BLACK PANTS /DEMI JACKET',
       'BLACK JACKET, BLACK JEANS, BLACK SNEAKERS, BLACK HAT WITH GOLD LETTERING. FACIAL','WHITE TOP, BLACK JEANS, BLACK HAT.',
       'BLUE AND WHITE STRIPED POLO, BLUE YANKEE BASEBALL CAP, BLUE JEANS',
       'BL SHOES WITH RED LACES, BLK SHORTS, WHITE T-SHIRT, BLUE HAT WITH WHITE LOGO','GREY JACKET, BLACK HAT',
       'MULTI COLOR SHIRT, BLACK HAT, AND BLUE JEANS','BLACK JACKET, BLUE JEANS, BLACK HAT',
       'BLACK SHIRT, TAN SHORTS, BASEBALL HAT','HARD HAT AND GREEN VEST','EYE GLASSES AND BLACK SKULL CAP',
       'RED HAT, DARK JACKET, BLACK JEANS, BROWN BOOTS',
       'GRAY HAT, NAVE BLUE SWEATER WITH YELLOW LETTERING "SECURITY", BLACK JEANS, GRAY','RED BEANY HAT',
       'BLACK JACKET, GREY SWEATPANTS, BLACK HAT, BLACK SHOES','SKULL CAP, BROWN JACKET, BLUE JEANS',
       'BLUE BASEBALL HAT, BLUE T SHIRT, CAMOUFLAGE SHORTS','BLUE JEANS, BLACK JACKET, BASEBALL CAP, BROWN BOOTS',
       'BLACK WINTER COAT, AND HAT','BLACK JACKET, BLACK TANK TOP, BLACK JEANS, BLACK SNEAKERS, BLACK HAT',
       'HAT AND JACKET','GREY HAT','BLACK JACKET, BLACK BASEBALL HAT','RED HAT, RED AND WHITE SNEAKERS',
       'RED NY YANKEES BASEBALL HAT','NO SHIRT, RED/YELLOW/GREEN "JAMAICAN HAT", BLACK SHORTS, BLACK BICYCLE WITH PUER',
       'BLACK TANK TOP, BLACK BASEBALL CAP','BLACK SHIRT, YELLOW METAL CHAIN, GREEN CARGO SHORTS, BLACK SNEAKERS, BLACK HAT',
       'BLACK HAT, BLACK SHIRT, BLUE PANTS, BLACK SNEAKERS','YANKEES BASEBALL CAP','ORANGE SHIRT, BLUE JEANS, BLACK HAT',
       'JEANS, PURPLE BEANIE','BLACK WAIST LENGHT JACKET AND SKULL CAP WTH FURRY BALL ATTACHED','EYE GLASSES AND SKULL CAP',
       'BLACK HAT, BLACK JACKET, BLUE JEANS','BASEBALL HAT, GRAY T-SHIRT, GLASSES, CAMO SHORTS, SHORT BEARD','COW BOY HAT',
       'PINK HAT, PINK SWEATER','RED HAT, BLACK SWEATSHIRT WITH ASSORTED COLORS ON FRONT, GLASSES, BLACK SWEATPAN',
       'MOHAWK HAIR CUT, GREY BEANIE, BLACK JACKET, BLUE PANTS','BLK HAT, GRAY SWEATER W/ ORANGE LETTERING',
       'RED HAT, BLUE SWEAT SHIRT, BLUE JEANS AND RED SNEAKERS','KNITTED CAP','BRIGHT GREEN HAT',
       'BLACK HAT WITH RED LETTERING, BLACK SHIRT WITH WHITE LETTERING','BLACK BASEBALL HAT','BLACK JACKET BLACK HAT',
       'BLACK HAT, BLACK JACKET, BLACK PANTS, BLUE SNEAKERS, LEOPARD PRINT PURSE','BLACK HAT, DARK CLOTHING, STORE EMPLOYEE',
       'BLACK SHIRT, WHITE BASEBALL HAT','YANKEES HAT','GREEN JACKET, MULTICOLORED KNIT CAP.',
       'GLASSES AND BEARD AND BLUE HAT SWEATER','HAT, TAN SHIRT','BLACK CAP AND DARK COLORED SHIRT',
       'GREEN BASEBALL HAT; BLUE JEANS, GREY SHIRT','PURPLE HAT, RED JACKET','GRAY COAT, WHITE CONSTRUCTION HAT, BLACK JEANS',
       'BLACK PANTS, BLACK TOP, RED SNEAKERS, RED HAT','STRAW HAT','BLACK JACKET WITH FUR, BLACK/RED HAT',
       'RED HAT, BLACK PANTS, RED SNEAKERS','BLACK HAT,BLACK COAT,BLUE JEANS,BLACK SNEAKERS','CANE, BLACK HAT, BLACK JACKET',
       'ARMY FATIGUE SHORTS, GREEN T-SHIRT, GREEN HAT','BLACK HAT AND BLACK SHIRT',
       'BLACK TANK TOP, NAVY SHORTS, BLACK/BLUE HAT','RIPPED BLACK JEANS WITH PAINT MARKS, BLACK JACKET, GREY HAT',
       'BLACK SHOES, BLUE JEANS, BLACK T-SHIRT, BLACK BASEBALL CAP WITH CHICAGO BULLS LO',
       'BLACK SWEATER, BLUE JEANS, BLACK HAT','WHITE T-SHIRT, BLACK PANTS, BLACK SNEAKERS','BLACK HAT, BLACK SHIRT, BLUE JEANS',
       'BLACK BEANIE HAT WITH GRAY POM POM AND GRAY LOGO, DARK COLORED BUBBLE JACKET','BLUE JACKET, GRAY JEANS, BLUE HAT',
       'HAIR WRAP','BLACK SHIRT, BLUE JEAN SHORTS, BLUE HAT','RED HAT, WHITE SHIRT, BLUE JEANS, RED SNEAKERS',
       'BLACK JACKET, BLUE PANTS, BLACK BEANIE','BROWN HAT, BROWN VEST','YANKEE HAT,  MUSTACHE, ACNE',
       'DARK BUBBLE JACKET, BASEBALL CAP. WALKING WHITE DOG','RED TURBAN, GREEN JACKET, BLACK BOOTS, BLUE JEANS, BLACK BEARD',
       'BLACK SHIRT, BLACK PANTS, BLACK COWBOY HAT','GRAY HAT','NEON GREEN HAT','WHITE PANTS, BURGUNDY HAT WITH LETTER "P"',
       'BLUE BASEBALL CAP, RED AND GRAY SWEAT SHIRT AND SWEAT PANTS, BLACK GLASSES','HAT, WHITE SHIRT, AND SHORTS',
       'GREEN SWEATER, JEANS, WORK BOOTS, SKI CAP','WHITE SHIRT, WHITE SHORTS, YANKEE BASEBALL CAP',
       'SAND COLORED KNIT SKULL CAP, BLUE JEANS, BROWN BOOTS, BLACK FADED DOWN JACKET WI',
       'BLACK LEATHER COAT WITH PATCHES, BLACK PANTS, BLACK BOOTS, BLACK HAT',
       'BLUE HOODIE,GRAY SWEATER PANTS, LIGHT GREEN SNEAKER,GREEN HAT','BLK SHOES, BLK PANTS, BROWN JACKET, BR HAT',
       'BLACK JACKET, BLACK SHIRT, BLACK PANTS, WHITE CAP','BLACK HAT, BLACK BOOTS',
       'BLACK SHIRT, BLACK JEANS BASEBALL CAP, GREEN UNDERWEAR','DENIM JACKET, BLUE JEANS, GOLD SNEAKERS, BLACK HAT',
       'BLUE SWEATER, BLUE KNIT HAT, BLACK BOOTS, GRAY JEANS, BLACK BEARD','WHITE SHIRT, BLACK HAT, BLUE JEANS',
       'BROWN COAT & SKULLY CAP','GREEN WORK VEST AND HARD HAT','BLACK NORTH FACE JACKET, WHITE HAT',
       'RED/BLK HAT, PINK SHIRT','BLUE HAT, DARK JACKET, BLUE JEANS','BLUE HAT, RED JACKET, GREY SWEAT PANTS',
       'PINK SOCKS, FLIP FLOPS, BLUE SHIRT, WHITE RAIN JACKET, YANKEES DARK BLUE BB CAP',
       'BLACK SHIRT, BLACK HAT, BLUE JEANS, BLACK SNEAKERS','RED JACKET, BLACK HAT','BLUE NY GIANTS HAT',
       'BLACK SKULL HAT, BLACK JACKET, TIMBERLAND BOOTS','BLACK SWEATSHIRT WITH A BLACK SKULL CAP AND TIMBERLAND BOOTS',
       'GRAY HAT, BLACK BASKETBALL JERSEY','WEARING BLACK BASEBALL SOX HATE WITH \'B\' ONFRONT OF THE HAT',
       'BLACK TANK TOP, SUNGLASSES, BLACK HAT, BLUE JEANS','BLACK PANTS, BLACK/RED BASEBALL CAP, BLACK SWEAT SHIRT.',
       'ORANGE BASEBALL CAP, BLACK SWEATER, BLACK SHORTS, WHITE SOCKS, BROWN SNEAKERS','PINK SHORTS, GREY HAT, NAVY TEE SHIRT',
       'BLUE HAT, GREY SHIRT, BLACK AND WHITE SNEAKERS','GLASSES, FORWARD HAT, BLACK SHIRT','HAT, SWEATSHIRT, SWEATPANTS',
       'BROWN HAT AND BROWN JACKET','GREY BUBBLE WINTER COAT, BLACK FLEECE PANTS, BLACK SKULL CAP, BLACK SNEAKERS, VE',
       'BLACK SHIRT, BLACK HAT, DARK BLUE JEANS, BLACK SNEAKERS','BLACK SKULL CAP, LEATHER JACKET',
       'BLUE JEAN SHORTS, BLUE SHIRT, RED AND BLUE HAT, BLACK SNEAKERS','BLACK JACKET, BLACK HAT, FACIAL HAIR',
       'NAVY BLUE HAT, GRAY SHIRT, BLUE JEANS, WORK BOOTS',
       'BLUE T-SHIRT, CAMO PANTS, WHITE BASEBALL HAT','BLUE LONG SLEEVE T SHIRT, GREY SWEAT PANTS, BLUE SNEAKERS, BLACK HAT.',
       'BLACK HAT, WHITE T-SHIRT','BLACK HAT, RED BUBBLE JACKET, BLUE JEANS, WHITE SNEAKERS','BLUE HAT, BLUE SHIRT, DARK PANTS',
       'BLACK CLOTHING, BLACK WINTER HAT.','GREY BEENIE HAT WITH BLACK JACKET DARK GREY PANTS BLACK SHOES ANSWERED TO THE NA',
       'BROWN JACKET, BLUE SWEATSHIRT, RED HAT, BROWN TIMBERLANDS','BLACK BEANIE HAT / BLACK TRENCH COAT','BLACK  HAT',
       'GREEN JACKET WITH BLACK KNIT HAT, HAD AMERICAN FLAG PATCH ON RIGHT SIDE','RED HAT AND RED T SHIRT','BASE BALL HAT',
       'BLUE JACKET, RED WINTER HAT','RED HAT, GREY SWEATER','ORANGE T-SHIRT, GRAY BASEBALL CAP, RED PANTS.',
       'WHITE T-SHIRT, BLACK BASEBALL HAT, BLACK PANTS, BLACK SNEAKERS','BLACK BOOTS, DARK BLUE JEANS, BLACK JACKET, GRAY HAT',
       'GRAY T-SHIRT, STRIPED BLUE RUNNING PANTS, BLUE BASEBALL HAT','MB WHITE HAT, RED SHIRT, LIGHT BLUE JEANS, RED SNEAKERS',
       'RED SWEATER VEST , BASEBALL HAT , BLUE JEANS, BLUE SHOPPING BAG, BLACK BACK PACK',
       'BLUE AND WHITE POLO SHIRT, JEAN SHORTS, BLACK SNEAKERS, BLUE HAT','BLUE HAT, STRIPED BLUE AND WHITE SHIRT',
       'BLACK JACKET, WINTER HAT, ADIDAS SWEATPANTS','BEARDED MUSTACHE, LONG GREEN JACKET, BLUE JEANS, BLACK HEAD WRAP.',
       'BLACK LEATHER JACKET, JEANS, BLACK HAT','BLACK HOODED JACKET, BLACK SKULL CAP','DARK BLUE HAT',
       'GREEN VEST,  HOODIE, GRAY AND BLACK HAT','BLACK SHORT SLEEVE HOODIE, PURPLE YANKEE HAT','BLACK HAT, BLACK JACKET',
       'BLUE SWEATSHIRT, DARK JEANS, BLUE BASEBALL HAT, WHITE SNEAKERS','RED HAT, BLUE/WHITE SHIRT/BLUE JEANS',
       'BLACK PANTS, BLACK JACKET, BLACK BLASEBALL CAP','BLUE HAT, YELLOW SEAT SUIT','LIGHT COLORED HOODED,BLUES AND DARK HAT',
       'GREEN COAT BLUE PANTS TAN SKULL CAP','ORANGE CAP, ALL BLACK CLOTHING, ORANGE SNEAKERS',
       'BLACK BASEBALL CAP, BLACK SHIRT, BLUE JEANS, BLACK SNEAKERS','BROWN COAT, BLUE JEANS, MUSTACHE, BROWN SKULL CAP',
       'WHITE JACKET, ASSORTED COLOR SHIRT AND HAT. BLACK SWEAT PANTS','BLACK JACKET, BLACK HAT, BLUE JEANS, BLACK SNEAKERS',
       'BLACK BEANIE, BLACK TRACK JACKET, BLACK SWEATPANTS, BLACK AND WHITE SNEAKERS','RED JACKET, DARK COLORED HAT',
       'BLACK TEE SHIRT, BLACK PANTS, NAVY SHOES, BLACK BASEBALL HAT, SILVER SUNGLASSES.','BLACK KNIT HAT , BLACK JACKET',
       'BLACK JOGGER PANTS, BLACK JACKET, BLACK HAT','BLACK SWEATSHIRT, BLACK PANTS, BLACK HEADWEAR','LEOPARD HEAD WRAP',
       'NO SHIRT, BLUE JEANS, BLUE HAT','YANKEES HAT, RED SHIRT, BLUE JEANS, BLACK SNEAKERS',
       'BROWN JACKET, GREEN CAP WITH YANKEES LOGO','BLK BASEBALL CAP, BLK HOODIE, BLK JEANS, BLK/WH CONVERSE, BL/GR JACKET',
       'BURGUNDY HAT, BLACK JACKET, BLUE JEANS.','BLACK LEATHER JACKET, BLACK CAP','WHITE T-SHIRT BLUE JEANS BUCKET HAT',
       'BUCKET HAT','BLUE SHIRT, BLACK BROOKLYN HAT, BLUE JEANS, WHITE SNEAKERS','GREEN JACKET, LIGHT GRAY HAT,',
       'WHITE LEGGINGS, GRAY SHORTS , BLACK HAT, WHITE LONG SLEEVE','BOWLER HAT, BLUE SHIRT','SHIRTLESS, CAP ON BACKWARDS',
       'GRAY HAT, BEIGE JACKET','YELLOW SKULL CAP','BLACK CLOTHING, WHITE AND BLACK HEAD WRAP',
       'BLACK JACKET, BURGUNDY PANTS, BLACK HAT','TAN BASEBALL CAP, BLACK T-SHIRT, GRAY SHORTS, GRAY AND WHITE SNEAKERS',
       'GRAY BASEBALL CAP, BLUE JEANS, RED JACKET, LIGHT SKIN, TALL, FACIAL HAIR','BLACK NET HAT, GLASSES, BLUE JACKET, JEANS',
       'BLACK HAT, BLACK SHIRT, BACKPACK WITH RED TRIM','DARK BLUE COAT, BLUE JEANS, BLACK WOOL HAT.',
       'BLUE HOOD JACKET, KHAKI SHORTS, GLASSES, BLUE HAT','BASEBALL CAP, GREY T-SHIRT, SHORTS AND PURPLE SOCKS',
       'BLACK AND WHITE BASEBALL HAT','GRAY HAT, GRAY SWEATER','BLACK HAT AND BLACK JACKET','BLUE HAT','BLACK BUCKET HAT',
       'BLACK GREEN HAT BLACK SWEATS','GLASSES, BLUE TRACKSUIT AND HAT','ORANGE SHIRT BLACK PANTS BLACK CAP',
       'SALT & PEPPER BEARD, BLACK BUBBLE JACKET, BLACK SWEATSHIRT, GREY WOOL HAT','SKULL CAP, GLASSES',
       'BLACK JACKET, GREEN PANTS, BLACK SNEAKERS, BLACK BASEBALL HAT','RED HAT,  FACIAL STUBBLE, WHITE T SHIRT, PLAID SHORTS',
       'BLACK JACKET, BLACK SKULL CAP, BLACK PANTS','BLACK HAT','HAT','BLACK JACKET, BLACK HAT','BLACK BASEBALL CAP'
      ]                                   

du_rag = ['WHITE DU-RAG, SUNGLASSES','GREEN DU RAG / BLACK HOODY / BLACK SWEATPANTS / BLUE SNEAKERS','ORANGE DOO RAG',
          'DU-RAG, DARK BLUE T-SHIRT, BLACK SHORTS, SLIDES','RED ADIDAS JACKET, BLACK PANTS, DU RAG',
          'OUTER GARMENTS: BLUE BUBBLE JACKET, BLACK JEANS, BLACK STOCKING HEADCAP','PURPLE DURAG, GRAY HOODIE, BLACK JEANS',
          'YELLOW JACKET, YELLOW DU-RAG, BLUE JEANS, WHITE SNEAKERS','PINK DURAG, GREY T SHIRT BLUE JEANS. PREGNANT',
          'GREEN CAMO COAT, MAROON DU RAG','WHITE DURAG, RED JACKET, BLUE JEANS','RED DU RAG. RED SNEAKERS',
          'RED DURAG, GRAY SWEATPANTS','BLUE DU RAG, WHITE T SHIRT','DU-RAG','YELLOW DU-RAG',
          'DURAG ON HEAD','GREEN CAMOUFLAGE T-SHIRT, GREEN JEANS, GOLD DU-RAG, CARRYING A BLACK BACKPACK.',
          'BLACK JACKET, BLACK DURAG, BLUE JEANS, BLACK SNEAKERS, BACKPACK','BLCK DURAG','BLUE COAT, BLUE JEANS, BLUE DURRAG',
          'NEON GREEN DU RAG','BURGUNDY DURAG','WHITE DU-RAG','BLUE DURAG, BLACK SHIRT, BLACK SWEATS',
          'BLACK LONG SLEEVE SHIRT/ BLACK PANTS/ BLACK DU RAG','PURPLE DURAG, GRAY SWEATER, BLACK SWEATS',
          'BLACK DOORAG,RED SWEATER,GREY SWEAT PANTS,WHITE SNEAKERS','DURAG ON TOP OF HEAD',
          'GREEN DURAG, WHITE HEAD PHONES, BLACK JACKET AND BLACK BACK PACK AND PULLING A B','WEARING BLUE DURAG',
          'BLACK SWEATSHIRT AND RED DU-RAG','RED DU RAG, BLACK SHIRT, BLACK PANTS','DUDERAG',
          'BLACK DURAG, GREY HAT, READING GLASSES AND BLACK T SHIRT','WHITE DURAG, GRAY AND WHITE HOODIE WITH "SAVAGE" ON FRONT.',
          'WHITE TANK TOP, DARK GRAY SWEATPANTS, BLUE DU RAG. WHITE SNEAKERS','BLK JEANS, BLK JACKET, PINK DURAG',
          'BLACK JACKET,BLACK JEANS, BLACK DU-RAG','RED DURAG , BLACK SWEAT PANTS , GRAY FLIP FLOPS',
          'PURPLE DURAG, NO SHIRT, BLACK JEANS','GRAY HOODY, GRAY SWEATS, PINK DOORAG','DURAG, BACKPACK',
          'BLACK TSHIRT // BLUE DURAG // DARK JEANS','GRAY DU-RAG, WHITE T-SHIRT, BLUE JEANS, WHITE SNEAKERS',
          'GREEN COAT, BLACK/GRAY SHIRT, WHITE SNEAKERS, BLACK DU-RAG','MULTICOLORED DU-RAG','WHITE DURAG, RED SHIRT',
          'GRAY SWEATER, BLACK SWEATPANTS, BLACK SNEAKERS, BLACK DURAG','ALL BLACK ,RED /MAROON DURAG','DU RAG',
          'BLACK HOODED SWEATSHIRT, BLACK SWEATPANTS, BLACK DURAG, BLACK AND GOLD SNEAKERS','RED DURAG',
          'BLACK DOORAG, BEARD, BLACK ADDIDAS PANTS WITH WHITE STRIPES, SWEATY DARK GREY SH','PINK DU-RAG ON HEAD',
          'BLACK SWEATER, BLACK PANTS, WHITE T-SHIRT, MULTICOLERED DU-RAG','RED DU-RAG','GRAY SWEATSUIT AND UNK COLOR DU-RAG',
          'RED SWEATSHIRT, BLACK PANTS, RED AND WHITE SNEAKERS, BLACK DURAG','WHITE T-SHIRT, WHITE DURAG, GREY SWEATPANTS',
          'WHITE CAP, RED DURAG, BLACK SHIRT, BLACK SHORTS, BLACK SNEAKERS','PURPLE DORAG','YELLOW DURAG','DURAG',
          'RED DURAG, BLACK JACKET, BLACK PANTS, GRAY SHIRT WITH WHAT APPEARED TO BE BLOOD','DARK BLUE JACKET, WHITE DU RAG',
          'WHITE DURAG, CAMO PANTS, RED BOOTS, GRAY SWEATER, FACIAL HAIR,','PURPLE DU-RAG','ORANGE DU-RAG','RED DOO-RAG',
          'BLACK PANTS, WHITE TANKTOP, WHITE AND BLACK SNEAKERS, BLACK DURAG','RED AND WHITE JACKET, BLACK JEANS, RED DURAG',
          'BLACK DU-RAG, BLACK SHIRT, BLACK PANTS, TATTOOS ON BOTH INNER FOREARMS','BLACK DU-RAG','PINK DURAG',
          'BLACK D-RAG, BLUE SNEAKERS, BLUE JEANS, WHITE TANG TOP, ARM TATTOOS','BLACK DU RAG','PINK DU RAG',
         ]

tattoo = ['TATTOOS ON RIGHT AND LEFT FOREARM; TATTOO ON RIGHT BICEP','ROSE ON ADAMS APPLE, 90S BABY (LEFT SIDE OF NECK)',
          'BRASS KNUCKLE TATTOR RIGHT ARM','TATTOO RIGHT SIDE OF FACE','TATTOO ON FACE A TEAR DROP.','TATTOOS WORD JORDAN',
          'TATTOOS (WORDS) ON HANDS AND NECK','CERTIFIED- R ARM','"HARDKNOCK LIFE" ON LEFT ARM','TATTOOS ON RIGHTSIDE OF NECK',
          'TATTOO ON LEFT ARM','FACIAL HAIR. TATTOO ON FINGERS AND HAND','SLEEVE','"MELISSA" TATTOO ON LEFT ARM',
          'HAND TATTOO OF A HEART','SIX STARS ON LEFT ARM','NAME TATTO ON NECK ( MYRA)','CHEST TATTOOS',
          'TATTOOS ON THE NECK','ROSE ON LEFT ARM','TATTOO ON HAND','TATOO ON NECK "BERNARDINE"','TATTOS ON ARM',
          'FISH TATTOO ON NECK, RABBIT WITH GUN ON RIGHT ARM','BLESSED TATTOO OVER HIS LEFT EYE','UNK TATTOOS ON NECK',
          'TATTOO DESIGN ON LEFT SIDE OF NECK','BEARD, TATTOOS, HAT, WHITE HOODIE, BLUE JEANS, BLACK BOOTS',
          'DESIGN ON LEFT HAND','TATTOO ON ARM STATING TRAYVON','HAND TATTOO','TATTOO RIGHT EYE',
          'CHINESE CHARACTER (WRITTEN) ON RIGHT SIDE NECK','HEART ON FOREARM AND SPIDER ON FINGER','TATOO ON RIGHT ARM',
          'TATOO UNDER RIGHT EYE','TATTOOS ON HIS NECK','SOFIA ON HIS RIGHT SIDE OF NECK',
          'WORD ONLY TATTOO ON RIGHT FOREARM " LOYALTY"','TATTTOO: "TIFFANY" ON LEFT SIDE OF NECK.','TATTOO OF A CROSS',
          'TATTOS ON CHEST STARS, WORDS','LEFT ARM "ITALY"','TRIBAL TATTOO ON RIGHT ARM',
          '"BEST FRIENDS" TATTOO(LEFT TRICEPT), LITTLE GIRL WITH RED BALLOON TATTOO(TORSO),','TATTOOS OF TRIANGLES OVER EYES',
          'DREADS, TATTOOS ON ARMS','SLEEVE TATTOO ON RIGHT ARM','RIGHT ARM SLEEVE, CROWN AND MULTIPLE','CROWN TATTOO ON HAND',
          'LA TATTO ON NECK','FUCK THE FEDS TATTOO ON NECK','LEFT LEG TATTOO','ARM SLEEVE','TATTOO UNDER HIS RIGHT EYE',
          'TATTOOS: (NECK- JESUS)','TATTO ON HAND','TATTOO ON NECK "SHARHONDA"','LIP TATTOO ON NECK','TATTOO ON HIS NECK',
          'TATTOO RIGHT ARM: "OHANNIS"','TATTOO OF TEAR DROPS ON CORNER OF RIGHT EYE','TATTOS OF TEAR DROPS UNDER RIGHT EYE',
          'RIGHT ARM FULL SLEEVE TATTOO WITH STARS AND SKULLS','MULTIPLE TATOOS','LEFT ARM "CARLITO"',
          'NECK TATTOO, WORD, RIGHT SIDE','CHEST TATTOO','HAND TATTOOS/ THICK GLASSES','TATTOO RIGHT FOREARM-TRIBAL',
          'LIPS TATTOOED ON NECK','TATTOO ON INNER LIGHT ARM SAYS BLESSED','TATTOOS ON THE FACE','TATTOOS ON ARM (MARQUIS)',
          'TATTOO ON WRIST','HEART ON SHOULDER','BLACK JACKET WITH BLUE JEANS.  FLAMES TATOOED ON BOTH ARMS',
          'SLEEVE TATOO OF ANGEL, ALSO ANCHOR TATTOO ON LEFT HAND','TATTOO RIGHT ARM','TATTOO ON LEFT ARM , RIGHT ELBOW',
          'FACE TATTOO ( RIGHT SIDE)','NECK TATTOO ( FUCK YOU PAY ME)','MOB (ARM)',
          'TATTOO CHEST; SKULL DESIGN. TATTOO LEG; "SAKARA", BLUE SHORTS, SHIRTLESS.','TATOOS ON LEGS','GOOD LIFE',
          'TATTOO ON LEFT FOREARM','TATOOS ON HANDS AND NECK','SAVAGE TATTOO ON ARM','TATOO ON NECK WITH LIPS',
          'HEART TATTO UNDER LEFT EYE','TATTOOS ON NECK "YSL", PICTURE TATTOO ON LEFT ARM','TATTOO ?LESLIE? ON LEFT ARM',
          'TATOO LEFT HAND "PNU"','TRIBAL TATTOO ON LEFT ARM',
          'TATTTOO;PAIN FOR THE AGONY,NEVERBROKEN ON RIGHT ARM, BLACK SHIRT,BLACK JEANS,DRE','TATTOO- 714',
          'RED AND BLACK SHIRT, BLUE SHORTS,TATTOOS OF SUN AND OF A ROSE',
          'UNK TATTOO ON RIGHT ARM, WHITE TEE SHIRT, BLUE SWEAT PANTS',
          '"JANET" ON ARM, TRIBAL ON ARM, BARBER CLIPPER LA ON ARM, UNKNOWN TATOO ON TORSO','5 POINT STAR ON ARM',
          'BLACK HAT, BLACK JACKET, BLUE JEANS, GOLD SNEAKERS, TATTOO OF PYRAMID ON ARM',
          'TRIBAL SYMBOLS(ARM), 3 SKULLS W FLAMES(ARM), DEDICATED BROOKLYN(ARM),CROSS WITH','NECK TATTOO UNDER LEFT EAR',
          'BOTH FOREARMS TATTOOES','RIGHT FORARM TATTOO','TATTOOS ON ARMS 1)"RIP" WITH PRAYING HANDS 2)CROSS WITH WINGS',
          'TATOOS ON FOREARM','TATTOOS ON HANDS AND NECK','NUMEROUS TATTOOS ON BOTH ARMS','TATOO ON SHOULDER',
          'SMALL P TATTOO UNDER EYE','STAR TATTOO ON FACE,GREEN JACKET, BLACK HOODIE','7 TATOOS',
          'TATTO LEFT UPPER ARM, HEART WITH WORDS','2 STARS ON RIGHT ARM','TATOOS ON LEFT ARM',
          'TATTOOS ON ARMS AND CHEST, WEARING BLACK WIFE BEATER BLACK HAT BLUE JEANS','TATTO ON CROWN OF HEAD',
          'ANTHONY RIGHT ARM/ WORDS LEFT ARM BEADS BOTH ARMS','TATTOO ON ARM OF A ROSE','LEFT FOREARM - LIFE GOES ON',
          'TATTOO OF TEAR DROP UNDER LEFT EYE','UNK/UNIDENTIFIED TATTOOS',
          'TATOO OF BULLDOG ON RIGHT ARM // BROWN JACKET, BLACK SWAETPANTS, BLUE DUFFLE BAG',
          'TATTOO: PICTURE OF GOD\'S HANDS ON TORSO AND THE WORD "DEREK" ON TORSO','OWL TATTOO','ARMS HANDS',
          'TATTOO OF A CROSS ON RIGHT LEG','TATTOO ON THE ARM: NENA. TATTOO ON THE LEG: PICTURE OF THE FLOWER',
          'TATTOOS ON ARMS.','PICTURE TATTOO ON LEFT SIDE OF NECK','TATTOO SLEEVE','TRIANGLE ON FACE',
          'STAR, NAKED WOMEN ON ARM. X ON FINGER','NECK TATTOO (BOMB)','3 DIAMONDS BELOW LEFT EYE',
          'TATTO ON UPPER LEFT EYEBROW, AND RIGHT HAND PICTURES/WORDS','TATTOO ON LEFT HAND, POSSIBLE FLOWER',
          'TATTOOS, BLUE DURAG','VIRGINIA TATTOO ON HIS RIGHT ARM.','CLOWN TATTOO ON LEFT BICEP (ARM)',
          'MULTIPLE TATOOS ON NECK','TATTOO ON FACE UNDER LEFT EYE, WHITE T SHIRT, BLACK JEANS','T-ROY, NECK',
          'ONE TATTOO ON NECK','BLK/SHIRT; TAN/CARGOSHORTS; RIGHTANDLEFT ARM TATTOOS','TATTOO OF SCALE ON LEFT SIDE OF NECK',
          'TATTOO-ARM PICTURE, BACK OF NECK WORDS','LEFT FOREARM, WORD','FLOWER ON RIGHT HAND',
          'SHIRTLESS MALE GREEN HAIR THIN BUILT TATOOS ON ARMS AND SHOULDERS','TATOO- MERCEDEES','"BBE" TATTOOED TO LEFT HAND',
          'HEART TATTOO ON LEFT CHEEK UNDER EYE, PURPLE SWEATSHIRT, PINK BELT, JEANS, BLUE','UNKNOWN TATTOO',
          'TBO TATTOO RIGHT ARM, 700 TATTOO LEFT ARM','HAND TATTOO - BUGZ','TATOOED HAIR','ARM SLEEVES','PICTURES ON ARMS',
          'VALENTIN (HAND); DREADS; LIGHT COLORED TANK TOP','ROC (ON ARM) , LIVE FAST DIE YOUNG (ON CHEST)',
          'TATTOO ON LEFT FOREARM, LATIN PRIDE','TATTOO OF SKULLS AND ROSARY BEADS ON ARM','1 PICTURE TATOO ON EACH ARM',
          'TATTOOS ON RIGHT AND LEFT HAND (WORDS)','FACE TATTOO','SEXY YVETTE','TATTOO OF FLAG (RIGHT FOREARM)',
          'TATTOO ON ARM "MONEY ON MY MIND", MARIJUANA LEAF ON ARM, "LOVE" ON HAND, UNKNOWN','TATTOOS ON RIGHT FOREARM',
          'UPPER LEFT CHEST TATTOO','TATTOS ON NECK AND FACE (WORDS)','RAMIAH TATTOO ON FOREARM','MOOMOO(ARM) & KIMORA (ARM)',
          'TATTOS ON BOTH ARMS','RIGHT LEG CALF TATTOO OF FACE','XAVIER TATTOO ON LEFT SIDE OF NECK',
          'TATTOOS ON CHEST, LIGHT BLUE JEANS, BLUE SNAKERS, BLACK BOXERS, NO SHIRT','FEATHERS ON CHEST',
          '"DANIEL" TATTOO ON RIGHT SIDE OF NECK','TATTOO "CYNTHIA" ARM','"RIP MICHELLE" WITH PRAYING HANDS',
          'YELLOW HAT, WHITE T-SHIRT, BLUE SHORTS, TATTOO SLEEVES ON BOTH ARMS, WHITE CHARG','UNK TATTO ON LEFT ARM',
          'TATTOOS RIGHT ARM','TATTOO ON LEFT UPPER ARM','MADE IN INDIA ON RIGHT SHOULDER ( WORDS ONLY)',
          'GAMEOVER TATTOO ON KNUCKLES','NY METS LOGO, FAMILY (RIGHT ARM)/ FIRST, RAM (LEFT ARM)',
          'TATTOO RIGHT FOREARM "CARMEN"','COY FISH TATTOOS','TATTOOS ON HAND','TATTOO ABOVE LEFT EYE.',
          'SPIDER TATTOOS ON SHOUDLERS, PRAYING HAND TATTOOS ON ARM AND THE NAME "MOISES" O',
          'GRIM REAPER TATTOO ON ARM, FEAR TATTOO ON ARM','MULTIPLE TATTOOS ON FACE AND NECK','TATTOOS ON KEFT ARM "3RD BLOCK"',
          'TATTOO PUZZLE PIECE WITH AN "R".','TATTOO ON RIGHT FOREARM "BORN ALONE"','TATTOOS RIGHT SHOULDER',
          'TATTOO "G" R/T SHOULDER; TATTOO "C" L/T SHOULDER','"THE KING" TATTOO ON NECK WITH PICTURE OF LIPS ABOVE WORDS',
          '007 LEFT ARM, TBO RIGHT ARM','ROSE TATTOO ON FACE, AND EYE TATTOO ON THROAT/NECK',
          'RIGHT FOREARM SKULL WITH CINDY ABOVE','OWL TATTOO ON ADAM APPLE','TATTOO OF THE LOWER CASE T IN BETWEEN EYEBROWS',
          'TATTOOS ON LEFT AND RIGHT SIDE OF FACE','TATOO ON NECK, BLACK HOODED SWEAT SHIRT','ANCHOR ON HAND',
          'TATTOO OF ARROW ON UPPER RIGHT ARM','TATTOO ON NECK, TATTOO ON FACE, TATTOO ON BOTH HANDS, SCAR ON LEFT HAND',
          'TATTOOS RIGHT HAND: AMERICAN LEFT HAND: GANGSTER','TATTOS- 86 ARM, LUIS ARM, 86 HAND, LOUIE HAND',
          'JORDAN JUMP MAN ON NECK','JESUS PICTURE ON UPPER LEFT ARM','BLACK SHIRT. TATTO OF A KINGS CROWN ON HIS NECK',
          'TATTOO: ARM - TRIBAL WOLF','FEATHER TATTOO ON HIS LEFT ARM','SKULL ON HIS RIGHT ARM',
          'LEAF TATTOO ON LEFT SIDE OF NECK. MULTIPLE TATTOOS ON ARMS','TATOOS ON ARM SAYS "BREEZE"',
          '"TRUST" ON LEFT WRIST, "RESPECT" ON RIGHT WRIST','TATTOOS (ANGEL) BLACK BROOKLYN BASEBALL CAP',
          'TATTOO OF CHINESE LETTERING ON ARM/ SCAR ON FACE.','TATTOO ON LEFT ARM "ALEX"','TATTOO RIGHT SIDE OF NECK',
          'TATTOOS ABOUT THE BODY','BARBED WIRE ( RIGHT ARM ) MOTHER\'S NAME ( NECK )','STARS ON LEFTSIDE OF FACE',
          'LIPS ON RIGHT SIDE OF NECK AND THREE STARS ON LEFT SIDE OF NECK','EST. 1996','TATTOOS ON ARMS (CLOUDS)',
          'TATTOO WITH WORDS 1 BILLION ON RIGHT EYEBROW','TATTOO-MARIA-NECK','SNAKE FACE ON ARM, PETER CHAU ON ARM',
          'TATTOOS HANDS','TATTOO ON RIGHT THUMB OF ASIAN SYMBOL','GOLD TEETH/TATOOS','TATTOOS/ARM',
          'TATTOO ON RIGHT SIDE OF NECK','TATTOOSON FACE AND NECK','SLEEVES ON ARM AND NECK PICTURES','TATTOO ARM',
          'WORD TATOO ON LEFT FOREARM','MARIJUANA PLANT ON NECK','NO T SHIRT RED TATTOO ON STOMACH',
          'FOREARM TATTOO, STOMACHE TATTOO','CHINESE SYMBOL TATTOO - NECK','TATOO ON LEFT ARM SHULL SMOKING A CIGAR',
          'RED KISS TATTOO IN THE NECK, LEFT SIDE.','LIPS ON LEFT SIDE OF NECK','"ANGEL" ON LEFT ARM',
          'TATTOOS ON SIDE OF FACE. DARK JACKET, BLACK SWEATS, GREEN HAT BACKWARDS.','LEFT ARM','RIGHT ARM /LEFT ARM',
          'SKULL WITH WINGS AND SWORD ON RIGHT UPPER ARM.','TATTOO ON RIGHT HAND CANT DESCRIBE.','TATTOOS ON CHEST',
          'AW (TATTOOS WORDS ONLY) LEFT SIDE OF FACE','FACE ON CHEST','ROMAN NUMERALS ON LEFT WRIST','TATTOO ON LIP',
          'HEART TATTOO ON LEFT SIDE AREA OF NECK.','MONEY SIGN TATTOO ON LEFT EYE','FOREARM TATTOO','ARM TATTOO',
          'TATTOOS, TAN PANTS, BEARD, GREY COAT, HAT','TATTO ON FACE','STAR ON HIS NECK','PICTURE TATTOO ON LEFT HAND',
          'TATTOO ON RIGHT ARM "RIP ALMA"','TATTOOS ON BOTH ARMS OF WORDS','BLACK JKT, HAND TATTO OF CROWN','"RIP ALMA" ON ARM',
          'YING YANG ON RIGHT ARM','ONLY GOD CAN JUDGE ME(CHEST TATTOO LETTERINGS), BLUE JACKET, BLUE JEANS, BLK SNE',
          'SCRIPTURE TATTOO TOP RIGHT ARM','NECK LEFT SIDE','MUSIC NOTE TATTOO AND STAR TATTOO ON OPPOSING SIDES OF NECK',
          '"JESTER" TATTOO','TEAR DROP NEAR EYE','TATTOO ON ARM DAMIAN','?TIKI?W/CROWN AND CROWN ?125? ON LEFT NECK',
          'TATTOO ON LEFT ARM (UNK WORDS)','SUPERMAN TATTOO ON NECK','SPIDER WEB TATTOOS ON ARM','TEAR DROP','THUG LIFE',
          'TATTOO ON STOMACH','SEMI COLON ON WRIST','TEAR DROP BELOW THE EYE','CROWN TATTOO ON NECK','TATTOS RIGHT ARM',
          'TEAR DROP TATTOO UNDER EYE','TATTOO LEFT ARM','TATTOOS SEVERAL, RIGHT LEG "FUCK YOU"','TATTOO WORDS ELI ON LEFT ARM',
          'BLACK T-SHIRT, BLUE JEANS, NEON YELLOW SNEAKERS, BEARD, TATTOO OF A SCROLL ON RI','TATTOO RIGHT ARM "PRINCESS"',
          'TATTOOS: "CYNTHIA" (WORDS ONLY), HEADPHONES (IMAGE)','TATTOOS ON FOREARM','VARIOUS TATTOOS ON HIS BODY',
          'TATTOO RIGHT "DJ LION"','TEAR DROP TATTOO UNDER RIGHT EYE','MULTIPLE TATTOOS AND SCARS','N$4L (ARM), OFS (ARM)',
          'RALPH LAUREN TATTOO ON LEFT SIDE OF FACE','HAND TATTOOS','ARM TATTOOS','FLOWER ON LEFT ARM','TATTOOS FOREARM',
          'TATTOO ON LEFT FOREARM SAYING JOE AND A GRAVESTONE WITH TWO ANGELS ON LEFT UPPER','ZEBA 06/20/1972',
          'TATTOOS ON FACE AND GREEN JACKET WITH THE NUMBER 6 ON THE BACK.','STAR TATTOO ON NECK','TATTOO LEFT BICEP',
          'STARS TATTOO ON THE BACK NECK.','TEAR DROP ON R/S CHEEK','TWO TEAR DROPS ON RIGHT EYE','TATTOO ON LEFT ARM.',
          'TATTOO LEFT HAND (PATRICIA)','TATTOO #95 ON RIGHT ARM, BLACK S/S SHIRT /BLACK SWEAT PANTS WHITE STRIPE',
          'WORD TATTOO "LOYALTY" ON RIGHT HAND','PRINCE TATTOOED ON RIGHT HAND','TATTOO ON FACE, STUDDED BELT, DURAG',
          'TATTOO ON LEFT SIDE OF NECK DOUBLE T SHIRTS','TATTOOS ARM/SHOULDER','THREE STAR OUTLINES ON LEFT SIDE OF FACE',
          'CROSS (ARM) JUS(ARM), JS (ARM), LEFT ARM- JAY JAY ARM,','ARM TATTOO "AMAYA"','BLACK CLOTHING, TATTOO OF CROWN',
          'DEVIL TATTOO ON RIGHT ARM','TATOOS ON ARM','TATTO ON LEFT THUMB ASIAN WRITING','TATTOO UNDER RIGHT EYE',
          'TEAR DROP TATOO BY LEFT EYE','WHITE POLO WITH STRIPES; TATOO OF NAOMI ON HAND','TATTOO INSIDE LEFT FOREARM',
          'ARM AND CHEST TATTOOS','LOYALTY ON ARM','SPIDER TATTOO ON RIGHT HAND','WEED LEAF','TATTOO OVER EYELID',
          'ELIJAH RIGHT ARM','TRIBAL TATTOO','TRIBAL TATTOOS IN BOTH FOREARMS','TATOOS ON FOREARMS','TATTOO ON LEGS',
          'TATTOOS ON FACE, ARMS','TATTOO LEFT ARM "SHERI"','CHERRY TATTOO ON R/S OF NECK','SMALL TATTOO ON HAND',
          'TATTOO "N" ON HAND','2 TATTOOS ON HIS NECK','"FEARLESS" ON NECK','TATTOO ON RIGHT FOREARM','TATTOOS ON ARMS',
          'CAMEO HAT, WHITE TANK TOP, CAMEO SHORTS, BLACK SNEAKERS, TATTOO ON SHOULDER','NO ONE (ABOVE LEFT EYEBROW)',
          'NECK TATTOO YANKEES LOGO, WRIST TATOO, KNUCKLE TATTOO, STAR ON RIGHT HAND, ASIAN','HAND TATTOO "KING"',
          'TATTOOS ON FACE AND BOTH ARMS','TATTOO ON LEFT SIDE NECK','TATTOO RIGHT ARM VIGEN DE LA ARTAGRACIA','TATTOS',
          'TATTOO ON LEFT HAND','TATTOO','NECK TATOO','TATTOOS ON RIGHT ARM','TATOOS','TATTOOS ON BOTH ARMS','NECK TATTOO',
          'TATTOOS ON LEFT ARM','TATTOO ON CHEST','TATTOO ON LEFT SIDE OF NECK','TATTOO ON RIGHT ARM','TATTOOS',
          'TATTOO ON NECK','TATTOO ON ARM','TATTOOS ON HANDS','TATTOOS ON NECK','TATTOOS ON FACE','TATTOOS ON ARM',
          'NECK TATTOOS','TATTOO ON RIGHT HAND','MULTIPLE TATTOOS','TATOO ON NECK','FACE TATTOOS',
         ]                                                                                                                                                                                                                                  

mask = ['BLACK HAT/SKI MASK, GRAY HOODIE SWEATSHIRT','PINK AND GREEN MASK','MASK','BLACK JACKET , SKI MASK , RED BACKPACK',
        'BLK SHOES, GREY SWEATPANTS, BLUE/TAN HOODIE','GREEN MASK / CAMOFLAUGE HOODY / WHITE GLOVES',
        'BLACK DU-RAG, MASK ON HEAD','WEARING A BLACK SKI MASK','BLACK JACKET, GRAY HOODIE SWEATSHIRT, GRAY SKI MASK',
        'FUR COAT PARKA (GRAY) W/ DISTINCTIVE HOOD AFFIXED, BLUE JEANS, BLACK SKI MASK','FACE MASK WITH SMILE',
        'TATTOOS ON HIS ARMS','BLK HAIR, BR EYES, FACIAL HAIR, BL/COLORED HOODIE, BLK FACEMASK, BLK SHOES, BLK',
        'BLACK HOODIE, BLACK SWEATPANTS, BLACK SKI MASK, BLACK SNEAKERS, BEARD','ARM','HORSE MASK ON PEDICAB, SUNGLASSES',
        'TATTOO ON NECK, SHORTS NAVY BLUE , NAVY BLUE HOODIE, WHITE SNEAKERS','FACE MASK','RIGHT ARM TATTOOS',
        'BLK SKI MASK, BLK JACKET, GREY HOODIE, BLK JEANS, BLK BOOTS, BLK BOOKBAG','TATOO ON TORSO',
        'TATTOOS - JAPANESE KANJI (REDEMPTION) RT ARM, BARBED WIRE WITH "NEW YORK" RT ARM',
        'RED SKI MASK, BLACK WAIST LENGTH JACKET, BLUE JEANS','BLACK SKI MASK','SKI MASK','MASK AND GLOVES',
        'BLACK SKI MASK, WHITE HOODIE, BLACK JEANS, BLACK SNEAKERS','BLACK GLOVES & SKI MASK',
        'FACIAL HAIR, DREAD LOCKS, TAN BOOTS, GREY SWEATPANTS, RED SHIRT (BLK SKI-MASK, B','LONG BRAIDS',
        'BLK SHOES, GREY SWEATPANTS, BLK HOODIE, BLK HOODED JACKET, GREY SKI-MASK, FACIAL','BLUE HOODED SWEATSHIRT',        
       ]

piercing = ['EYEBROW PIECING, BLUE JEANS, BLACK JACKET','FACE PIERCING, SLICK BACK BLACK HAIR','EARING ON EYE BROW',
            'TWO EARRINGS','PIERCED EARS','BLUE T-SHIRT, TAN SHORTS, WHITE SNEAKERS, EARINGS','EARINGS IN BOTH EARS',
            'LIP PIERCINGS','FACIAL PIERCINGS','EARINGS, BLACK LEATHER JACKET, BLUE JEANS','NOSE PIERCING',
           ]
                                                                       
unusual = ['RED/WHT SANTA HAT, WHT PANTS/ BLK JACKET','WEARING CHRISTMAS CLOTHS','BROKEN ARM (CAST)','CROOKED TEETH IN FRONT',
           'WHITE RELIGOUS HEAD COVERING. WHITE ROBE COVERING WHOLE BODY','REFUSE','NICKNAME FREAK',
           'ROTTING FRONT TEETH','RED TUTU','PUERTO RICAN FLAG','BLUE WIG','TAN COAT, GOLD TEETH',
           'BLACK PLASTIC BAG AS RAIN JACKET, BLUE LONG SLEEVE SHIRT, BLUE JEANS, AND BLACK','LONG HAIR, MISSING TEETH',
           'JUJU','PREGNANT','NO FRONT TEETH','BLACK PAJAMA WITH CANDY CANE DESIGNS','EYE PATCH',
           'BLACK JACKET, BLACK JEANS, MOHAWK, YELLOW AND BLACK SNEAKERS','TEETH CROOKED','VERY BAGGY CLOTHING',
           'WHITE T SHIRT, CAMO BOTTOMS, WHITE SNEAKERS, VISIBLY SWEATING','MISSING FRONT TOOTH','SUNKEN IN FACE',
           'BULGIE EYES','THREE FINGERS ON RIGHT HAND','MISSING LEFT HAND','DEFORMED RIGHT HAND','TANK TOP IN COLD WEATHER',
           'HOSPITAL BRACELET','LOTS OF JEWELRY','LAZY EYE','CLOWN COSTUME','LEFT EYE','MISSING TEETH','GOLD TEETH',
           'BLACK SWEATER, MULTIPLE GOLD CHAINS, GOLD WATCH','NO SHIRT, BLACK PANTS, DIRTY HAT',
           'NAVY BLUE T-SHIRT, NO SHOES, SHORTS ARMY FATIGUES','BOXER SHORTS','WHITE SHIRT, TWIST IN HAIR, 1 SHOE ON',
           'BLACK SHORTS, NO SHIRT','SHIRTLESS, SHORTS','SHIRTLESS','BOXER BRIEFS','NO SHOES','NO SHIRT',
          ]             

menacing = ['WEARING A WHITE SHIRT, YELLOW PANTS, WHITE SNEAKERS, AND HOLDING A KNIFE','BULLET','FIREARM ON CHEST',
            'HOCKEY STICK, GRAY SNEAKERS','DARK PANTS, GREY SNEAKERS, TAN COAT, BLUE HAT, HOLDING CRICKET BAT',
            'RED JACKET WITH KNOWN GANG AFFILIATION BEADS AROUND NECK','3 DOTS ON HAND. MS-13 ON TORSO'
           ]

bandana = ['YELLOW BANDANA', 'GREEN JACKET, RED HOODIE, MULTICOLORED BANDANA, DARK JEANS','BLUE BANDANA AROUND NECK',
          'RED HOODED SWEAT SHIRT','GRAY HOODIE, GRAY SWEATPANTS, BLACK BANDANA','RED BANDANA AROUND NECK',
          'BLUE TSHIRT, GREY SWEATPANTS AND SILVER BANDANA','BLACK SHIRT, BLUE JEANS, BLACK SNEAKERS, BLUE BANDANA',
          'BLACK PANTS, BLUE BANDANA OUT BACK LEFT POCKET, BLUE SNEAKERS','BLUE BANDANA ON HEAD',
           'RED/BLACK SCARF, BLACK BANDANA','WHITE BANDANA, BLACK SKULL CAP','BANDANA ON HEAD','RED BANDANA, SHIRTLESS',
           'CAMO BANDANA, BRAIDS, GREEN SHIRT, CAMO PANTS, BEIGE SNEAKERS','BANDANA, BLUE SHIRT, JEANS',
           'GREY HOODIE, DARK JEANS, RED BANDANA','RED BANDANA','BLUE BANDANA',
           'BLACK HAT, RED BANDANA, BLACK JACKET, BLACK JEANS, BROWN COWBOY BOOTS & BLACK BA',
          ]

wounds = ['WHITE T-SHIRT (BLOOD ON IT) GRAY SHORTS','WHITE T SHIRT WITH BLOOD STAINS, BASKETBALL SHORTS',
          'LONG BEARD, GREY JACKET, LONG HAIR, BLUE JEANS, BLUE SNEAKERS, BLOODY RIGHT FIST','BEARD, BLOODY TANK TOP',
          'BRUISING AROUND LEFT AND RIGHT EYES AND SCRATCHES ON HIS NECK','SCRATCH ON FACE','BLEEDING LIP',
          'LEFT HAND BRUISE, SCAR ON ARM','BLUE SHIRT, BLUE JEANS, BLUE SNEAKERS, SWEATY, BLOOD ON HAND',
          'SHIRT/SHORTS COVERED IN BLOOD','BLACK WAIST LENGTH JACKET, BLUE JEANS, LEFT EYE SOCKET BRUISE (BLACK EYE)',
          'LUMP TO LEFT CHEEKBONE','BRUISING AND LACERATIONS TO FACE, TORN GARMENTS','CUT ON FACE','BURN LEFT ARM',
          'SLASH RIGHT CHEEK','BANDAGE LEFT ARM'
         ]

In [30]:
# for norm in normal:
#     appearance_db['appearance_normal'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda norm: 1 if norm in normal else 0)
    
for ba in bag:
    appearance_db['appearance_bag'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda ba: 1 if ba in bag else 0)
    
for hair in facial_hair:
    appearance_db['appearance_facial_hair'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda hair: 1 if hair in facial_hair else 0)
    
for scar in scars:
    appearance_db['appearance_scars'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda scar: 1 if scar in scars else 0)
    
for braid in braids:
    appearance_db['appearance_braids'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda braid: 1 if braid in braids else 0)
    
for hood in hoodie:
    appearance_db['appearance_hoodie'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda hood: 1 if hood in hoodie else 0)
    
for ha in hat:
    appearance_db['appearance_hat'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda ha: 1 if ha in hat else 0)
    
for rag in du_rag:
    appearance_db['appearance_du_rag'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda rag: 1 if rag in du_rag else 0)
    
for tat in tattoo:
    appearance_db['appearance_tattoo'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda tat: 1 if tat in tattoo else 0)
    
for mas in mask:
    appearance_db['appearance_mask'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda mas: 1 if mas in mask else 0)
    
for pierce in piercing:
    appearance_db['appearance_piercing'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda pierce: 1 if pierce in piercing else 0)
    
for unus in unusual:
    appearance_db['appearance_unusual'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda unus: 1 if unus in unusual else 0)
    
for men in menacing:
    appearance_db['appearance_menacing'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda men: 1 if men in menacing else 0)
    
for band in bandana:
    appearance_db['appearance_bandana'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda band: 1 if band in bandana else 0)
    
for wound in wounds:
    appearance_db['appearance_wounds'] = appearance_db.SUSPECT_OTHER_DESCRIPTION.apply(lambda wound: 1 if wound in wounds else 0)

In [31]:
data = pd.concat([data, appearance_db], axis=1)

## Drop more columns

In [33]:
# Previously cleaned or made dummies out of. No longer needed.
data = data.drop(['SUSPECTED_CRIME_DESCRIPTION','FIREARM_FLAG','KNIFE_CUTTER_FLAG','OTHER_WEAPON_FLAG','WEAPON_FOUND_FLAG',
            'PHYSICAL_FORCE_CEW_FLAG','PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG','PHYSICAL_FORCE_OC_SPRAY_USED_FLAG',
            'PHYSICAL_FORCE_OTHER_FLAG','PHYSICAL_FORCE_WEAPON_IMPACT_FLAG', 'STOP_DURATION_MINUTES',
                 'DEMEANOR_OF_PERSON_STOPPED','SUSPECT_OTHER_DESCRIPTION',
                  'STOP_LOCATION_BORO_NAME', ], axis=1)

In [34]:
# Taking stock of features that have already been modified.
cleaned = ['OFFICER_EXPLAINED_STOP_FLAG','OTHER_PERSON_STOPPED_FLAG','OFFICER_IN_UNIFORM_FLAG','FRISKED_FLAG',
           'SEARCHED_FLAG','OTHER_CONTRABAND_FLAG','ID_CARD_IDENTIFIES_OFFICER_FLAG',
           'SHIELD_IDENTIFIES_OFFICER_FLAG','VERBAL_IDENTIFIES_OFFICER_FLAG','BACKGROUND_VIOLENT_CRIME',
           'BACKGROUND_SUSPECT_KNOWN_CARRY_WPN','SUSPECTS_ACTIONS_CASING_FLAG',
           'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG','SUSPECTS_ACTIONS_DECRIPTION_FLAG',
           'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG','SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG',
           'SUSPECTS_ACTIONS_LOOKOUT_FLAG', 'SUSPECTS_ACTIONS_OTHER_FLAG','SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG',
           'SEARCH_BASIS_ADMISSION_FLAG', 'SEARCH_BASIS_CONSENT_FLAG','SEARCH_BASIS_HARD_OBJECT_FLAG',
           'SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG', 'SEARCH_BASIS_OTHER_FLAG','SEARCH_BASIS_OUTLINE_FLAG'
          ]

# Taking stock of features that have been created as consolidations of pre-existing features.
consolidated_and_dummied = ['weapon','endangerment','physical_harm','theft','property_damage','trespass','auto',
                            'sexual_assault','drugs','robbery','other','prostitution',
                           'aggressive_force', ]

## Columns left to dummy

In [35]:
columns_for_dummies = ['MONTH2', 'DAY2','STOP_WAS_INITIATED','ISSUING_OFFICER_RANK','SUPERVISING_OFFICER_RANK',
                       'JURISDICTION_CODE','stop_duration_group','SUSPECT_BODY_BUILD_TYPE','SUSPECT_EYE_COLOR','SUSPECT_HAIR_COLOR',
                       'stop_time_period','SUSPECT_SEX','STOP_LOCATION_PATROL_BORO_NAME',
                       'SUSPECT_RACE_DESCRIPTION','suspect_height_group', 'suspect_weight_group', 'suspect_age_group'
                      ]

def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

for col in columns_for_dummies:
    data = create_dummies(data, col)
    
data = data.drop(['MONTH2', 'DAY2','STOP_WAS_INITIATED','ISSUING_OFFICER_RANK','SUPERVISING_OFFICER_RANK',
                  'JURISDICTION_CODE','stop_duration_group','SUSPECT_BODY_BUILD_TYPE','SUSPECT_EYE_COLOR','SUSPECT_HAIR_COLOR',
                  'stop_time_period','SUSPECT_SEX',
                  'SUSPECT_RACE_DESCRIPTION','suspect_height_group', 'suspect_weight_group', 'suspect_age_group'], axis=1)

# PICKLING FILE

In [36]:
with open("../data/NYPD_EDA.pkl", 'wb') as NYPD_EDA:
    pickle.dump(data, NYPD_EDA)